# # Notebook Initialization

In [ ]:
check the information of the Notebook
import sys
print(sys.version)
!pip list > ./pip_installed_packages.txt
!conda list > ./conda_installed_packages.txt

# Set up the Environment and read Functions

In [ ]:
import importlib.util
import subprocess

def install_packages(packages):
    """Check and install required packages."""
    for package in packages:
        try:
            spec = importlib.util.find_spec(package)
            if spec is None:
                print(f"{package} is not installed. Installing...")
                subprocess.check_call(["pip", "install", package])
            else:
                print(f"{package} is already installed.")
        except Exception as e:
            print(f"Error while installing {package}: {e}")

# List of packages to check/install
required_packages = [
    "geopandas", "pgeocode", "geopy", "flexitext", "umap-learn","lifelines",
    "datashader", "bokeh", "holoviews", "scikit-image", "colorcet", "dask[complete]"
]

# Call the function to install packages if needed
install_packages(required_packages)

# Main Imports
import numpy as np
from scipy.interpolate import make_interp_spline
import pandas as pd
import os
import scipy
from scipy import stats
import statsmodels.stats.multicomp as mc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hail as hl
from hail.plot import show, output_notebook
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer, OneHotEncoder
from sklearn.cluster import KMeans
from tabulate import tabulate
import ast
from scipy.stats import ttest_1samp
import matplotlib.cm as cm
from collections import defaultdict
import statsmodels.formula.api as smf
from scipy.stats import mannwhitneyu
import itertools
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import f_oneway
from scipy.stats import skew, kurtosis
from lifelines import KaplanMeierFitter
import matplotlib.lines as mlines
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_1samp, wilcoxon
from statsmodels.stats.proportion import proportions_ztest


# create some fonctions new and old

In [ ]:
def save(folder_name,filename,file):
    # Replace df with THE NAME OF YOUR DATAFRAME
    my_dataframe = file
    destination_filename = '{}/{}'.format(folder_name,filename)
    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe.to_csv(destination_filename, index=False)
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    # copy csv file to the bucket
    args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
    output = subprocess.run(args, capture_output=True)
    # print output from gsutil
    output.stderr
def read_tsv(file_name,path='/'):
    # Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
    if path == '/':
        name_of_file_in_bucket = '{}'.format(file_name)
    else:
        name_of_file_in_bucket = '{}/{}'.format(path,file_name)
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    # copy csv file from the bucket to the current working space
    os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")
    print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
    # save dataframe in a csv file in the same workspace as the notebook
    file = pd.read_csv(name_of_file_in_bucket,sep='\t')
    return file
def read_csv(file_name,path='/'):
    # Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
    if path == '/':
        name_of_file_in_bucket = '{}'.format(file_name)
    else:
        name_of_file_in_bucket = '{}/{}'.format(path,file_name)
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    # copy csv file from the bucket to the current working space
    os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")
    print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
    # save dataframe in a csv file in the same workspace as the notebook
    file = pd.read_csv(name_of_file_in_bucket,sep=',')
    return file
def initialize_hail_environment():
    """
    Initialize the Hail environment and return the bucket variable and genomic location.

    Returns:
    str: Bucket variable for storing Hail data.
    str: Genomic location.
    """
    # Initialize Hail
    hl.init(default_reference="GRCh38")
    
    # Retrieve bucket variable and genomic location
    bucket = os.getenv('WORKSPACE_BUCKET')
    genomic_location = os.getenv("CDR_STORAGE_PATH")
    
    return bucket, genomic_location
def read_person_information():
    """
    Reads person information from a given file and filters it to keep only data with ICD10 and ICD9 codes.

    
    Returns:
    pd.DataFrame: DataFrame containing filtered person information.
    str: Message indicating the action taken.
    """
    # This query represents dataset "All participants" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
    dataset_25030714_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_concept_id IN  (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                select
                                    cast(cr.id as string) as id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                WHERE
                                    concept_id IN (
                                        133468, 134057, 134736, 135930, 138239, 138525, 140190, 141960, 192438, 193460, 194133, 200174, 200219, 200452, 201618, 201891, 253549, 254068, 254761, 255919, 31610, 316866, 31821, 318800, 320128, 320136, 321588, 37018424, 37018713, 37203927, 37206233, 372887, 37311677, 37311678, 373499, 375252, 376106, 376208, 376337, 4000609, 4000610, 4002898, 4002905, 4006969, 4009890, 4011630, 4020346, 4021667, 4021780, 4021915, 4022201, 4022922, 4022923, 4022924, 4023995, 4024000, 4024013, 4024561, 4024566, 4024567, 4025215, 4027384, 4027553, 4028071, 4028076, 4028387, 4031814, 4038502, 4038677, 4038678, 4041283, 4041284, 4041285, 4041436, 4042056, 4042140, 4042141, 4042142, 4042503, 4042505, 4042835, 4042836, 4042837, 4043346, 4043371, 4043671, 4047779, 40480457, 40481517, 40481841, 40484102, 40484533, 40484935, 4051221, 4054501, 4071689, 4080992, 4081176, 4083787, 4086181, 4090426, 4090614, 4090615, 4091213, 4091363, 4091532, 4093227, 4093228, 4093991, 4094294, 4095793, 4096864, 4100932, 4101212, 4101343, 4101796, 4102111, 4103183, 4103320, 4103352, 4104157, 4113563, 4113999, 4115259, 4115386, 4115390, 4116811, 4117779, 4117930, 4132555, 4132926, 4134294, 4134440, 4150129, 4160062, 4162282, 4168335, 4170143, 4170226, 4170962, 4171379, 4175154, 4178545, 4178680, 4178818, 4179141, 4179167, 4179871, 4179873, 4180154, 4180169, 4180170, 4180628, 4181063, 4181187, 4182161, 4182165, 4182633, 4184252, 4185503, 4185640, 4190185, 4197094, 4198525, 4199402, 4200532, 4201745, 4206591, 4208786, 4212577, 4213101, 4221108, 4227253, 4244662, 4247371, 42538830, 4260918, 4266186, 4267789, 4269314, 4272867, 4274025, 4276569, 4276572, 4277352, 4293175, 4297887, 4301699, 4302537, 4304916, 4305027, 4308811, 4309188, 4316083, 4317258, 4318379, 432250, 432586, 432795, 432867, 4329041, 433128, 433736, 4338120, 4339410, 4339468, 4344497, 43530815, 43530877, 43531053, 43531054, 43531056, 43531057, 43531058, 43531059, 43531060, 435506, 435524, 436096, 436670, 437515, 438112, 440029, 440142, 440371, 440383, 440921, 441542, 442077, 443343, 443723, 443783, 443784, 443883, 444089, 444100, 444108, 444112, 444363, 44782620, 44783587, 44784102, 73553, 75865, 75909, 77074, 77670, 77960, 79106, 80180
                                    ) 
                                    AND full_text LIKE '%_rank1]%'
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1
                            ) 
                            OR  condition_source_concept_id IN  (
                                SELECT
                                    DISTINCT c.concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                JOIN
                                    (
                                        select
                                            cast(cr.id as string) as id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                        WHERE
                                            concept_id IN (
                                                1326492, 1326493, 1326788, 1326896, 1326897, 1326898, 1326899, 1553844, 1567858, 1567893, 1567956, 1567958, 1567959, 1567960, 1567964, 1567965, 1567966, 1567969, 1567971, 1568017, 1568022, 1568023, 1568062, 1568177, 1568178, 1568179, 1568180, 1568217, 1568232, 1568233, 1568330, 1568337, 1568339, 1568344, 1568362, 1568363, 1568364, 1568366, 1568420, 1568421, 1568422, 1569133, 1569134, 1569135, 1569145, 1569447, 1569472, 1569473, 1569488, 1569489, 1569490, 1569491, 1569558, 1569639, 1569650, 1569657, 1569658, 1569659, 1569660, 1569661, 1569662, 1569663, 1569664, 1569665, 1569811, 1570333, 1570557, 1570563, 1570565, 1570566, 1570567, 1570568, 1570569, 1570570, 1570663, 1570671, 1570701, 1570703, 1570704, 1570705, 1570790, 1570791, 1570824, 1570832, 1571057, 1571059, 1571060, 1571061, 1571062, 1571063, 1571064, 1571177, 1571515, 1571516, 1571517, 1572190, 1572191, 1572193, 1572198, 1572201, 1572203, 1572209, 1572226, 1572229, 1572255, 1572256, 1572261, 1572273, 1572274, 1572281, 1576304, 1595534, 1595535, 1595617, 1595618, 1595619, 1595620, 1595697, 1595698, 1595699, 1595700, 1595701, 1595798, 35206751, 35206752, 35206753, 35206754, 35206755, 35206757, 35206852, 35206853, 35206854, 35206855, 35206856, 35206857, 35206858, 35206859, 35206881, 35206882, 35206991, 35206992, 35207021, 35207022, 35207023, 35207024, 35207060, 35207061, 35207062, 35207063, 35207064, 35207065, 35207066, 35207067, 35207092, 35207093, 35207094, 35207095, 35207096, 35207097, 35207098, 35207099, 35207100, 35207150, 35207151, 35207152, 35207153, 35207154, 35207155, 35207169, 35207170, 35207171, 35207172, 35207173, 35207175, 35207176, 35207177, 35207381, 35207409, 35207410, 35207543, 35207566, 35207567, 35207568, 35207668, 35207702, 35207703, 35207704, 35207705, 35207706, 35207922, 35207923, 35207924, 35207928, 35207929, 35207971, 35207972, 35207973, 35208189, 35208190, 35208288, 35208289, 35208290, 35208291, 35208292, 35208293, 35208772, 35208773, 35208774, 35208775, 35208776, 35208810, 35208871, 35208968, 35208969, 35208970, 35208971, 35208974, 35208975, 35209005, 35209006, 35209007, 35209008, 35209009, 35209010, 35209011, 35209013, 35209098, 35209099, 35209324, 35209325, 35209326, 35209327, 35211261, 35211262, 35211263, 35211264, 35211275, 35211276, 35211277, 35211278, 35211279, 35211280, 35211281, 35211282, 35211283, 35211284, 35211285, 35211286, 35211287, 35211288, 35211289, 35211290, 35211299, 35211300, 35211301, 35211302, 35211303, 35211304, 35211305, 35211306, 35211307, 35211308, 35211309, 35211336, 35211337, 35211338, 35211350, 35211388, 35211400, 35211401, 35211402, 35211414, 35211415, 35211416, 35211420, 35211442, 35211443, 35211512, 35211513, 35211514, 35211515, 35211516, 35211517, 35211518, 35211519, 35211520, 35211521, 35211522, 35211523, 35211524, 35211525, 35211527, 35225038, 35225264, 35225374, 35225375, 35225376, 35225378, 35225379, 37200198, 37200199, 37200200, 37200201, 37200202, 37200203, 37200204, 37200205, 37200206, 37200207, 37200208, 37200209, 37200210, 37200211, 37200212, 37200213, 37200214, 37200215, 37200216, 37200217, 37200218, 37200219, 37200220, 37200221, 37200222, 37200223, 37200224, 37200225, 37200227, 37200228, 37200229, 37200230, 37200232, 37200233, 37200234, 37200235, 37200237, 37200238, 37200239, 37200240, 37200242, 37200243, 37200244, 37200245, 37200246, 37200247, 37200248, 37200249, 37200251, 37200252, 37200253, 37200254, 37200312, 37200313, 37200319, 37200320, 37200649, 37200650, 37200651, 37200652, 37200701, 37200702, 37200703, 37200744, 37200877, 37200878, 37201113, 37201116, 37201118, 37201119, 37201120, 37201121, 37202304, 37202305, 44819500, 44819501, 44819502, 44819503, 44819504, 44819524, 44819525, 44819544, 44819554, 44819555, 44819556, 44819777, 44819778, 44819819, 44819946, 44819947, 44819948, 44819964, 44819965, 44819966, 44819967, 44819968, 44819970, 44819973, 44819974, 44819975, 44819978, 44819984, 44819985, 44819986, 44819987, 44820052, 44820053, 44820054, 44820055, 44820056, 44820057, 44820059, 44820060, 44820061, 44820062, 44820063, 44820064, 44820065, 44820066, 44820067, 44820068, 44820069, 44820070, 44820489, 44820682, 44820683, 44820684, 44820685, 44820725, 44820730, 44820731, 44820732, 44820733, 44820889, 44820924, 44820925, 44820926, 44820979, 44820980, 44820981, 44821091, 44821092, 44821101, 44821125, 44821142, 44821143, 44821144, 44821145, 44821146, 44821147, 44821150, 44821151, 44821152, 44821153, 44821154, 44821160, 44821161, 44821166, 44821236, 44821237, 44821238, 44821239, 44821241, 44821242, 44821243, 44821244, 44821245, 44821246, 44821247, 44821248, 44821249, 44821250, 44821251, 44821252, 44821253, 44821254, 44821255, 44821256, 44821557, 44821608, 44821787, 44821824, 44821827, 44821828, 44821949, 44821978, 44821979, 44821987, 44821988, 44822017, 44822018, 44822019, 44822165, 44822166, 44822167, 44822188, 44822189, 44822190, 44822191, 44822208, 44822209, 44822210, 44822211, 44822212, 44822213, 44822216, 44822217, 44822218, 44822219, 44822220, 44822221, 44822227, 44822228, 44822229, 44822235, 44822236, 44822237, 44822289, 44822292, 44822293, 44822294, 44822295, 44822296, 44822297, 44822298, 44822299, 44822300, 44822301, 44822302, 44822303, 44822304, 44822305, 44822306, 44822307, 44822308, 44822309, 44822310, 44822311, 44822312, 44822313, 44822316, 44822668, 44822751, 44822934, 44822935, 44822936, 44822948, 44822953, 44822985, 44822987, 44822988, 44822989, 44823109, 44823144, 44823196, 44823197, 44823333, 44823349, 44823350, 44823360, 44823361, 44823362, 44823367, 44823368, 44823369, 44823370, 44823371, 44823372, 44823373, 44823376, 44823377, 44823378, 44823382, 44823441, 44823442, 44823443, 44823444, 44823445, 44823446, 44823448, 44823449, 44823450, 44823451, 44823452, 44823453, 44823454, 44823455, 44823456, 44823457, 44823458, 44823459, 44823461, 44824071, 44824072, 44824073, 44824074, 44824082, 44824083, 44824092, 44824093, 44824120, 44824121, 44824132, 44824287, 44824288, 44824289, 44824311, 44824312, 44824350, 44824351, 44824352, 44824473, 44824474, 44824475, 44824486, 44824487, 44824510, 44824511, 44824512, 44824513, 44824514, 44824515, 44824519, 44824520, 44824521, 44824522, 44824523, 44824528, 44824536, 44824537, 44824538, 44824540, 44824541, 44824616, 44824617, 44824618, 44824619, 44824620, 44824621, 44824622, 44824623, 44824624, 44824625, 44824626, 44824627, 44824628, 44824629, 44824630, 44824631, 44824632, 44824633, 44824634, 44824635, 44824636, 44825264, 44825280, 44825281, 44825282, 44825298, 44825307, 44825308, 44825499, 44825500, 44825501, 44825551, 44825651, 44825661, 44825677, 44825678, 44825679, 44825680, 44825681, 44825682, 44825696, 44825697, 44825698, 44825699, 44825700, 44825701, 44825702, 44825703, 44825707, 44825708, 44825709, 44825710, 44825711, 44825717, 44825718, 44825719, 44825729, 44825730, 44825731, 44825732, 44825733, 44825734, 44825735, 44825796, 44825801, 44825802, 44825803, 44825804, 44825805, 44825806, 44825807, 44825808, 44825809, 44825810, 44825811, 44825812, 44825813, 44825814, 44825815, 44825816, 44825817, 44825821, 44826155, 44826193, 44826459, 44826460, 44826461, 44826471, 44826503, 44826504, 44826505, 44826506, 44826507, 44826518, 44826519, 44826662, 44826666, 44826678, 44826679, 44826700, 44826701, 44826736, 44826737, 44826848, 44826854, 44826869, 44826870, 44826885, 44826886, 44826887, 44826888, 44826889, 44826890, 44826896, 44826897, 44826902, 44826911, 44826912, 44826913, 44826914, 44826915, 44826969, 44826970, 44826973, 44826974, 44826975, 44826976, 44826977, 44826979, 44826981, 44826983, 44826984, 44827406, 44827615, 44827616, 44827617, 44827629, 44827630, 44827635, 44827661, 44827668, 44827894, 44828003, 44828004, 44828005, 44828019, 44828035, 44828036, 44828037, 44828047, 44828048, 44828049, 44828050, 44828051, 44828052, 44828053, 44828059, 44828060, 44828061, 44828075, 44828076, 44828077, 44828144, 44828145, 44828146, 44828147, 44828148, 44828151, 44828152, 44828153, 44828154, 44828155, 44828156, 44828157, 44828158, 44828159, 44828160, 44828161, 44828162, 44828163, 44828164, 44828165, 44828166, 44828793, 44828794, 44828795, 44828820, 44828833, 44828834, 44828835, 44828841, 44828842, 44829001, 44829012, 44829068, 44829178, 44829179, 44829180, 44829188, 44829195, 44829196, 44829197, 44829213, 44829214, 44829215, 44829216, 44829217, 44829218, 44829219, 44829220, 44829223, 44829225, 44829226, 44829232, 44829233, 44829234, 44829292, 44829293, 44829294, 44829298, 44829299, 44829300, 44829301, 44829302, 44829303, 44829304, 44829305, 44829308, 44829639, 44829878, 44829879, 44829880, 44829881, 44829882, 44829903, 44829926, 44829927, 44829928, 44829936, 44829937, 44829938, 44829939, 44829940, 44829941, 44829942, 44830105, 44830115, 44830137, 44830182, 44830299, 44830310, 44830323, 44830324, 44830325, 44830326, 44830343, 44830345, 44830346, 44830347, 44830348, 44830350, 44830351, 44830352, 44830353, 44830364, 44830365, 44830366, 44830439, 44830440, 44830441, 44830442, 44830443, 44830445, 44830446, 44830447, 44830448, 44830449, 44830450, 44830451, 44830452, 44830453, 44830454, 44830455, 44830457, 44830794, 44830848, 44831045, 44831046, 44831047, 44831059, 44831068, 44831093, 44831094, 44831103, 44831104, 44831105, 44831106, 44831107, 44831108, 44831272, 44831278, 44831279, 44831280, 44831298, 44831299, 44831336, 44831488, 44831489, 44831490, 44831504, 44831505, 44831506, 44831507, 44831508, 44831509, 44831510, 44831512, 44831517, 44831519, 44831524, 44831525, 44831529, 44831583, 44831587, 44831588, 44831589, 44831590, 44831591, 44831592, 44831593, 44831594, 44831595, 44831596, 44831597, 44831598, 44831599, 44831600, 44831601, 44831602, 44831604, 44831605, 44831606, 44832190, 44832191, 44832192, 44832193, 44832194, 44832214, 44832239, 44832240, 44832241, 44832248, 44832249, 44832250, 44832251, 44832423, 44832424, 44832451, 44832602, 44832630, 44832646, 44832647, 44832648, 44832649, 44832650, 44832651, 44832652, 44832653, 44832654, 44832659, 44832663, 44832708, 44832709, 44832710, 44832711, 44832714, 44832715, 44832716, 44832717, 44832718, 44832719, 44832720, 44832722, 44832723, 44833100, 44833101, 44833365, 44833366, 44833367, 44833368, 44833387, 44833394, 44833403, 44833414, 44833415, 44833421, 44833556, 44833611, 44833628, 44833629, 44833630, 44833631, 44833632, 44833670, 44833787, 44833810, 44833811, 44833812, 44833813, 44833831, 44833832, 44833833, 44833834, 44833835, 44833836, 44833837, 44833838, 44833847, 44833848, 44833849, 44833850, 44833851, 44833865, 44833866, 44833912, 44833913, 44833914, 44833917, 44833918, 44833919, 44833920, 44833921, 44833922, 44833923, 44833924, 44833925, 44834342, 44834548, 44834549, 44834563, 44834564, 44834565, 44834569, 44834600, 44834601, 44834602, 44834603, 44834715, 44834764, 44834769, 44834797, 44834841, 44834842, 44834952, 44834953, 44834975, 44834976, 44834977, 44834978, 44834979, 44834980, 44834998, 44834999, 44835000, 44835001, 44835002, 44835003, 44835006, 44835007, 44835008, 44835009, 44835015, 44835022, 44835023, 44835081, 44835082, 44835083, 44835084, 44835085, 44835086, 44835087, 44835088, 44835089, 44835090, 44835093, 44835474, 44835758, 44835769, 44835791, 44835792, 44835974, 44836009, 44836041, 44836042, 44836149, 44836150, 44836159, 44836171, 44836172, 44836173, 44836174, 44836175, 44836189, 44836190, 44836191, 44836192, 44836193, 44836194, 44836195, 44836202, 44836203, 44836204, 44836205, 44836214, 44836215, 44836223, 44836224, 44836225, 44836226, 44836287, 44836288, 44836289, 44836293, 44836294, 44836295, 44836296, 44836297, 44836298, 44836302, 44836303, 44836914, 44836915, 44836916, 44836917, 44836918, 44836929, 44836944, 44836945, 44836970, 44836971, 44836972, 44836973, 44836984, 44836985, 44837129, 44837135, 44837136, 44837309, 44837310, 44837317, 44837335, 44837347, 44837348, 44837349, 44837350, 44837351, 44837352, 44837353, 44837354, 44837358, 44837359, 44837360, 44837361, 44837362, 44837363, 44837369, 44837375, 44837376, 44837423, 44837424, 44837425, 44837430, 44837433, 44837434, 44837435, 44837436, 44837437, 44837438, 44837439, 44837440, 44837441, 44837442, 44837443, 44837444, 44837445, 44837446, 44837449, 44837823, 45533019, 45533020, 45533021, 45533022, 45533023, 45533085, 45533136, 45533137, 45533138, 45533140, 45533147, 45533148, 45533149, 45533277, 45533353, 45533354, 45533439, 45533746, 45533747, 45533748, 45533749, 45533807, 45533808, 45533810, 45533811, 45533812, 45533813, 45533814, 45533815, 45533816, 45533817, 45533818, 45533819, 45533848, 45533849, 45533850, 45533851, 45533852, 45533853, 45533867, 45533868, 45533881, 45533883, 45533884, 45533885, 45533886, 45533887, 45533939, 45533940, 45533941, 45533942, 45534020, 45534021, 45534023, 45534024, 45534025, 45534026, 45534027, 45534130, 45534422, 45534424, 45534427, 45534428, 45534429, 45534430, 45534433, 45534434, 45534435, 45534440, 45534458, 45534479, 45537591, 45537600, 45537698, 45537961, 45537962, 45538046, 45538069, 45538108, 45538109, 45538110, 45538111, 45538112, 45538114, 45538115, 45538121, 45538122, 45538123, 45538143, 45538241, 45538312, 45538313, 45538314, 45538315, 45538373, 45538374, 45538375, 45538377, 45538586, 45538681, 45538682, 45538683, 45538687, 45538688, 45538689, 45538735, 45538737, 45538738, 45538739, 45538740, 45538742, 45538743, 45538744, 45538745, 45538746, 45538747, 45538748, 45538773, 45538774, 45538788, 45538809, 45538814, 45538815, 45538816, 45538817, 45538818, 45538819, 45538866, 45538867, 45538868, 45538944, 45538949, 45538950, 45538951, 45538952, 45539314, 45539371, 45542487, 45542738, 45542780, 45542781, 45542821, 45542887, 45542893, 45542911, 45542912, 45543167, 45543168, 45543269, 45543270, 45543322, 45543324, 45543485, 45543486, 45543487, 45543547, 45543548, 45543549, 45543551, 45543553, 45543554, 45543555, 45543556, 45543557, 45543558, 45543560, 45543561, 45543590, 45543592, 45543593, 45543594, 45543612, 45543613, 45543631, 45543633, 45543634, 45543635, 45543636, 45543689, 45543690, 45543772, 45543773, 45543774, 45543775, 45544130, 45544133, 45544134, 45544169, 45547372, 45547625, 45547626, 45547627, 45547657, 45547704, 45547707, 45547708, 45547732, 45547734, 45547735, 45547743, 45547744, 45547763, 45547840, 45547841, 45547842, 45547843, 45547933, 45547936, 45547937, 45547938, 45548010, 45548011, 45548012, 45548013, 45548116, 45548117, 45548163, 45548321, 45548323, 45548324, 45548325, 45548326, 45548384, 45548385, 45548386, 45548387, 45548388, 45548392, 45548393, 45548394, 45548395, 45548396, 45548397, 45548423, 45548424, 45548443, 45548444, 45548465, 45548466, 45548503, 45548569, 45548570, 45548571, 45548575, 45548576, 45548944, 45548948, 45548951, 45548952, 45548953, 45548954, 45548996, 45552047, 45552385, 45552386, 45552417, 45552453, 45552454, 45552527, 45552528, 45552530, 45552537, 45552538, 45552539, 45552551, 45552638, 45552639, 45552720, 45552721, 45552885, 45552937, 45552938, 45552939, 45552940, 45552941, 45553098, 45553099, 45553148, 45553149, 45553151, 45553152, 45553153, 45553154, 45553155, 45553156, 45553157, 45553158, 45553159, 45553160, 45553161, 45553162, 45553163, 45553189, 45553204, 45553206, 45553224, 45553225, 45553226, 45553227, 45553271, 45553352, 45553353, 45553355, 45553356, 45553357, 45553714, 45553715, 45553716, 45553717, 45553718, 45553719, 45553748, 45556807, 45557112, 45557113, 45557236, 45557237, 45557238, 45557240, 45557376, 45557377, 45557378, 45557379, 45557464, 45557465, 45557538, 45557539, 45557626, 45557661, 45557662, 45557664, 45557807, 45557808, 45557809, 45557850, 45557851, 45557852, 45557853, 45557854, 45557855, 45557856, 45557857, 45557858, 45557859, 45557884, 45557885, 45557886, 45557901, 45557902, 45557903, 45557925, 45557926, 45557927, 45557928, 45557929, 45557992, 45557993, 45557994, 45558062, 45558063, 45558064, 45558066, 45558069, 45558453, 45558454, 45558455, 45558456, 45558459, 45558483, 45558493, 45561616, 45561949, 45561985, 45561986, 45562025, 45562087, 45562088, 45562093, 45562095, 45562096, 45562109, 45562204, 45562206, 45562207, 45562271, 45562272, 45562273, 45562274, 45562343, 45562344, 45562457, 45562650, 45562651, 45562691, 45562692, 45562693, 45562694, 45562696, 45562697, 45562698, 45562699, 45562700, 45562701, 45562732, 45562733, 45562769, 45562771, 45562772, 45562773, 45562774, 45562775, 45562776, 45562777, 45562778, 45562837, 45562838, 45562839, 45562840, 45562926, 45562927, 45562930, 45562933, 45562934, 45562935, 45562936, 45562937, 45563013, 45563288, 45563290, 45563299, 45563315, 45566384, 45566731, 45566810, 45566874, 45566875, 45566876, 45566884, 45566885, 45566886, 45566887, 45566906, 45566907, 45567013, 45567093, 45567094, 45567095, 45567097, 45567167, 45567168, 45567265, 45567266, 45567315, 45567317, 45567469, 45567472, 45567473, 45567474, 45567524, 45567525, 45567526, 45567528, 45567529, 45567530, 45567531, 45567532, 45567533, 45567534, 45567535, 45567564, 45567578, 45567579, 45567580, 45567603, 45567605, 45567608, 45567610, 45567611, 45567659, 45567660, 45567743, 45567746, 45567748, 45567749, 45567750, 45567751, 45567752, 45567753, 45567754, 45567842, 45568112, 45568114, 45568115, 45568118, 45568146, 45568157, 45571289, 45571314, 45571329, 45571741, 45571742, 45571743, 45571802, 45571803, 45571804, 45571805, 45571810, 45571811, 45571812, 45572168, 45572169, 45572170, 45572171, 45572217, 45572219, 45572374, 45572375, 45572416, 45572417, 45572418, 45572419, 45572420, 45572421, 45572422, 45572423, 45572424, 45572425, 45572426, 45572427, 45572428, 45572429, 45572430, 45572431, 45572432, 45572433, 45572434, 45572435, 45572436, 45572437, 45572438, 45572439, 45572458, 45572459, 45572460, 45572461, 45572471, 45572492, 45572494, 45572495, 45572496, 45572497, 45572498, 45572552, 45572627, 45572630, 45572635, 45572636, 45572637, 45572638, 45572639, 45573007, 45573009, 45573010, 45573011, 45573012, 45573016, 45573032, 45573037, 45576443, 45576480, 45576521, 45576541, 45576545, 45576546, 45576547, 45576590, 45576591, 45576592, 45576593, 45576700, 45576787, 45576788, 45576866, 45576868, 45576951, 45576952, 45576994, 45577165, 45577168, 45577217, 45577219, 45577221, 45577222, 45577223, 45577224, 45577225, 45577226, 45577227, 45577228, 45577229, 45577230, 45577231, 45577232, 45577233, 45577234, 45577235, 45577268, 45577269, 45577306, 45577307, 45577310, 45577313, 45577314, 45577315, 45577358, 45577359, 45577435, 45577437, 45577776, 45577778, 45577781, 45577782, 45577783, 45577786, 45577787, 45577807, 45577808, 45577809, 45577823, 45577828, 45577829, 45581352, 45581353, 45581354, 45581355, 45581435, 45581461, 45581491, 45581492, 45581497, 45581498, 45581499, 45581500, 45581519, 45581626, 45581627, 45581702, 45581703, 45581766, 45581853, 45581860, 45581904, 45582061, 45582062, 45582063, 45582064, 45582108, 45582109, 45582110, 45582111, 45582112, 45582113, 45582114, 45582115, 45582116, 45582117, 45582118, 45582120, 45582121, 45582122, 45582156, 45582157, 45582174, 45582175, 45582176, 45582197, 45582201, 45582202, 45582247, 45582248, 45582249, 45582250, 45582251, 45582316, 45582318, 45582319, 45582694, 45582695, 45582701, 45582718, 45582732, 45582734, 45582735, 45585781, 45585793, 45585857, 45586119, 45586139, 45586140, 45586173, 45586174, 45586219, 45586220, 45586243, 45586289, 45586290, 45586291, 45586292, 45586297, 45586298, 45586299, 45586421, 45586506, 45586572, 45586573, 45586574, 45586575, 45586674, 45586675, 45586725, 45586726, 45586891, 45586892, 45586893, 45586894, 45586936, 45586937, 45586938, 45586939, 45586940, 45586942, 45586943, 45586944, 45586945, 45586946, 45586947, 45586948, 45586975, 45586993, 45587008, 45587013, 45587014, 45587015, 45587016, 45587018, 45587064, 45587065, 45587066, 45587067, 45587068, 45587069, 45587157, 45587160, 45587161, 45587162, 45587163, 45587496, 45587497, 45587523, 45587538, 45590768, 45591027, 45591029, 45591030, 45591031, 45591051, 45591115, 45591116, 45591178, 45591181, 45591185, 45591186, 45591201, 45591297, 45591298, 45591299, 45591300, 45591388, 45591456, 45591458, 45591459, 45591460, 45591461, 45591555, 45591558, 45591559, 45591596, 45591597, 45591598, 45591599, 45591743, 45591744, 45591745, 45591746, 45591747, 45591800, 45591802, 45591804, 45591805, 45591806, 45591807, 45591808, 45591809, 45591810, 45591811, 45591812, 45591846, 45591847, 45591848, 45591851, 45591852, 45591853, 45591885, 45591888, 45591889, 45591890, 45591934, 45591935, 45591936, 45592028, 45592030, 45592034, 45592133, 45592134, 45592135, 45592407, 45592429, 45592434, 45592442, 45592444, 45595569, 45595797, 45595798, 45595799, 45595884, 45595885, 45595940, 45595942, 45595947, 45596049, 45596129, 45596130, 45596131, 45596197, 45596198, 45596199, 45596335, 45596388, 45596495, 45596497, 45596498, 45596546, 45596548, 45596549, 45596550, 45596551, 45596552, 45596553, 45596554, 45596592, 45596593, 45596594, 45596595, 45596630, 45596752, 45596757, 45596762, 45596763, 45596764, 45596766, 45596767, 45596768, 45596769, 45596770, 45596771, 45596864, 45597165, 45597167, 45597168, 45597169, 45597170, 45597171, 45597192, 45597197, 45597199, 45597211, 45597212, 45600374, 45600375, 45600609, 45600641, 45600642, 45600659, 45600719, 45600720, 45600746, 45600747, 45600785, 45600786, 45600793, 45600888, 45600889, 45600958, 45601024, 45601026, 45601027, 45601028, 45601125, 45601133, 45601134, 45601182, 45601354, 45601356, 45601416, 45601418, 45601419, 45601421, 45601422, 45601423, 45601424, 45601425, 45601479, 45601480, 45601495, 45601496, 45601497, 45601498, 45601499, 45601545, 45601546, 45601547, 45601623, 45601625, 45601626, 45601627, 45601629, 45601630, 45601631, 45601633, 45601634, 45601635, 45601727, 45602002, 45602003, 45602004, 45602005, 45602006, 45602007, 45602008, 45602011, 45602032, 45602041, 45605095, 45605401, 45605402, 45605403, 45605404, 45605405, 45605476, 45605543, 45605545, 45605547, 45605556, 45605558, 45605559, 45605560, 45605643, 45605645, 45605784, 45605785, 45605786, 45605787, 45605788, 45605943, 45605944, 45605946, 45606131, 45606132, 45606133, 45606191, 45606192, 45606193, 45606194, 45606195, 45606196, 45606197, 45606198, 45606199, 45606200, 45606201, 45606202, 45606203, 45606204, 45606205, 45606206, 45606227, 45606229, 45606270, 45606272, 45606273, 45606274, 45606276, 45606323, 45606324, 45606411, 45606412, 45606413, 45606414, 45606793, 45606794, 45606795, 45606796, 45606797, 45606799, 45606832, 45606833, 45609901, 713855, 725363, 725364, 725365, 725366, 725378, 725379, 725380, 725388, 725389, 725390, 725458, 725459, 766349, 766412, 766413, 766414, 766424, 766425, 766426, 766427, 766428, 766429, 766430, 766431, 766437
                                            ) 
                                            AND full_text LIKE '%_rank1]%'
                                    ) a 
                                        ON (
                                            c.path LIKE CONCAT('%.',
                                        a.id,
                                        '.%') 
                                        OR c.path LIKE CONCAT('%.',
                                        a.id) 
                                        OR c.path LIKE CONCAT(a.id,
                                        '.%') 
                                        OR c.path = a.id) 
                                    WHERE
                                        is_standard = 0 
                                        AND is_selectable = 1
                                    )
                            )
                        ) c_occurrence 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                            ON v.visit_concept_id = visit.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    person_information_all = pd.read_gbq(
        dataset_25030714_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
    # Keep only the information we want
    person_information = person_information_all[['person_id', 'standard_concept_name', 'condition_start_datetime',
                                               'condition_end_datetime', 'source_concept_name', 'source_concept_code',
                                               'source_vocabulary']]

    # Filter for ICD10CM and ICD9CM source vocabularies
    person_information_all = person_information_all[person_information_all['source_vocabulary'].isin(['ICD10CM', 'ICD9CM'])]

    # Prepare message
    message = "Read the personal information about the patient and kept only data with ICD-10 and ICD-9."

    return person_information_all, message
def read_person_information_df():
    """
    Reads person information from BigQuery and filters it to keep only data with 'Male' and 'Female' sexes.

    Returns:
    pd.DataFrame: DataFrame containing filtered person information.
    str: Message indicating the action taken.
    """
    # This query represents dataset "All participants" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
    dataset_25030714_person_sql = """
        SELECT
            person.person_id,
            person.gender_concept_id,
            p_gender_concept.concept_name as gender,
            person.birth_datetime as date_of_birth,
            person.race_concept_id,
            p_race_concept.concept_name as race,
            person.ethnicity_concept_id,
            p_ethnicity_concept.concept_name as ethnicity,
            person.sex_at_birth_concept_id,
            p_sex_at_birth_concept.concept_name as sex_at_birth
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.person` person
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
                ON person.gender_concept_id = p_gender_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
                ON person.race_concept_id = p_race_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
                ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
                ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id"""

    person_df = pd.read_gbq(
        dataset_25030714_person_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
    # Keep only the information we want
    person_df_all = person_df[['person_id', 'date_of_birth', 'race', 'ethnicity', 'sex_at_birth']]

    # Filter for 'Male' and 'Female' sexes
    person_df_all = person_df_all[person_df_all['sex_at_birth'].isin(['Male', 'Female'])]

    # Prepare message
    message = "Read the personal information about the patient and kept only data with 'Male' and 'Female' sexes."

    return person_df_all, message
def copy_files_to_local(bucket):
    """
    Copies files from Google Cloud Storage to the local directory './data' if they don't already exist.

    Args:
    bucket (str): The Google Cloud Storage bucket where the files are located.

    Returns:
    None
    """
    # Check if the folders exist at the specified path './'
    folders = ['data', 'new_run', 'download']
    for folder in folders:
        folder_path = os.path.join('.', folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Created folder: {folder}")

    # Define the paths for the files
    files = {
        'ancestry_preds.tsv': 'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv',
        'relatedness.tsv': 'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/relatedness.tsv',
        'relatedness_flagged_samples.tsv': 'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/relatedness_flagged_samples.tsv'
    }

    # Copy files using gsutil
    for file_name, file_path in files.items():
        local_file_path = os.path.join('.', 'data', file_name)
        if not os.path.exists(local_file_path):
            gsutil_command = f"gsutil -u $GOOGLE_PROJECT cp {file_path} {bucket}/data/"
            print(f"Copying file {file_name} to {bucket}/data/ ...")
            !{gsutil_command}
            if os.path.exists(local_file_path):
                print(f"File {file_name} copied successfully.")
            else:
                print(f"Failed to copy file {file_name} you have the file there already.")
def read_zip_code_socioeconomic_data():
    """
    Reads zip code socioeconomic data from BigQuery.

    Returns:
    pd.DataFrame: DataFrame containing zip code socioeconomic data.
    str: Message indicating the action taken.
    """
    # This query represents dataset "test2" for domain "zip_code_socioeconomic" and was generated for All of Us Controlled Tier Dataset v7
    dataset_65271084_zip_code_socioeconomic_sql = """
        SELECT
            observation.person_id,
            observation.observation_datetime,
            zip_code.zip3_as_string as zip_code,
            zip_code.fraction_assisted_income as assisted_income,
            zip_code.fraction_high_school_edu as high_school_education,
            zip_code.median_income,
            zip_code.fraction_no_health_ins as no_health_insurance,
            zip_code.fraction_poverty as poverty,
            zip_code.fraction_vacant_housing as vacant_housing,
            zip_code.deprivation_index,
            zip_code.acs as american_community_survey_year 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.zip3_ses_map` zip_code 
        JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
                ON CAST(SUBSTR(observation.value_as_string,
            0,
            STRPOS(observation.value_as_string,
            '*') - 1) AS INT64) = zip_code.zip3 
            AND observation_source_concept_id = 1585250 
            AND observation.value_as_string NOT LIKE 'Res%'"""

    zip_code_socioeconomic_df = pd.read_gbq(
        dataset_65271084_zip_code_socioeconomic_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
    message = "Read zip code socioeconomic data successfully."

    return zip_code_socioeconomic_df, message
def run_analysis():
    """
    Runs the analysis, resets the Person_df and Person_information, and returns the necessary dataframes along with messages.

    Returns:
    pd.DataFrame: DataFrame containing Person_df after resetting.
    pd.DataFrame: DataFrame containing Person_information after resetting.
    pd.Series: Series containing counts of individuals in each race.
    pd.Series: Series containing counts of Hispanic individuals in each race.
    pd.DataFrame: DataFrame containing Population data.
    str: Message indicating the action taken.
    """

    # reset Person_df and Person_information
    person_df = person_df_all.copy()
    person_information = person_information_all.copy()

    # Count the number of individuals in each race
    race_counts = person_df['race'].value_counts()
    race_counts = race_counts.apply(lambda x: '<20' if x < 20 else x)
    race_counts_message = "Number of individuals in each race:\n" + str(race_counts)

    # Count the number of Hispanic individuals in each race
    hispanic_counts = person_df[person_df['ethnicity'] == 'Hispanic or Latino']['race'].value_counts()
    hispanic_counts = hispanic_counts.apply(lambda x: '<20' if x < 20 else x)
    hispanic_counts_message = "Number of Hispanic individuals in each race:\n" + str(hispanic_counts)

    # Prepare Population data
    population = read_tsv('ancestry_preds.tsv')
    population_message = "\nPopulation data loaded successfully."

    # Convert string representation of lists to actual lists
    population['pca_features'] = population['pca_features'].apply(ast.literal_eval)
    population['probabilities'] = population['probabilities'].apply(ast.literal_eval)

    # Extracting PCA features into separate columns
    pca_columns = [f'principal_component_{i}' for i in range(1, 17)]
    population[pca_columns] = pd.DataFrame(population['pca_features'].tolist(), index=population.index)

    # Extracting probabilities into separate columns
    probability_columns = [f'probability_{i}' for i in range(1, 7)]
    population[probability_columns] = pd.DataFrame(population['probabilities'].tolist(), index=population.index)

    # Create a new column 'max_probability' with the maximum probability across columns
    population['max_probability'] = population[probability_columns].max(axis=1)

    max_probability_message = "Maximum probability calculated successfully."

    return person_df, person_information, race_counts, hispanic_counts, population, race_counts_message, hispanic_counts_message, population_message, max_probability_message
def plot_pca_graphs(population):
    """
    Generates PCA scatterplots and a line chart based on ancestry prediction and saves them as images.

    Args:
    population (pd.DataFrame): DataFrame containing population data.

    Returns:
    str: Message indicating the action taken.
    """

    # Scatterplot for 'ancestry_pred_other'
    plt.figure(figsize=(10, 6))
    colors = {'eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'oth': 'gold', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    for ancestry, color in colors.items():
        subset = population[population['ancestry_pred_other'].eq(ancestry)]
        plt.scatter(subset['principal_component_1'], subset['principal_component_2'], label=ancestry, color=color, alpha=0.7, s=3)

    plt.title('PCA Scatterplot - Ancestry_pred_other')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.savefig('./download/PCA_Scatterplot_Ancestry_pred_other.pdf',dpi=300)
    plt.close()

    # Scatterplot for 'ancestry_pred'
    plt.figure(figsize=(10, 6))
    colors = {'eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    for ancestry, color in colors.items():
        subset = population[population['ancestry_pred'].eq(ancestry)]
        plt.scatter(subset['principal_component_1'], subset['principal_component_2'], label=ancestry, color=color, alpha=0.5, s=3)

    plt.title('PCA Scatterplot - Ancestry_pred')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.savefig('./download/PCA_Scatterplot_Ancestry_pred.pdf',dpi=300)
    plt.close()

    # Line Chart
    principal_components = ['principal_component_{}'.format(i) for i in range(1, 17)]
    colors = {'oth':'gold','eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    plt.figure(figsize=(12, 8))
    for ancestry, color in colors.items():
        mean_values = population.loc[population['ancestry_pred_other'] == ancestry, principal_components].median()
        plt.plot(principal_components, mean_values, label=ancestry, color=color, marker='o',alpha=0.5)

    plt.xticks(rotation=90)
    plt.xticks(ticks=range(0, 16), labels=['PCA{}'.format(i) for i in range(1, 17)])
    plt.title('Line Chart - Principal Components by Ancestry Prediction')
    plt.xlabel('Principal Components')
    plt.ylabel('Median Values')
    plt.legend()
    plt.savefig('./download/Line_Chart_Principal_Components_by_Ancestry_Prediction.pdf',dpi=300)
    plt.close()

    return "PCA graphs saved successfully."
def perform_pca(population_data):
    """
    Performs PCA (Principal Component Analysis) on the population data and creates scatterplots and a line chart.

    Args:
    population_data (pd.DataFrame): DataFrame containing population data.

    Returns:
    PCA: PCA object containing the transformed data.
    str: Message indicating the action taken.
    """
    # Selecting features for PCA
    pca_features = population_data.iloc[:, 8:24]

    # Initialize PCA
    pca = PCA(n_components=2)

    # Fit and transform the data
    pca_data = pca.fit_transform(pca_features)

    # Create scatterplot for 'ancestry_pred_other'
    plt.figure(figsize=(10, 6))
    colors = {'eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'oth': 'gold', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    for ancestry, color in colors.items():
        subset = population_data[population_data['ancestry_pred_other'].eq(ancestry)]
        plt.scatter(pca_data[subset.index, 0], pca_data[subset.index, 1], label=ancestry, color=color, alpha=0.7, s=3)

    plt.title('PCA Scatterplot - Ancestry_pred_other')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.savefig('./download/PCA_Scatterplot_Ancestry_pred_other_1.pdf',dpi=300)
    plt.show()

    # Create scatterplot for 'ancestry_pred'
    plt.figure(figsize=(10, 6))
    colors = {'eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    for ancestry, color in colors.items():
        subset = population_data[population_data['ancestry_pred'].eq(ancestry)]
        plt.scatter(pca_data[subset.index, 0], pca_data[subset.index, 1], label=ancestry, color=color, alpha=0.7, s=3)

    plt.title('PCA Scatterplot - Ancestry_pred')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend()
    plt.savefig('./download/PCA_Scatterplot_Ancestry_pred_1.pdf',dpi=300)
    plt.show()

    # Create line chart
    plt.figure(figsize=(12, 8))

    # List of principal components
    principal_components = ['principal_component_{}'.format(i) for i in range(1, 17)]

    # Set the color palette for each ancestry prediction
    colors = {'oth':'gold','eur': 'darkblue', 'afr': 'darkred', 'amr': 'darkgreen', 'eas': 'purple', 'sas': 'darkorange', 'mid': 'black'}

    # Loop through each ancestry prediction and plot a line for each
    for ancestry, color in colors.items():
        # Calculate the mean of each principal component for the current ancestry
        mean_values = population_data.loc[population_data['ancestry_pred_other'] == ancestry, principal_components].median()
        plt.plot(principal_components, mean_values, label=ancestry, color=color, marker='o')

    # Rotate x-axis labels
    plt.xticks(rotation=90)

    # Rename x-axis labels
    plt.xticks(ticks=range(0, 16), labels=['PCA{}'.format(i) for i in range(1, 17)])

    plt.title('Line Chart - Principal Components by Ancestry Prediction')
    plt.xlabel('Principal Components')
    plt.ylabel('Median Values')
    plt.legend()
    plt.show()

    return pca, "PCA performed successfully. Scatterplots and line chart created."
def prepare_genotyping_data(pop_variable, population, person_df, person_information):
    """
    Prepares data for genotyping analysis based on the specified population variable.

    Args:
    pop_variable (str): The population variable to filter the data.
    population (pd.DataFrame): DataFrame containing population data.
    person_df (pd.DataFrame): DataFrame containing person data.
    person_information (pd.DataFrame): DataFrame containing person information data.

    Returns:
    pd.DataFrame: DataFrame containing filtered population data.
    pd.DataFrame: DataFrame containing filtered person data.
    pd.DataFrame: DataFrame containing filtered person information data.
    str: Message indicating the action taken.
    """
    if pop_variable == 'all':
        message = "Using all Ancestries for genotyping analysis."
    else:
        population = population[population['ancestry_pred'] == pop_variable]
        message = f"Using {pop_variable} population ancestry for genotyping analysis."

    population.rename(columns={'research_id': 'person_id'}, inplace=True)
    ids = population['person_id'].astype(str).tolist()

    person_df['person_id'] = person_df['person_id'].astype(str)
    person_df = person_df[person_df['person_id'].isin(ids)].reset_index(drop=True)

    person_information['person_id'] = person_information['person_id'].astype(str)
    person_information = person_information[person_information['person_id'].isin(ids)].reset_index(drop=True)

    return population, person_df, person_information, message
def remove_related_samples(person_df):
    """
    Removes samples that are flagged as related from the person DataFrame.

    Args:
    person_df (pd.DataFrame): DataFrame containing person data.
    relatedness_flaged_samples (pd.DataFrame): DataFrame containing relatedness flagged samples.

    Returns:
    pd.DataFrame: DataFrame containing person data with related samples removed.
    str: Message indicating the action taken.
    """
    relatedness = read_tsv('relatedness.tsv')
    relatedness_flaged_samples = read_tsv('relatedness_flagged_samples.tsv')
    ids = relatedness_flaged_samples['sample_id'].astype(str).tolist()
    person_df = person_df[~person_df['person_id'].isin(ids)].reset_index(drop=True)
    message = f"{len(ids)} samples are flagged as related and removed from the person DataFrame."
    return person_df, message
def transform_genotypes(df, alleles_row, reference_allele=None, alternative_allele=None):
    # Step 1: Initialize the message
    message = ""

    # Step 2: Remove rows with missing genotype information
    df = df.dropna(subset=['GT'])

    # Step 4: Convert 'GT' column to string type
    df['GT'] = df['GT'].astype(str)

    # Step 4: Split 'GT' column into separate alleles
    df[['Allele_1', 'Allele_2']] = df['GT'].str.split('/', expand=True)

    # Step 6: Set the reference allele
    if reference_allele is None:
        if len(alleles_row) == 2:
            reference_allele = alleles_row[0]
            alternative_allele = alleles_row[1]
        else:
            reference_allele = alleles_row[0]

    # Step 8: Map allele values to letters
    allele_mapping = {str(i): allele for i, allele in enumerate(alleles_row)}
    df['Allele_1_map'] = df['Allele_1'].map(allele_mapping)
    df['Allele_2_map'] = df['Allele_2'].map(allele_mapping)

    # Step 9: Create 'Allele_combination' column and count occurrences
    df['Allele_combination'] = df['Allele_1_map'] + '/' + df['Allele_2_map']
    allele_counts = df['Allele_combination'].value_counts().reset_index()
    allele_counts.columns = ['Allele_combination', 'Allele_count']

    # Step 10: Determine alternative allele
    if len(alleles_row) != 2:
        alternative_combination = ""
        if alternative_allele is None:
            # Step 11: Find all combinations except reference/reference
            alternative_combinations = allele_counts[~allele_counts['Allele_combination'].str.contains(reference_allele)]
            if not alternative_combinations.empty:
                # Step 13: Find the most frequent combination with reference allele
                max_combination = alternative_combinations.loc[alternative_combinations['Allele_count'].idxmax()]
                alternative_combination = max_combination['Allele_combination']
                # Step 14: Set alternative allele as the non-reference allele in the most frequent combination
                alternative_allele = alternative_combination.replace(reference_allele, '')
    else:
        # If alleles_row has length 2, set the second allele as the alternative allele
        alternative_allele = alleles_row[1]

    # At this point, we have the reference and alternative alleles
    # Step 12 Remove reference allele from alternative_combination
    if len(alleles_row) != 2:
        if alternative_combination:
            alternative_allele = alternative_combination.replace(reference_allele, '')

    # Step 13 Create 'GT' column
    df['GT'] = -1  # Initialize all to -1

    # handle combination alternative allele
    if alternative_allele is None:
        alternative_combination = ""
        alternative_combinations = allele_counts[~allele_counts['Allele_combination'].str.contains(reference_allele)]
        if not alternative_combinations.empty:
            max_combination = alternative_combinations.loc[alternative_combinations['Allele_count'].idxmax()]
            alternative_combination = max_combination['Allele_combination']
            alternative_allele = alternative_combination.replace(reference_allele, '')

    # Step 14 Split the alternative allele if it's a combination
    if alternative_allele and '/' in alternative_allele:
        alternative_alleles = alternative_allele.split('/')
    else:
        alternative_alleles = [alternative_allele] if alternative_allele else []

    # Step 15 Assign genotype values based on reference and alternative alleles
    df.loc[(df['Allele_1_map'] == reference_allele) & (df['Allele_2_map'] == reference_allele), 'GT'] = 0
    if alternative_alleles:
        df.loc[(df['Allele_1_map'] == reference_allele) & (df['Allele_2_map'] == alternative_alleles[0]), 'GT'] = 1
        df.loc[(df['Allele_1_map'] == alternative_alleles[0]) & (df['Allele_2_map'] == reference_allele), 'GT'] = 1

    # Check if genotype 1 is present but genotype 2 is not
    if 1 in df['GT'].values and 2 not in df['GT'].values:
        # Set genotype 1 where Allele_1_map is the first allele in alternative_alleles
        df.loc[(df['Allele_1_map'] == alternative_alleles[0]) & (df['Allele_2_map'] == alternative_alleles[0]), 'GT'] = 1

    # Step 16 Check if alternative_alleles contains two or more elements
    if len(alternative_alleles) >=1:
        df.loc[(df['Allele_1_map'] == alternative_alleles[0]) & (df['Allele_2_map'] == alternative_alleles[0]), 'GT'] = 2

    # Step 17 Remove rows with GT == -1 and keep only the reference and alternative alleles
    df = df[df['GT'] != -1]
    
    # Filter allele combinations with counts less than 20
    filtered_allele_counts = allele_counts[allele_counts['Allele_count'] >= 20]

    
    # Step 18: Generate message
    message += "Alleles for SNP: {}\n".format(alleles_row)
    message += "Reference allele: {}\n".format(reference_allele)
    message += "Alternative allele: {}\n".format(alternative_allele)
    message += "Allele combinations count:\n{}\n".format(allele_counts)
    message += "Removed {} rows with missing genotype information.\n".format(len(df[df['GT'].isna()]))

    return df, message
def create_pie_chart(genotype_counts, rs_name, pop_variable):
    """
    Create a pie chart based on genotype counts.

    Args:
    genotype_counts (pd.Series): Series containing genotype counts.
    rs_name (str): Name of the SNP.
    pop_variable (str): Name of the population variable.

    Returns:
    None
    """
    # Replace values less than 20 with a placeholder
    genotype_counts = genotype_counts.apply(lambda x: x if x >= 20 else '<20')
    if 1 not in genotype_counts.index:
        # If '2' is not present, set its count to 0
        genotype_counts[1] = 0
    # Check if genotype '2' is present in the counts
    if 2 not in genotype_counts.index:
        # If '2' is not present, set its count to 0
        genotype_counts[2] = 0
    

    # Plot the pie chart
    pie_chart = genotype_counts.plot(kind='pie', title='{}'.format(rs_name), legend=True)
    
    
    # Add the genotype counts to the title
    title_with_counts = '{}  with {} population(0: {}, 1: {}, 2: {})'.format(rs_name,pop_variable, 
                                                         genotype_counts.get(0, 0),
                                                         genotype_counts.get(1, 0),
                                                         genotype_counts.get(2, 0))
    pie_chart.set_title(title_with_counts)

    # Save the figure
    plt.savefig('./download/{}_{}_pie_GT_my_data_before_remove_data.pdf'.format(rs_name, pop_variable), dpi=300, facecolor='w',bbox_inches='tight')

    # Show the plot
    plt.show()
def filter_by_ethnicity(person_df, ethnicity=None):
    """
    Filters the person DataFrame based on ethnicity.

    Args:
    person_df (pd.DataFrame): DataFrame containing person data.
    ethnicity (str or None): The ethnicity to filter by. If None, no filtering is applied.

    Returns:
    pd.DataFrame: DataFrame containing filtered person data.
    str: Message indicating the action taken.
    """
    if ethnicity is None:
        message = "No specific ethnicity selected. Performing a classic run."
        return person_df, message

    if ethnicity.lower() == 'hispanic':
        filtered_df = person_df[person_df['ethnicity'] == 'Hispanic or Latino']
        message = "Filtering by Hispanic ethnicity."
    elif ethnicity.lower() == 'not_hispanic':
        person_df = person_df[person_df['ethnicity'] == 'Not Hispanic or Latino']
        message = "Filtering by Non-Hispanic ethnicity."
    else:
        message = f"Invalid ethnicity option: {ethnicity}. Performing a classic run."
        return person_df, message

    return person_df, message
def perform_hail_analysis(rs_name, rs_position, sex, person_df, mt_acaf):
    """
    Performs Hail analysis for the given SNP and sex variable.

    Args:
    rs_name (str): The name of the SNP.
    rs_position (str): The position of the SNP.
    sex (str): The sex variable to filter the data ('Male', 'Female', or None).
    person_df (pd.DataFrame): DataFrame containing person data.
    mt_acaf (str): Path to the MatrixTable file.

    Returns:
    pd.DataFrame: DataFrame containing genotype information.
    str: Message indicating the action taken.
    """
    # Initialize message
    message = ""

    # Set up Hail environment
    mt = hl.read_matrix_table(mt_acaf)

    # Filter MatrixTable for the given SNP position
    test_intervals = [rs_position]
    mt = hl.filter_intervals(mt, [hl.parse_locus_interval(x) for x in test_intervals])

    # Get genotype quality (GQ) histogram
    p = hl.plot.histogram(mt.GQ, range=(0, 100), bins=10, title='GQ Histogram', legend='GQ')

    # Add alleles message
    alleles_row = mt.aggregate_rows(hl.agg.collect(mt.alleles))
    alleles_row = alleles_row[0]
    message += f"Alleles for SNP {rs_name}: {alleles_row[0]}\n"

    # Create DataFrame with genotype information
    s = mt.aggregate_entries(hl.agg.collect(mt.s))
    AD = mt.aggregate_entries(hl.agg.collect(mt.AD))
    GT = mt.aggregate_entries(hl.agg.collect(mt.GT))
    FT = mt.aggregate_entries(hl.agg.collect(mt.FT))
    GQ = mt.aggregate_entries(hl.agg.collect(mt.GQ))
    RGQ = mt.aggregate_entries(hl.agg.collect(mt.RGQ))
    df = pd.DataFrame({'s': s, 'GT': GT, 'AD': AD, 'FT': FT, 'GQ': GQ, 'RGQ': RGQ})

    # Rename columns
    df = df.rename(columns={'s': 'id'})

    # Filter person DataFrame based on sex
    if sex.lower() == 'male':
        person_df_filtered = person_df[person_df['sex_at_birth'] == 'Male']
        message += f"Filtering by Male sex.\n"
    elif sex.lower() == 'female':
        person_df_filtered = person_df[person_df['sex_at_birth'] == 'Female']
        message += f"Filtering by Female sex.\n"
    else:
        person_df_filtered = person_df
        message += "No specific sex selected. Performing analysis on all individuals.\n"

    # Filter DataFrame based on person DataFrame
    person_df_filtered = person_df_filtered.rename(columns={'person_id': 'id'})
    person_df_filtered['id'] = person_df_filtered['id'].astype(str)
    samples = person_df_filtered['id'].tolist()
    df['id'] = df['id'].astype(str)
    df = df[df['id'].isin(samples)]
    df = pd.merge(df, person_df_filtered, on='id')

    # Save pie chart
    df['GT'].value_counts().plot(kind='pie', title='{}_{}'.format(rs_name, sex),
                                  legend=True).get_figure().savefig('./download/{}_{}_pie_GT_all_1.pdf'.format(rs_name, sex),
                                                                     dpi=300, facecolor='w',bbox_inches='tight')
    plt.show()

    return df, message,alleles_row
def filter_data_and_print_value_counts(df, rs_name, person_information, GQ_threshold, phecode_min):
    """
    Filter the genotype data based on genotype quality threshold and print the value counts of genotypes.

    Args:
    df (pd.DataFrame): DataFrame containing genotype data.
    rs_name (str): Name of the SNP.
    person_information (pd.DataFrame): DataFrame containing person information for conditions.
    GQ_threshold (int): Genotype quality threshold (default is 20).
    phecode_min (int): Minimum number of PheCodes in the dataset (default is 0).

    Returns:
    pd.DataFrame: Filtered DataFrame for genotype data.
    str: Message indicating the number of entries removed based on GQ threshold and PheCode minimum.
    pd.DataFrame or None: DataFrame containing conditions data if phecode_min is greater than zero, otherwise None.
    """
    # Choose samples with genotype quality >= GQ_threshold
    num_samples_removed_gq = df.shape[0] - df[df['GQ'] >= GQ_threshold].shape[0]
    df = df[df['GQ'] >= GQ_threshold]

    # Rename the 'GT' column to 'rs_name'
    df = df.rename(columns={'GT': rs_name})

    # Print value counts of genotypes
    genotype_value_counts = df[rs_name].value_counts()
    print(genotype_value_counts)

    # Sort DataFrame by 'id'
    df = df.sort_values(by=['id'])

    # Create conditions DataFrame
    conditions = pd.DataFrame(data=person_information, columns=['person_id','source_vocabulary','source_concept_code','source_concept_name','count'])
    conditions['count'] = conditions.groupby(['person_id','source_vocabulary', 'source_concept_code','source_concept_name'])['source_concept_code'].transform('count')
    conditions.sort_values(by=['person_id'], inplace=True)
    conditions.reset_index(drop=True, inplace=True)

    # Filter DataFrame based on minimum PheCode count if phecode_min is greater than zero
    if phecode_min > 0:
        list_con = conditions['source_concept_code'].value_counts().rename_axis('source_concept_code').to_frame('counts')
        list_con = list_con.reset_index()
        list_con = list_con[list_con['counts'] >= phecode_min]
        num_entries_removed_phecode = conditions.shape[0] - conditions[conditions['source_concept_code'].isin(list_con['source_concept_code'])].shape[0]
        conditions = conditions[conditions['source_concept_code'].isin(list_con['source_concept_code'])]
        conditions = conditions.reset_index(drop=True)
    else:
        num_entries_removed_phecode = 0

    # Create message
    message = f"Number of entries removed based on GQ threshold ({GQ_threshold}): {num_samples_removed_gq}\n"
    message += f"Number of entries removed based on PheCode minimum threshold ({phecode_min}): {num_entries_removed_phecode}"

    return df, message, conditions
def filter_common_ids(samples_GT, conditions, rs_name):
    """
    Keep only the IDs that have data in both samples_GT and conditions DataFrames.

    Args:
    samples_GT (pd.DataFrame): DataFrame containing genotype data with 'id' column.
    conditions (pd.DataFrame): DataFrame containing conditions data with 'id' column.
    rs_name (str): Name of the SNP.

    Returns:
    pd.DataFrame: DataFrame containing samples_GT with common IDs.
    pd.DataFrame: DataFrame containing conditions with common IDs.
    str: Message indicating the number of IDs removed and the number of unique IDs remaining in both DataFrames.
    """
    # Rename the 'person_id' column in conditions to 'id'
    conditions = conditions.rename(columns={'person_id': 'id'})

    # Convert 'id' column in samples_GT and conditions to string
    df['id'] = df['id'].astype(str)
    conditions['id'] = conditions['id'].astype(str)

    # Get unique values of 'id' columns in samples_GT and conditions
    df_unique_list = set(df['id'].unique())
    conditions_unique_list = set(conditions['id'].unique())

    # Keep only values in conditions_unique_list that are in conditions_unique_list
    common_ids = df_unique_list.intersection(conditions_unique_list)

    # Filter df and conditions_unique based on the common 'id' values
    df_common = df[df['id'].isin(common_ids)]
    conditions_common = conditions[conditions['id'].isin(common_ids)]

    # Select specific columns in df
    df_common = df_common[['id', rs_name]]

    # Create message
    message = f"Number of IDs removed: {len(df_unique_list) - len(common_ids)}\n"
    message += f"Number of unique IDs remaining in both DataFrames: {len(common_ids)}"

    return df_common, conditions_common, message
def filter_df_by_common_ids(df, df_common):
    """
    Keep only the IDs that are present in the df_common DataFrame from the original df.

    Args:
    df (pd.DataFrame): Original DataFrame containing genotype data.
    df_common (pd.DataFrame): DataFrame containing common IDs.

    Returns:
    pd.DataFrame: DataFrame containing df with common IDs.
    """
    # Filter the original DataFrame based on common IDs
    filtered_df = df[df['id'].isin(df_common['id'])]

    return filtered_df
def generate_genotype_frequency_table(filtered_df, rs_name):
    """
    Generate a genotype frequency table for the given DataFrame and SNP name.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing genotype data.
    rs_name (str): Name of the SNP.

    Returns:
    pd.DataFrame: DataFrame containing the genotype frequency table.
    str: Message indicating the table has been created and saved.
    """
    # Count the occurrences of each genotype value for all samples
    all_genotype_counts = filtered_df[rs_name].value_counts()

    # Count the occurrences of each genotype value for male samples
    male_genotype_counts = filtered_df[filtered_df['sex_at_birth'] == 'Male'][rs_name].value_counts()

    # Count the occurrences of each genotype value for female samples
    female_genotype_counts = filtered_df[filtered_df['sex_at_birth'] == 'Female'][rs_name].value_counts()

    # Define the genotypes
    genotypes = [0, 1, 2]

    # Create Series for all_gt, male_gt, and female_gt
    all_gt = pd.Series(all_genotype_counts, name='all_gt', index=genotypes)
    male_gt = pd.Series(male_genotype_counts, name='male_gt', index=genotypes)
    female_gt = pd.Series(female_genotype_counts, name='female_gt', index=genotypes)

    # Calculate the total genotype count for all samples
    total_all_gt = all_gt.sum()

    # Calculate the total genotype count for males
    total_male_gt = male_gt.sum()

    # Calculate the total genotype count for females
    total_female_gt = female_gt.sum()

    # Check if counts for male, female, or all genotypes are less than 20
    if total_all_gt < 20 or total_male_gt < 20 or total_female_gt < 20:
        print("Counts for male, female, or all genotypes are less than 20. Changing counts to <20.")
        all_gt = all_gt.apply(lambda x: x if x >= 20 else '<20')
        male_gt = male_gt.apply(lambda x: x if x >= 20 else '<20')
        female_gt = female_gt.apply(lambda x: x if x >= 20 else '<20')

    # Replace NaN counts with 0
    all_gt.fillna(0, inplace=True)
    male_gt.fillna(0, inplace=True)
    female_gt.fillna(0, inplace=True)
    # Convert counts to integers
    all_gt = all_gt.astype(int)
    male_gt = male_gt.astype(int)
    female_gt = female_gt.astype(int)

    # Create a DataFrame to store the frequencies of each genotype, with NaN for missing values
    frequency_df = pd.DataFrame({'all_gt': all_gt, 'male_gt': male_gt, 'female_gt': female_gt})

    # Replace counts less than 20 with "<20"
    frequency_df = frequency_df.applymap(lambda x: x if isinstance(x, str) else '<20' if x < 20 else x)

    # Display the frequency DataFrame using tabulate
    table_str = tabulate(frequency_df, headers='keys', tablefmt='pretty')

    # Save the DataFrame to a CSV file
    file_path = './download/{}_frequency_of_genotype.csv'.format(rs_name)
    frequency_df.to_csv(file_path, index=False)

    # Message indicating the table has been created and saved
    message = f"Genotype frequency table has been created and saved.\n"
    message += table_str

    return frequency_df, message
def plot_genotype_proportion(filtered_df, rs_name, pop_variable):
    """
    Plot the proportion of male and female genotypes from the filtered DataFrame and save it as an image.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing the filtered genotype data.
    rs_name (str): Name of the SNP.
    pop_variable (str): Name of the population.

    Returns:
    None
    """
    # Count the occurrences of each genotype value for all samples
    all_gt = filtered_df[rs_name].value_counts().tolist()

    # Count the occurrences of each genotype value for male samples
    male_gt = filtered_df[filtered_df['sex_at_birth'] == 'Male'][rs_name].value_counts().tolist()
    if len(male_gt) != 3:
        male_gt.append(0)

    # Count the occurrences of each genotype value for female samples
    female_gt = filtered_df[filtered_df['sex_at_birth'] == 'Female'][rs_name].value_counts().tolist()
    if len(female_gt) != 3:
        female_gt.append(0)

    # Total number of samples
    total_samples = sum(all_gt)

    # Calculate proportions
    all_gt_proportion = [count / total_samples * 100 for count in all_gt]
    male_gt_proportion = [count / total_samples * 100 for count in male_gt]
    female_gt_proportion = [count / total_samples * 100 for count in female_gt]

    # Define the genotypes
    genotypes = ['0', '1', '2']

    # Create DataFrame to store the proportions of each genotype
    proportion_df = pd.DataFrame({'All': all_gt_proportion, 'Male': male_gt_proportion, 'Female': female_gt_proportion}, index=genotypes)

    # Melt the DataFrame to long format for easier plotting
    melted_df = pd.melt(proportion_df.reset_index(), id_vars='index', var_name='Group', value_name='Proportion')

    # Plotting grouped bar graph
    plt.figure(figsize=(10, 6))
    sns.barplot(x='index', y='Proportion', hue='Group', data=melted_df, palette='viridis')

    # Set y-axis limits
    plt.ylim(0, 100)

    # Adding labels and title
    plt.xlabel('Genotypes')
    plt.ylabel('Proportion (%)')
    plt.title('Proportion of Male and Female Genotypes')
    plt.legend(frameon=False)
    ax = plt.gca()
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    # do not show the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    plt.grid(False)
    plt.tight_layout()
    # Save the figure
    plt.savefig('./download/{}_{}_Genotype_Proportions_for_Different_Sex_Groups.pdf'.format(rs_name, pop_variable), bbox_inches='tight', pad_inches=0.5, dpi=300, facecolor='w')

    # Show plot
    plt.show()
def transform_sex_labels_and_calculate_age(df):
    """
    Transform sex labels from 'Male' to 'M' and 'Female' to 'F' in the DataFrame,
    and calculate the age based on the date of birth.

    Args:
    df (pd.DataFrame): DataFrame containing the sex labels and date of birth.

    Returns:
    pd.DataFrame: DataFrame with transformed sex labels and calculated age.
    str: Message indicating the transformation and age calculation.
    """
    # Make a copy of the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()

    # Map 'Male' to 'M' and 'Female' to 'F' in a new 'sex' column
    df_copy['sex'] = df_copy['sex_at_birth'].map({'Male': 'M', 'Female': 'F'})

    # Calculate age based on date of birth
    df_copy['date_of_birth'] = df_copy['date_of_birth'].astype(str).str[:4]
    df_copy['age'] = 2023 - df_copy['date_of_birth'].astype(int)

    # Add a message
    message = 'Transformed sex labels to "M" and "F" and calculated age.'

    return df_copy, message
def plot_age_distribution_for_genotypes(filtered_df, rs_name):
    """
    Generate plots for age distribution for different genotypes in males, females, and overall.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing the filtered genotype data.
    rs_name (str): Name of the SNP.

    Returns:
    str: Message indicating that plots have been saved.
    """
    print(filtered_df.head())  # Debug print to check the filtered DataFrame
    genotypes = [0, 1, 2]
    sex_labels = {'M': 'Male', 'F': 'Female'}

    for genotype in genotypes:
        # Debug print to check the number of samples for each genotype
        print(f"Number of samples for genotype {genotype}: {len(filtered_df[filtered_df[rs_name] == genotype])}")

        # Skip plotting if there are no samples for the current genotype
        if len(filtered_df[filtered_df[rs_name] == genotype]) == 0:
            print(f"No samples found for genotype {genotype}. Skipping plotting.")
            continue

        # Visualize age distribution for different genotypes in males
        plt.figure(figsize=(8, 6))
        for sex in ['M', 'F']:
            sns.histplot(filtered_df[(filtered_df['sex'] == sex) & (filtered_df[rs_name] == genotype)]['age'], kde=True, label=f'{sex_labels[sex]}')
        plt.title(f'Age Distribution for Genotype {genotype} (Male vs Female)')
        plt.xlabel('Age')
        plt.ylabel('Density')
        plt.legend(frameon=False)
        ax = plt.gca()
        ax.spines['left'].set_color('black')
        ax.spines['bottom'].set_color('black')
        # do not show the right and top spines
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_linewidth(2)
        ax.spines['left'].set_linewidth(2)
        plt.grid(False)
        plt.tight_layout()
        plt.savefig(f'./download/{rs_name}_age_distribution_genotype_{genotype}_male_vs_female.pdf', dpi=300)
        plt.show()

        # Visualize overall age distribution for different genotypes
        plt.figure(figsize=(8, 6))
        sns.histplot(filtered_df[filtered_df[rs_name] == genotype]['age'], kde=True)
        plt.title(f'Overall Age Distribution for Genotype {genotype}')
        plt.xlabel('Age')
        plt.ylabel('Density')
        plt.legend(frameon=False)
        ax = plt.gca()
        ax.spines['left'].set_color('black')
        ax.spines['bottom'].set_color('black')
        # do not show the right and top spines
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_linewidth(2)
        ax.spines['left'].set_linewidth(2)
        plt.grid(False)
        plt.tight_layout()
        plt.savefig(f'./download/{rs_name}_age_distribution_genotype_{genotype}_overall.pdf', dpi=300)
        plt.show()

    message = 'Plots for age distribution for different genotypes have been saved.'
    return message
def plot_age_distribution_for_genotypes(filtered_df, rs_name, pop_variable):
    """
    Generate plots for age distribution for different genotypes in males, females, and overall.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing the filtered genotype data.
    rs_name (str): Name of the SNP.
    pop_variable (str): Population variable.

    Returns:
    str: Message indicating that plots have been saved.
    """
    # Convert genotypes to string
    genotypes = [0, 1, 2]

    # Sex labels mapping
    sex_labels = {'M': 'Male', 'F': 'Female'}

    # Visualize age distribution for different genotypes in males
    plt.figure(figsize=(8, 6))
    for genotype in genotypes:
        sns.histplot(filtered_df[(filtered_df['sex'] == 'M') & (filtered_df[rs_name] == genotype)]['age'], kde=True, label=f'GT={genotype}')
    plt.title('Age Distribution in Males for Different Genotypes without sex aggregation')
    plt.xlabel('Age')
    plt.ylabel('Density')
    plt.legend(frameon=False)
    ax = plt.gca()
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    # do not show the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(f'./download/{rs_name}_{pop_variable}_age_Male_distribution_with_dif_Genotypes_without_sex_aggregation.pdf', dpi=300, facecolor='w')
    plt.show()

    # Visualize age distribution for different genotypes in females
    plt.figure(figsize=(8, 6))
    for genotype in genotypes:
        sns.histplot(filtered_df[(filtered_df['sex'] == 'F') & (filtered_df[rs_name] == genotype)]['age'], kde=True, label=f'GT={genotype}')
    plt.title('Age Distribution in Females for Different Genotypes without sex aggregation')
    plt.xlabel('Age')
    plt.ylabel('Density')
    plt.legend(frameon=False)
    ax = plt.gca()
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    # do not show the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(f'./download/{rs_name}_{pop_variable}_age_Female_distribution_with_dif_Genotypes_without_sex_agg.pdf', dpi=300, facecolor='w')
    plt.show()

    # Visualize overall age distribution for different genotypes
    plt.figure(figsize=(8, 6))
    for genotype in genotypes:
        sns.histplot(filtered_df[filtered_df[rs_name] == genotype]['age'], kde=True, label=f'GT={genotype}')
    plt.title('Overall Age Distribution for Different Genotypes without sex agg')
    plt.xlabel('Age')
    plt.ylabel('Density')
    plt.legend(frameon=False)
    ax = plt.gca()
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    # do not show the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(f'./download/{rs_name}_{pop_variable}_age_all_population_distribution_with_dif_Genotypes_without_sex_agg.pdf', dpi=300, facecolor='w')
    plt.show()

    message = f'Plots for age distribution for different genotypes have been saved.'
    return message
def merge_population_data(filtered_df, population):
    """
    Merge population data with genotype data.

    Args:
    filtered_df (pd.DataFrame): DataFrame containing the filtered genotype data.
    population (pd.DataFrame): DataFrame containing population data.

    Returns:
    pd.DataFrame: Combined DataFrame with genotype and population data.
    str: Message indicating that population data has been merged.
    str: Message indicating if any samples were removed after the merge.
    """

    # Rename the column 'person_id' to 'id' in population_new
    population_new = population.rename(columns={'person_id': 'id'})

    # Ensure 'id' column in population_new is of the same data type as in filtered_df
    population_new['id'] = population_new['id'].astype(str)

    # Merge population_new with filtered_df on 'id' column
    merged_df = pd.merge(filtered_df, population_new, on='id', how='left')

    # Drop columns ending with '_x' or '_y' (resulting from merge)
    merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_x') & ~merged_df.columns.str.endswith('_y')]

    # Check if any samples were removed after the merge
    removed_samples = len(filtered_df) - len(merged_df)
    if removed_samples > 0:
        removed_message = f"{removed_samples} samples were removed after the merge."
    else:
        removed_message = "No samples were removed after the merge."

    # Message indicating that population data has been merged
    merge_message = "Population data has been merged with genotype data."

    return merged_df, merge_message, removed_message
def filter_and_create_dataframes(merged_df, conditions_common, icd9cm=True, icd10cm=True):
    """
    Filter the merged DataFrame based on common IDs and create separate DataFrames for ICD9CM and ICD10CM, or combine both.

    Args:
    merged_df (pd.DataFrame): Merged DataFrame containing genotype and population data.
    conditions_common (pd.DataFrame): DataFrame containing condition data.
    icd9cm (bool): Whether to create a DataFrame for ICD9CM (default is True).
    icd10cm (bool): Whether to create a DataFrame for ICD10CM (default is True).

    Returns:
    pd.DataFrame: Combined DataFrame with ICD9CM and ICD10CM data, or separate DataFrames based on user choice.
    str: Message indicating how many data points were removed and how many remain.
    """

    # Find filtered ids
    final_data_ids = merged_df['id'].tolist()

    # Filter the ids from conditions_common that are present in final_data
    filtered_ids = conditions_common[conditions_common['id'].isin(final_data_ids)]
    removed_count = len(conditions_common) - len(filtered_ids)

    if icd9cm and icd10cm:
        # Separate DataFrames for ICD9CM and ICD10CM
        conditions_unique_ICD10CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD10CM']
        conditions_unique_ICD9CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD9CM']

        # Save separate DataFrames
        save_conditions_unique_ICD9CM = conditions_unique_ICD9CM[['id', 'source_vocabulary', 'source_concept_code', 'count']]
        save_conditions_unique_ICD10CM = conditions_unique_ICD10CM[['id', 'source_vocabulary', 'source_concept_code', 'count']]

        # Combine ICD9CM and ICD10CM DataFrames
        conditions_unique_10_9 = pd.concat([save_conditions_unique_ICD10CM, save_conditions_unique_ICD9CM], axis=0)
        remaining_count = len(conditions_unique_10_9)

        message = f"{removed_count} Disease were removed. Disease points: {remaining_count}"

        return conditions_unique_10_9, message
    elif icd9cm:
        # Filter only ICD9CM data
        conditions_unique_ICD9CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD9CM']
        remaining_count = len(conditions_unique_ICD9CM)

        message = f"{removed_count} Disease were removed. Disease points for ICD9CM: {remaining_count}"

        return conditions_unique_ICD9CM[['id', 'source_vocabulary', 'source_concept_code', 'count']], message
    elif icd10cm:
        # Filter only ICD10CM data
        conditions_unique_ICD10CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD10CM']
        remaining_count = len(conditions_unique_ICD10CM)

        message = f"{removed_count} Disease were removed. Remaining Disease for ICD10CM: {remaining_count}"

        return conditions_unique_ICD10CM[['id', 'source_vocabulary', 'source_concept_code', 'count']], message
    else:
        # Combine ICD9CM and ICD10CM DataFrames if both icd9cm and icd10cm are False
        conditions_unique_ICD10CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD10CM']
        conditions_unique_ICD9CM = filtered_ids[filtered_ids['source_vocabulary'] == 'ICD9CM']

        # Save separate DataFrames
        save_conditions_unique_ICD9CM = conditions_unique_ICD9CM[['id', 'source_vocabulary', 'source_concept_code', 'count']]
        save_conditions_unique_ICD10CM = conditions_unique_ICD10CM[['id', 'source_vocabulary', 'source_concept_code', 'count']]

        # Combine ICD9CM and ICD10CM DataFrames
        conditions_unique_10_9 = pd.concat([save_conditions_unique_ICD10CM, save_conditions_unique_ICD9CM], axis=0)
        remaining_count = len(conditions_unique_10_9)

        message = f"{removed_count} Disease were removed. Disease patients for ICD9CM and ICD10CM: {remaining_count}"

        return conditions_unique_10_9, message
def create_pie_chart_after_filtering(genotype_counts, rs_name, pop_variable):
    """
    Create a pie chart based on genotype counts.

    Args:
    genotype_counts (pd.Series): Series containing genotype counts.
    rs_name (str): Name of the SNP.
    pop_variable (str): Name of the population variable.

    Returns:
    None
    """
    # Replace values less than 20 with a placeholder
    genotype_counts = genotype_counts.apply(lambda x: x if x >= 20 else '<20')

    # Plot the pie chart
    pie_chart = genotype_counts.plot(kind='pie', title='{}'.format(rs_name), legend=True)

    # Add the genotype counts to the title
    title_with_counts = '{} (0: {}, 1: {}, 2: {})'.format(rs_name, 
                                                         genotype_counts.get(0, 0),
                                                         genotype_counts.get(1, 0),
                                                         genotype_counts.get(2, 0))
    pie_chart.set_title(title_with_counts)
    

    # Save the figure
    plt.savefig('./download/{}_{}_pie_GT_my_data_after_filtering.pdf'.format(rs_name, pop_variable), dpi=300, facecolor='w')

    # Show the plot
    plt.show()
def create_genotype_files_and_plots(merged_df, rs_name, pop_variable):
    """
    Create two files and two plots for genotype information from the merged DataFrame.
    
    Args:
    merged_df (pd.DataFrame): DataFrame containing genotype and population data.
    rs_name (str): Name of the genotype column.
    pop_variable (str): Name of the population variable.

    Returns:
    None
    """

    # Check if the merged_df is empty
    if merged_df.empty:
        print("Error: The merged DataFrame is empty.")
        return
    
    # Define the genotype values to count
    genotypes_to_count = [0, 1, 2]

    # Group by 'ancestry_pred' and aggregate the data
    aggregated_data = merged_df.groupby('ancestry_pred').agg({
        'sex': lambda x: x.value_counts().to_dict(),  # Count of each sex
    })

    # Count each genotype for each ancestry_pred group
    for genotype in genotypes_to_count:
        aggregated_data[f'M_gt_{genotype}'] = merged_df.loc[merged_df['sex'] == 'M'].groupby('ancestry_pred')[rs_name].apply(lambda x: (x == genotype).sum())
        aggregated_data[f'F_gt_{genotype}'] = merged_df.loc[merged_df['sex'] == 'F'].groupby('ancestry_pred')[rs_name].apply(lambda x: (x == genotype).sum())
    # Count each genotype for each ancestry_pred group
    for genotype in genotypes_to_count:
        aggregated_data[f'genotype_{genotype}_count'] = merged_df.groupby('ancestry_pred')[rs_name].apply(lambda x: (x == genotype).sum())

    # Ensure counts for males ('M') and females ('F') are integers and replace NaN with 0
    aggregated_data['male_count'] = aggregated_data['sex'].apply(lambda x: int(x.get('M', 0)) if isinstance(x, dict) else 0)
    aggregated_data['female_count'] = aggregated_data['sex'].apply(lambda x: int(x.get('F', 0)) if isinstance(x, dict) else 0)

    # Drop the original 'sex' column
    aggregated_data = aggregated_data.drop('sex', axis=1)

    aggregated_data.reset_index(inplace=True)
    df1 = aggregated_data
    # Now 'ancestry_pred' is a regular column
    print(aggregated_data)
    ancestry_pred = df1['ancestry_pred']
    genotypes = ['0', '1', '2']

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    # Define bar width
    bar_width = 0.15

    # Define positions for each group of bars
    r1 = range(len(ancestry_pred))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    r4 = [x + bar_width for x in r3]
    r5 = [x + bar_width for x in r4]
    r6 = [x + bar_width for x in r5]

    # Plot bars for each genotype and gender combination
    plt.bar(r1, df1['M_gt_0'], color='b', width=bar_width, edgecolor='grey', label='M_gt_0')
    plt.bar(r2, df1['F_gt_0'], color='g', width=bar_width, edgecolor='grey', label='F_gt_0')
    plt.bar(r3, df1['M_gt_1'], color='r', width=bar_width, edgecolor='grey', label='M_gt_1')
    plt.bar(r4, df1['F_gt_1'], color='c', width=bar_width, edgecolor='grey', label='F_gt_1')
    plt.bar(r5, df1['M_gt_2'], color='m', width=bar_width, edgecolor='grey', label='M_gt_2')
    plt.bar(r6, df1['F_gt_2'], color='y', width=bar_width, edgecolor='grey', label='F_gt_2')

    # Add xticks on the middle of the group bars
    plt.xlabel('Ancestry prediction', fontweight='bold')
    plt.xticks([r + bar_width * 2.5 for r in range(len(ancestry_pred))], ancestry_pred)
    plt.title(f'Number of genotype in all ancestry for {rs_name}')
    # Add y label
    plt.ylabel('Count')
    ax = plt.gca()
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    # do not show the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    plt.grid(False)

    # Create legend & Show graphic
    plt.legend(frameon=False)
    # save it 
    plt.tight_layout()
    plt.savefig('./download/{}_{}_Number of genotype in all ancestry.pdf'.format(rs_name,pop_variable),dpi=300,facecolor='w')
    plt.show()

    # Define the genotype values to count
    genotypes_to_count = [0, 1, 2]

    # Group by 'ancestry_pred' and aggregate the data
    aggregated_data_other = merged_df.groupby('ancestry_pred_other').agg({
        'sex': lambda x: x.value_counts().to_dict(),  # Count of each sex
    })

    # Count each genotype for each ancestry_pred group
    for genotype in genotypes_to_count:
        aggregated_data_other[f'M_gt_{genotype}'] = merged_df.loc[merged_df['sex'] == 'M'].groupby('ancestry_pred_other')[rs_name].apply(lambda x: (x == genotype).sum())
        aggregated_data_other[f'F_gt_{genotype}'] = merged_df.loc[merged_df['sex'] == 'F'].groupby('ancestry_pred_other')[rs_name].apply(lambda x: (x == genotype).sum())
    # Count each genotype for each ancestry_pred group
    for genotype in genotypes_to_count:
        aggregated_data_other[f'genotype_{genotype}_count'] = merged_df.groupby('ancestry_pred_other')[rs_name].apply(lambda x: (x == genotype).sum())

    # Ensure counts for males ('M') and females ('F') are integers and replace NaN with 0
    aggregated_data_other['male_count'] = aggregated_data_other['sex'].apply(lambda x: int(x.get('M', 0)) if isinstance(x, dict) else 0)
    aggregated_data_other['female_count'] = aggregated_data_other['sex'].apply(lambda x: int(x.get('F', 0)) if isinstance(x, dict) else 0)

    # Drop the original 'sex' column
    aggregated_data_other = aggregated_data_other.drop('sex', axis=1)

    aggregated_data_other.reset_index(inplace=True)
    df1 = aggregated_data_other
    # Now 'ancestry_pred' is a regular column
    print(aggregated_data_other)
    ancestry_pred = df1['ancestry_pred_other']
    genotypes = [0, 1, 2]

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    # Define bar width
    bar_width = 0.15

    # Define positions for each group of bars
    r1 = range(len(ancestry_pred))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    r4 = [x + bar_width for x in r3]
    r5 = [x + bar_width for x in r4]
    r6 = [x + bar_width for x in r5]

    # Plot bars for each genotype and gender combination
    plt.bar(r1, df1['M_gt_0'], color='b', width=bar_width, edgecolor='grey', label='M_gt_0')
    plt.bar(r2, df1['F_gt_0'], color='g', width=bar_width, edgecolor='grey', label='F_gt_0')
    plt.bar(r3, df1['M_gt_1'], color='r', width=bar_width, edgecolor='grey', label='M_gt_1')
    plt.bar(r4, df1['F_gt_1'], color='c', width=bar_width, edgecolor='grey', label='F_gt_1')
    plt.bar(r5, df1['M_gt_2'], color='m', width=bar_width, edgecolor='grey', label='M_gt_2')
    plt.bar(r6, df1['F_gt_2'], color='y', width=bar_width, edgecolor='grey', label='F_gt_2')

    # Add xticks on the middle of the group bars
    plt.xlabel('Ancestry prediction', fontweight='bold')

    plt.xticks([r + bar_width * 2.5 for r in range(len(ancestry_pred))], ancestry_pred)

    plt.title(f'Number of genotype in all ancestry and other for {rs_name}')
    # Add y label
    plt.ylabel('Count')

    ax = plt.gca()
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    # do not show the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    plt.grid(False)

    # Create legend & Show graphic
    plt.legend(frameon=False)
    # save it 
    plt.savefig('./download/{}_{}_Number of genotype in all ancestry with other.pdf'.format(rs_name,pop_variable),dpi=300,facecolor='w')
    plt.show()
    # Define a function to replace values less than 20 with '<20'
    def replace_less_than_20(value):
        return '<20' if value < 20 else value

    # Iterate over the columns of the first dataframe (aggregated_data)
    for column in aggregated_data.columns[1:]:  # Exclude the 'ancestry_pred' column
        aggregated_data[column] = aggregated_data[column].apply(replace_less_than_20)

    # Iterate over the columns of the second dataframe (aggregated_data_other)
    for column in aggregated_data_other.columns[1:]:  # Exclude the 'ancestry_pred' column
        aggregated_data_other[column] = aggregated_data_other[column].apply(replace_less_than_20)

    # Save to CSV
    aggregated_data.to_csv('download/{}_{} Count of Genotypes in ancestry.csv'.format(rs_name, pop_variable), index=False)
    aggregated_data_other.to_csv('download/{}_{} Count of Genotypes in ancestry with other.csv'.format(rs_name, pop_variable), index=False)
def generate_genotype_plots(merged_df, rs_name, pop_variable):
    # Set the color palette for each genotype and specify the order of legend labels
    
    genotype_colors = {0: 'purple', 1: 'orange', 2: 'blue'}
    genotype_order = [0, 1, 2]
    
    # Combine all genotypes for the violin plot
    combined_genotypes = merged_df[merged_df[rs_name].isin(genotype_colors.keys())]

    # Melt the DataFrame for easier plotting
    melted_data = pd.melt(combined_genotypes, id_vars=rs_name, value_vars=[f'principal_component_{i}' for i in range(1, 17)])

    # Initialize the figure and axes for the combined violin plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Create a violin plot for all genotypes
    sns.violinplot(x="variable", y="value", hue=rs_name, data=melted_data, palette=genotype_colors.values(), inner='quartile', ax=ax)

    # Set chart title and labels
    ax.set_title(f'Violin Plot - PCA Values for All Genotypes for {rs_name}')
    ax.set_xlabel('Principal Components')
    ax.set_ylabel('PCA Values')
    ax.set_xticks(range(0, 16))
    ax.set_xticklabels(['PCA{}'.format(i) for i in range(1, 17)])
    ax.tick_params(axis='x', rotation=90)
    ax = plt.gca()
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    # do not show the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    plt.grid(False)
    
    # Create a custom legend with the correct color-number combination
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=f'Genotype {key}', markerfacecolor=value, markersize=10) for key, value in genotype_colors.items()]

    # Add the custom legend
    ax.legend(handles=legend_elements, title=rs_name, loc='upper right', bbox_to_anchor=(1.12, 1), labels=genotype_order,frameon=False)
    plt.tight_layout()
    # Save the combined plot
    plt.savefig(f'./download/{rs_name}_All_Genotypes_Violin_Plot {pop_variable}.pdf', dpi=300, facecolor='w')
def create_genotype_boxplot(merged_df, rs_name, pop_variable):
    """
    Create a boxplot and swarmplot for genotypes.

    Args:
    merged_df (pd.DataFrame): DataFrame containing genotype and population data.
    rs_name (str): Name of the genotype column.
    pop_variable (str): Name of the population variable.

    Returns:
    None
    """
    
    # Convert genotype values to different colors
    genotype_colors = {0: 'blue', 1: 'red', 2: 'orange'}

    # Create a boxplot and swarmplot
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='sex', y='age', data=merged_df, hue=rs_name, palette=genotype_colors)

    # Add swarmplot (optional)
    # sns.swarmplot(x='sex', y='age', data=all_data.sample(100), color='green', size=3, marker='o', label='Random Samples')

    plt.title(f'Boxplot of Age by Sex and Genotype for {rs_name}')
    plt.legend(frameon=False)
    ax = plt.gca()
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    # do not show the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    plt.grid(False)
    plt.tight_layout()
    # Save the plot
    plt.savefig(f'./download/{rs_name}_{pop_variable}_Boxplot_Swarmplot.pdf', dpi=300, facecolor='w')

    # Show the plot
    plt.show()
def read_data_information_file(file_path):
    try:
        with open(file_path, "r") as file:
            data = file.read()
        return data
    except FileNotFoundError:
        return "File not found. Please make sure the file path is correct."


# Start the analysis

In [ ]:
# Accumulate messages
all_messages_first = ""
bucket, genomic_location = initialize_hail_environment()
# Output the bucket and genomic location
print("Bucket:", bucket)
all_messages_first += f"**Address of your Bucket:**\n{bucket}\n\n"
print("Genomic Location:", genomic_location)
all_messages_first += f"**Your Genomic Location that you using:**\n{genomic_location}\n\n"
person_information_all, message = read_person_information()
print(message)
all_messages_first += f"**Download information about the patient's conditions:**\n{message}\n\n"
# print(person_information_all.head())
person_df_all, message = read_person_information_df()
print(message)
all_messages_first += f"**Download information about the patients:**\n{message}\n\n"
# print(person_df_all.head())
bucket = os.getenv('WORKSPACE_BUCKET')
copy_files_to_local(bucket)
zip_code_socioeconomic_df, message = read_zip_code_socioeconomic_data()
print(message)
all_messages_first+= f"**Download Zip-Code information about the patients:**\n{message}\n\n"
# print(zip_code_socioeconomic_df.head())

# Set Your Variables

## test variables and information for pheWAS if neaded

In [ ]:
all_messages = ""
all_messages+=all_messages_first
rs_name = 'rs439358' # SNP's Name 
rs_position = 'chr19:44908683-44908684' 
sex = 'None'  # Choose the sex variable ('Male', 'Female', or None)
pop_variable = 'all'  # Choose the population variable ['afr', 'amr', 'eas','eur', 'mid', 'sas', 'all']
ethnicity = 'None'  # Choose the ethnicity ('Hispanic or Latino' or 'Not Hispanic or Latino'), or None for classic run
reference_allele= None #None
alternative_allele= None
GQ_threshold=20
phecode_min=0
icd9cm=True
icd10cm=True
all_messages += f"**Population Information:** : ** {pop_variable} **\n\n"
# Print the text variable
print(all_messages)

In [ ]:
person_df, person_information, race_counts, hispanic_counts, population, race_counts_message, hispanic_counts_message, population_message, max_probability_message = run_analysis()
print(race_counts_message)
print(hispanic_counts_message)
print(population_message)
print(max_probability_message)
all_messages += f"**Race Counts:**\n{race_counts_message}\n\n"
all_messages += f"**Hispanic Counts:**\n{hispanic_counts_message}\n\n"
all_messages += f"**Population Message:**\n{population_message}\n\n"
all_messages += f"**Max Probability Message:**\n{max_probability_message}\n\n"

## do some filtering in the patients information

In [ ]:
# Find the common person_id values
common_ids = set(person_df_all['person_id']).intersection(set(person_information['person_id']))

# Filter both dataframes to keep only rows with common person_id
person_df_all_filtered = person_df_all[person_df_all['person_id'].isin(common_ids)]
person_information_filtered = person_information[person_information['person_id'].isin(common_ids)]

# Optional: Sort the filtered dataframes again by person_id
person_df_all_filtered = person_df_all_filtered.sort_values(by='person_id', ascending=True)
person_information_filtered = person_information_filtered.sort_values(by='person_id', ascending=True)
# # Display results
# print(person_df_all_filtered)
# print(person_information_filtered)
# print(population.columns)
# print(population)

## copy the dataframes to keep the originals the same 

In [ ]:
person_df_all = person_df_all_filtered.copy()
person_information = person_information_filtered.copy()

# read bacteria information

In [ ]:
def read_gram_positive_data():
    # This query represents dataset "gram-positive" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
    dataset_67733641_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_concept_id IN (SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (37116366, 4111261, 4161193, 4248801, 43530753)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1)
                )) c_occurrence 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                ON c_occurrence.condition_type_concept_id = c_type.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                ON v.visit_concept_id = visit.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                ON c_occurrence.condition_status_concept_id = c_status.concept_id"""
    
    dataset_67733641_condition_df = pd.read_gbq(
        dataset_67733641_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
# Filter the gram-positive data based on specific conditions
    gram_positive = dataset_67733641_condition_df.dropna(subset=['source_vocabulary','source_concept_name'])
    gram_positive = gram_positive[gram_positive['source_vocabulary'].str.contains('ICD9CM|ICD10CM', case=False)]
    gram_positive_filtered = gram_positive.copy()
#     gram_positive_filtered = gram_positive[gram_positive['source_concept_name'].str.contains('gram', case=False)]

    # Extract information about the gram-positive dataset
    gram_positive_unique_person_ids = gram_positive_filtered['person_id'].nunique()
    gram_positive_unique_concept_codes = gram_positive_filtered['source_concept_code'].nunique()
    gram_positive_unique_codes_list = gram_positive_filtered['source_concept_code'].unique()

    # Display the gram-positive data and information
    print("Gram-positive dataset:")
    print("Number of unique person_ids:", gram_positive_unique_person_ids)
    print("Number of unique source_concept_codes:", gram_positive_unique_concept_codes)
    print("Unique source_concept_codes:", ', '.join(gram_positive_unique_codes_list))
    print(gram_positive_filtered)
    return gram_positive_filtered, gram_positive_unique_person_ids, gram_positive_unique_concept_codes, gram_positive_unique_codes_list
# Call the function to read gram-positive data
gram_positive_data = read_gram_positive_data()
# gram_positive_data

In [ ]:
def gram_negative_data():
    # This query represents dataset "Gram-negative bacterial" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
    dataset_41959053_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_concept_id IN (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (
                                        258180, 40493038, 4214520, 4252534, 4253013, 43530750, 435742, 438133, 46274129
                                    )       
                                    AND full_text LIKE '%_rank1]%'      
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1
                            )
                    )  
                    AND (
                        c_occurrence.PERSON_ID IN (
                            SELECT
                                distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_ehr_data = 1 
                                ) 
                            )
                    )
                ) c_occurrence 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                    ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                    ON c_occurrence.condition_type_concept_id = c_type.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                    ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                    ON v.visit_concept_id = visit.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                    ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                    ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    gram_negative_df = pd.read_gbq(
        dataset_41959053_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
   
    return gram_negative_df
gram_negative_data = gram_negative_data()
# gram_negative_data

In [ ]:
# Filter the E. coli data based on specific conditions
gram_negative = gram_negative_data.dropna(subset=['source_vocabulary','source_concept_name'])
gram_negative = gram_negative[gram_negative['source_vocabulary'].str.contains('ICD9CM|ICD10CM', case=False)]
gram_negative_data_filtered =gram_negative.copy()
# gram_negative_data_filtered = gram_negative_data_filtered[gram_negative_data_filtered['source_concept_name'].str.contains('Gram', case=False)]
# gram_negative_data_filtered

In [ ]:

# Extract information about the E. coli dataset
gram_negative_unique_person_ids = gram_negative_data_filtered['person_id'].nunique()
gram_negative_unique_concept_codes = gram_negative_data_filtered['source_concept_code'].nunique()
gram_negative_unique_codes_list = gram_negative_data_filtered['source_concept_code'].unique()

# # Display the E. coli data and information
# print("Escherichia coli dataset:")
# print("Number of unique person_ids:", gram_negative_unique_person_ids)
# print("Number of unique source_concept_codes:", gram_negative_unique_concept_codes)
# print("Unique source_concept_codes:", ', '.join(gram_negative_unique_codes_list))
# print(gram_negative_data_filtered)

# read the bacteria information and filter the data

In [ ]:
gram_positive_icd_info = [
    {'ICD-9': ['008.45', '041.83'], 'ICD-10': ['A04.7', 'B96.89'], 'Bacteria': 'Clostridium difficile'},
    {'ICD-9': ['041.04', '041.82'], 'ICD-10': ['B95.2', 'B96.81'], 'Bacteria': 'Enterococcus faecalis'},
    {'ICD-9': ['022.1', '022.2', '022.3'], 'ICD-10': ['A22.1', 'A22.2', 'A22.3'], 'Bacteria': 'Bacillus anthracis'},
    {'ICD-9': ['027.0', '027.8', '027.9'], 'ICD-10': ['A32.0', 'A32.1', 'A32.9'], 'Bacteria': 'Listeria monocytogenes'},
    {'ICD-9': ['032.0', '032.1', '032.2'], 'ICD-10': ['A36.0', 'A36.1', 'A36.2'], 'Bacteria': 'Corynebacterium diphtheriae'},
    {'ICD-9': ['037.0', '037.9'], 'ICD-10': ['A35', 'A34'], 'Bacteria': 'Clostridium tetani'},
    {'ICD-9': ['041.04', '041.82'], 'ICD-10': ['B95.0', 'B96.81'], 'Bacteria': 'Lactobacillus acidophilus'},
    {'ICD-9': ['039.1', '039.8'], 'ICD-10': ['B95.89', 'A43.8'], 'Bacteria': 'Nocardia species'},
    {'ICD-9': ['011.0', '011.1', '011.2'], 'ICD-10': ['A15.0', 'A15.1', 'A15.2'], 'Bacteria': 'Mycobacterium tuberculosis'},
    {'ICD-9': ['039.0', '039.3', '039.9'], 'ICD-10': ['A42.0', 'A42.3', 'A42.9'], 'Bacteria': 'Actinomyces israelii'}
]
gram_positive_icd_info
gram_negative_icd_info = [
    {'ICD-9': ['041.3', '482.2'], 'ICD-10': ['B96.2', 'J15.0'], 'Bacteria': 'Klebsiella pneumoniae'},
    {'ICD-9': ['041.49', '482.84'], 'ICD-10': ['B96.2', 'A41.9'], 'Bacteria': 'Acinetobacter baumannii'},
    {'ICD-9': ['098.0', '098.11', '098.12'], 'ICD-10': ['A54.0', 'A54.1', 'A54.2'], 'Bacteria': 'Neisseria gonorrhoeae'},
    {'ICD-9': ['036.0', '036.1'], 'ICD-10': ['A39.0', 'A39.2'], 'Bacteria': 'Neisseria meningitidis'},
    {'ICD-9': ['041.5', '482.2'], 'ICD-10': ['B96.3', 'J14'], 'Bacteria': 'Haemophilus influenzae'},
    {'ICD-9': ['003.0', '003.1', '003.2'], 'ICD-10': ['A02.0', 'A02.1', 'A02.2'], 'Bacteria': 'Salmonella enterica'},
    {'ICD-9': ['004.0', '004.1', '004.2'], 'ICD-10': ['A03.0', 'A03.1', 'A03.2'], 'Bacteria': 'Shigella species'},
    {'ICD-9': ['041.4', '599.0'], 'ICD-10': ['N39.0', 'B96.4'], 'Bacteria': 'Proteus mirabilis'},
    {'ICD-9': ['008.43', '041.85'], 'ICD-10': ['A04.5', 'B96.82'], 'Bacteria': 'Campylobacter jejuni'},
    {'ICD-9': ['020.0', '020.1', '020.2'], 'ICD-10': ['A20.0', 'A20.1', 'A20.2'], 'Bacteria': 'Yersinia pestis'}
]
# gram_negative_icd_info

In [ ]:
# Create lists of ICD-9 and ICD-10 codes for gram-positive and gram-negative bacteria
gram_positive_icd_9 = [code for entry in gram_positive_icd_info for code in entry['ICD-9']]
gram_positive_icd_10 = [code for entry in gram_positive_icd_info for code in entry['ICD-10']]

gram_negative_icd_9 = [code for entry in gram_negative_icd_info for code in entry['ICD-9']]
gram_negative_icd_10 = [code for entry in gram_negative_icd_info for code in entry['ICD-10']]

# Filter gram_positive_data
gram_positive_data = gram_positive_data[0][gram_positive_data[0]['source_concept_code'].isin(gram_positive_icd_9 + gram_positive_icd_10)]
# gram_positive_data


In [ ]:
# Filter gram_negative_data_filtered
gram_negative_data_filtered = gram_negative_data_filtered[gram_negative_data_filtered['source_concept_code'].isin(gram_negative_icd_9 + gram_negative_icd_10)]
# gram_negative_data_filtered

In [ ]:
gram_positive_data_filtered =gram_positive_data.copy()
gram_negative_data = gram_negative_data_filtered.copy()
gram_positive_unique_person_ids = gram_positive_data['person_id'].nunique()
gram_negative_unique_person_ids = gram_negative_data_filtered['person_id'].nunique()

In [ ]:
def read_ecoli_data():
    # Define the SQL query
    dataset_11549975_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_concept_id IN (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (
                                        192815, 260430, 36715935, 40487064, 4070038, 4071727, 4116491, 4345206, 4345207, 4345208, 4345354, 4345355, 440320, 45757389, 45757690
                                    )       
                                    AND full_text LIKE '%_rank1]%'      
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1
                            )
                    )  
                    AND (
                        c_occurrence.PERSON_ID IN (
                            SELECT
                                distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_ehr_data = 1 
                                ) 
                            )
                    )
                ) c_occurrence 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                    ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                    ON c_occurrence.condition_type_concept_id = c_type.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                    ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                    ON v.visit_concept_id = visit.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                    ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                    ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    # Execute the SQL query and return the DataFrame
    ecoli_df = pd.read_gbq(
        dataset_11549975_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
    return ecoli_df
# Call the function to read E. coli data
ecoli_data = read_ecoli_data()
# Filter the E. coli data based on specific conditions
ecoli_data = ecoli_data.dropna(subset=['source_vocabulary','source_concept_name'])

ecoli_data_filtered = ecoli_data[ecoli_data['source_vocabulary'].str.contains('ICD9CM|ICD10CM', case=False)]
ecoli_data_filtered = ecoli_data_filtered[ecoli_data_filtered['source_concept_name'].str.contains('Escherichia coli', case=False)]

# Extract information about the E. coli dataset
ecoli_unique_person_ids = ecoli_data_filtered['person_id'].nunique()
ecoli_unique_concept_codes = ecoli_data_filtered['source_concept_code'].nunique()
ecoli_unique_codes_list = ecoli_data_filtered['source_concept_code'].unique()
ecoli_data=ecoli_data_filtered.copy()
ecoli_unique_name_list =ecoli_data_filtered['source_concept_name'].unique()
# print("Unique source_concept_name:", ', '.join(ecoli_unique_name_list))
# # Display the E. coli data and information
# print("Escherichia coli dataset:")
# print("Number of unique person_ids:", ecoli_unique_person_ids)
# print("Number of unique source_concept_codes:", ecoli_unique_concept_codes)
# print("Unique source_concept_codes:", ', '.join(ecoli_unique_codes_list))
# print(ecoli_data_filtered)

In [ ]:
def read_staphylococcus_data():
    # Define the SQL query
    dataset_54672821_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_source_concept_id IN (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (
                                        1567453, 35205549, 35205550, 35206024, 35206025, 35206026, 35206027, 35206028, 35206029, 35206030, 35206031, 35206032, 35210490, 44819357, 44820361, 44823143, 44823773, 44823971, 44825094, 44825143, 44825147, 44825468, 44827488, 44829006, 44830920, 44834766, 44835610, 44835978, 45532833, 45533545, 45537779, 45552181, 45557620, 45567261, 45571485, 45576140, 45581112, 45590710, 45591550, 45600413, 45601122
                                    )       
                                    AND full_text LIKE '%_rank1]%'      
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1
                            )
                    )  
                    AND (
                        c_occurrence.PERSON_ID IN (
                            SELECT
                                distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_ehr_data = 1 
                                ) 
                            )
                    )
                ) c_occurrence 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                    ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                    ON c_occurrence.condition_type_concept_id = c_type.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                    ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                    ON v.visit_concept_id = visit.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                    ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                    ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    # Execute the SQL query and return the DataFrame
    staphylococcus_df = pd.read_gbq(
        dataset_54672821_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
    return staphylococcus_df
# Call the function to read staphylococcus data
staphylococcus_data = read_staphylococcus_data()

# Filter the staphylococcus data based on specific conditions
staphylococcus_data = staphylococcus_data[staphylococcus_data['source_vocabulary'].str.contains(('ICD9CM|ICD10CM'), case=False)]
staphylococcus_data = staphylococcus_data[staphylococcus_data['source_concept_name'].str.contains('staphylococcus', case=False)]

# Extract information about the staphylococcus dataset
staphylococcus_unique_person_ids = staphylococcus_data['person_id'].nunique()
staphylococcus_unique_concept_codes = staphylococcus_data['source_concept_code'].nunique()
staphylococcus_unique_codes_list = staphylococcus_data['source_concept_code'].unique()

# # Display the staphylococcus data and information
# print("Staphylococcus dataset:")
# print("Number of unique person_ids:", staphylococcus_unique_person_ids)
# print("Number of unique source_concept_codes:", staphylococcus_unique_concept_codes)
# print("Unique source_concept_codes:", ', '.join(staphylococcus_unique_codes_list))
# staphylococcus_data

In [ ]:
def read_retrovirus_data():
    # Define the SQL query for Retrovirus dataset
    dataset_25209559_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_concept_id IN (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (
                                        196484, 196486, 37019071, 4034870, 4158313, 433694
                                    )       
                                    AND full_text LIKE '%_rank1]%'      
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1
                            ) 
                            OR  condition_source_concept_id IN (
                                SELECT
                                    DISTINCT c.concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                JOIN
                                    (
                                        SELECT
                                            CAST(cr.id as string) AS id       
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                        WHERE
                                            concept_id IN (
                                                35205795, 44819384, 44821693, 44822121, 44828699, 44832092, 44832093, 44836099, 45539139, 45543962, 45553515, 45561777, 45566568, 45601817, 45605254, 45605255
                                            )       
                                            AND full_text LIKE '%_rank1]%'      
                                    ) a 
                                        ON (
                                            c.path LIKE CONCAT('%.',
                                        a.id,
                                        '.%') 
                                        OR c.path LIKE CONCAT('%.',
                                        a.id) 
                                        OR c.path LIKE CONCAT(a.id,
                                        '.%') 
                                        OR c.path = a.id) 
                                    WHERE
                                        is_standard = 0 
                                        AND is_selectable = 1
                                    )
                            )  
                            AND (
                                c_occurrence.PERSON_ID IN (
                                    SELECT
                                        distinct person_id  
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                                    WHERE
                                        cb_search_person.person_id IN (
                                            SELECT
                                                person_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                            WHERE
                                                has_ehr_data = 1 
                                        ) 
                                    )
                            )
                        ) c_occurrence 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                            ON v.visit_concept_id = visit.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    # Execute the SQL query and return the DataFrame
    retrovirus_df = pd.read_gbq(
        dataset_25209559_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    # Select desired columns
    selected_columns = ['person_id', 'source_concept_code', 'source_concept_name', 'source_vocabulary', 'condition_start_datetime']

    # Create a boolean mask to filter out rows with NaN or NA values in the 'source_vocabulary' column
    mask = retrovirus_df['source_vocabulary'].notna()

    # Apply the boolean mask to filter the DataFrame
    retrovirus_filtered = retrovirus_df[selected_columns][mask]

    # Count the number of unique person_ids and unique source_concept_codes for Retrovirus dataset
    retrovirus_unique_person_ids = retrovirus_filtered['person_id'].nunique()
    retrovirus_unique_concept_codes = retrovirus_filtered['source_concept_code'].nunique()
    retrovirus_unique_codes_list = retrovirus_filtered['source_concept_code'].unique()

    # Construct the message
    retrovirus_message = f"Retrovirus dataset:\nNumber of unique person_ids: {retrovirus_unique_person_ids}\nNumber of unique source_concept_codes: {retrovirus_unique_concept_codes}\nUnique source_concept_codes: {', '.join(retrovirus_unique_codes_list)}"

    # Return the filtered DataFrame and message
    return retrovirus_filtered, retrovirus_message, retrovirus_unique_person_ids, retrovirus_unique_concept_codes, retrovirus_unique_codes_list
retrovirus_data = read_retrovirus_data()
# retrovirus_data

In [ ]:
# Filter the Retrovirus data based on specific conditions
retrovirus_data_filtered = retrovirus_data[0].dropna(subset=['source_vocabulary', 'source_concept_name'])
retrovirus_data_filtered = retrovirus_data_filtered[retrovirus_data_filtered['source_vocabulary'].str.contains('ICD9CM|ICD10CM', case=False)]
# retrovirus_data_filtered = retrovirus_data_filtered[retrovirus_data_filtered['source_concept_name'].str.contains('Retro', case=False)]

# Extract information about the Retrovirus dataset
retrovirus_unique_person_ids = retrovirus_data_filtered['person_id'].nunique()
retrovirus_unique_concept_codes = retrovirus_data_filtered['source_concept_code'].nunique()
retrovirus_unique_codes_list = retrovirus_data_filtered['source_concept_code'].unique()
retrovirus_unique_name_list =retrovirus_data_filtered['source_concept_name'].unique()
# print("Unique source_concept_name:", ', '.join(retrovirus_unique_name_list))
# # Display the Retrovirus data and information
# print("Retrovirus dataset:")
# print("Number of unique person_ids:", retrovirus_unique_person_ids)
# print("Number of unique source_concept_codes:", retrovirus_unique_concept_codes)
# print("Unique source_concept_codes:", ', '.join(retrovirus_unique_codes_list))

# print(retrovirus_data_filtered)
retrovirus_data=retrovirus_data_filtered.copy()

In [ ]:
def read_data_information_file(file_path):
    try:
        with open(file_path, "r") as file:
            data = file.read()
        return data
    except FileNotFoundError:
        return "File not found. Please make sure the file path is correct."
def read_pseudomonas_data():
    # Define the SQL query to retrieve the Pseudomonas dataset
    dataset_13876108_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_concept_id IN (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (
                                        195462, 37016370, 4008261, 4145468, 437492, 438064, 443542
                                    )       
                                    AND full_text LIKE '%_rank1]%'      
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1
                            ) 
                            OR  condition_source_concept_id IN (
                                SELECT
                                    DISTINCT c.concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                JOIN
                                    (
                                        SELECT
                                            CAST(cr.id as string) AS id       
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                        WHERE
                                            concept_id IN (
                                                44827490, 44834387
                                            )       
                                            AND full_text LIKE '%_rank1]%'      
                                    ) a 
                                        ON (
                                            c.path LIKE CONCAT('%.',
                                        a.id,
                                        '.%') 
                                        OR c.path LIKE CONCAT('%.',
                                        a.id) 
                                        OR c.path LIKE CONCAT(a.id,
                                        '.%') 
                                        OR c.path = a.id) 
                                    WHERE
                                        is_standard = 0 
                                        AND is_selectable = 1
                                    )
                            )
                        ) c_occurrence 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                            ON v.visit_concept_id = visit.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    # Read the Pseudomonas dataset from BigQuery
    dataset_13876108_condition_df = pd.read_gbq(
        dataset_13876108_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook"
    )

    # Select desired columns
    selected_columns = ['person_id', 'source_concept_code', 'source_concept_name', 'source_vocabulary', 'condition_start_datetime']

    # Create a boolean mask to filter out rows with NaN or NA values in the 'source_vocabulary' column
    mask = dataset_13876108_condition_df['source_vocabulary'].notna()

    # Apply the boolean mask to filter the DataFrame
    pseudomonas = dataset_13876108_condition_df[selected_columns][mask]

    # Count the number of unique person_ids and unique source_concept_codes for Pseudomonas dataset
    pseudomonas_unique_person_ids = pseudomonas['person_id'].nunique()
    pseudomonas_unique_concept_codes = pseudomonas['source_concept_code'].nunique()
    pseudomonas_unique_codes_list = pseudomonas['source_concept_code'].unique()

    # Create a message about Pseudomonas dataset
    pseudomonas_message = f"\nPseudomonas dataset:\nNumber of unique person_ids: {pseudomonas_unique_person_ids}\nNumber of unique source_concept_codes: {pseudomonas_unique_concept_codes}\nUnique source_concept_codes: {', '.join(pseudomonas_unique_codes_list)}"

    return pseudomonas, pseudomonas_message, pseudomonas_unique_person_ids, pseudomonas_unique_concept_codes, pseudomonas_unique_codes_list
def read_alzheimers_data():
    # Define the SQL query to retrieve the Alzheimer's Disease dataset
    dataset_44170505_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_concept_id IN (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (
                                        35608576, 37018688, 37109056, 37110513, 37117145, 372241, 37312035, 374888, 375791, 376085, 376095, 376946, 377254, 377527, 377788, 378125, 378419, 378726, 379778, 379784, 380701, 380986, 381832, 4009647, 4043378, 4046090, 4047745, 4047747, 40483103, 4048875, 4092747, 4101137, 4103534, 4139421, 4182210, 4196433, 4218017, 4220313, 4228133, 42538857, 4277444, 4278830, 43021816, 43530664, 43530666, 441002, 441535, 443605, 443790, 443864, 444091, 44782432, 44782763, 44782771, 762497
                                    )       
                                    AND full_text LIKE '%_rank1]%'      
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1
                            ) 
                            OR  condition_source_concept_id IN (
                                SELECT
                                    DISTINCT c.concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                JOIN
                                    (
                                        SELECT
                                            CAST(cr.id as string) AS id       
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                        WHERE
                                            concept_id IN (
                                                1568088, 1568090, 1568293, 1568295, 35207114, 35207356, 35207357, 35207358, 35207359, 44819534, 44819535, 44820708, 44820709, 44821810, 44821811, 44821814, 44824105, 44824106, 44824152, 44826489, 44826537, 44827641, 44827644, 44829914, 44829915, 44829917, 44831078, 44831079, 44831083, 44831122, 44832219, 44833435, 44834581, 44835772, 44835773, 44835825, 44836954, 44836959, 45538000, 45538103, 45547675, 45547690, 45547730, 45552458, 45566776, 45591073, 45591076, 45595842, 45595843, 45600684, 45605533
                                            )       
                                            AND full_text LIKE '%_rank1]%'      
                                    ) a 
                                        ON (
                                            c.path LIKE CONCAT('%.',
                                        a.id,
                                        '.%') 
                                        OR c.path LIKE CONCAT('%.',
                                        a.id) 
                                        OR c.path LIKE CONCAT(a.id,
                                        '.%') 
                                        OR c.path = a.id) 
                                    WHERE
                                        is_standard = 0 
                                        AND is_selectable = 1
                                    )
                            )
                        ) c_occurrence 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                            ON v.visit_concept_id = visit.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    # Read the Alzheimer's Disease data from BigQuery
    alzheimer_df = pd.read_gbq(
        dataset_44170505_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    # Select desired columns
    selected_columns = ['person_id', 'source_concept_code', 'source_concept_name', 'source_vocabulary', 'condition_start_datetime']

    # Create a boolean mask to filter out rows with NaN or NA values in the 'source_vocabulary' column
    mask = alzheimer_df['source_vocabulary'].notna()

    # Apply the boolean mask to filter the DataFrame
    alzheimer_filtered = alzheimer_df[selected_columns][mask]

    # Count the number of unique person_ids and unique source_concept_codes for Alzheimer's dataset
    alzheimer_unique_person_ids = alzheimer_filtered['person_id'].nunique()
    alzheimer_unique_concept_codes = alzheimer_filtered['source_concept_code'].nunique()
    alzheimer_unique_codes_list = alzheimer_filtered['source_concept_code'].unique()

    # Construct the message
    alzheimer_message = f"Alzheimer's dataset:\nNumber of unique person_ids: {alzheimer_unique_person_ids}\nNumber of unique source_concept_codes: {alzheimer_unique_concept_codes}\nUnique source_concept_codes: {', '.join(alzheimer_unique_codes_list)}"

    # Return the filtered DataFrame and message
    return alzheimer_filtered, alzheimer_message, alzheimer_unique_person_ids, alzheimer_unique_concept_codes, alzheimer_unique_codes_list

In [ ]:
def read_streptococcus_data():
    # Define the SQL query
    dataset_50428675_condition_sql = """
        SELECT
            c_occurrence.person_id,
            c_occurrence.condition_concept_id,
            c_standard_concept.concept_name as standard_concept_name,
            c_standard_concept.concept_code as standard_concept_code,
            c_standard_concept.vocabulary_id as standard_vocabulary,
            c_occurrence.condition_start_datetime,
            c_occurrence.condition_end_datetime,
            c_occurrence.condition_type_concept_id,
            c_type.concept_name as condition_type_concept_name,
            c_occurrence.stop_reason,
            c_occurrence.visit_occurrence_id,
            visit.concept_name as visit_occurrence_concept_name,
            c_occurrence.condition_source_value,
            c_occurrence.condition_source_concept_id,
            c_source_concept.concept_name as source_concept_name,
            c_source_concept.concept_code as source_concept_code,
            c_source_concept.vocabulary_id as source_vocabulary,
            c_occurrence.condition_status_source_value,
            c_occurrence.condition_status_concept_id,
            c_status.concept_name as condition_status_concept_name 
        FROM
            ( SELECT
                * 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
            WHERE
                (
                    condition_source_concept_id IN (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (
                                        1567453, 35205543, 35205544, 35205545, 35206024, 35206025, 35206026, 35206027, 35206028, 35206029, 35206030, 35206031, 35206032, 35207938, 35207942, 35207959, 44819743, 44820548, 44820886, 44821669, 44821983, 44822816, 44823772, 44825146, 44825467, 44828677, 44829735, 44830730, 44832417, 44833226, 44833607, 44834423, 44835607, 45537779, 45571485, 45572911, 45576141, 45577677, 45582601, 45609916, 766415
                                    )       
                                    AND full_text LIKE '%_rank1]%'      
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1
                            )
                    )  
                    AND (
                        c_occurrence.PERSON_ID IN (
                            SELECT
                                distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        has_ehr_data = 1 
                                ) 
                            )
                    )
                ) c_occurrence 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                    ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                    ON c_occurrence.condition_type_concept_id = c_type.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                    ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                    ON v.visit_concept_id = visit.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                    ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
            LEFT JOIN
                `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                    ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

    # Execute the SQL query and return the DataFrame
    streptococcus_df = pd.read_gbq(
        dataset_50428675_condition_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    
     # Select desired columns
    selected_columns = ['person_id', 'source_concept_code', 'source_concept_name', 'source_vocabulary',"" 'condition_start_datetime']

    # Create a boolean mask to filter out rows with NaN or NA values in the 'source_vocabulary' column
    mask = streptococcus_df['source_vocabulary'].notna()

    # Apply the boolean mask to filter the DataFrame
    streptococcus_filtered =streptococcus_df[selected_columns][mask]

    # Count the number of unique person_ids and unique source_concept_codes for Alzheimer's dataset
    streptococcus_unique_person_ids = streptococcus_filtered['person_id'].nunique()
    streptococcus_unique_concept_codes = streptococcus_filtered['source_concept_code'].nunique()
    streptococcus_unique_codes_list =  streptococcus_filtered['source_concept_code'].unique()

    # Construct the message
    streptococcus_message = f"Streptococcus's dataset:\nNumber of unique person_ids: {streptococcus_unique_person_ids}\nNumber of unique source_concept_codes: {streptococcus_unique_concept_codes}\nUnique source_concept_codes: {', '.join(streptococcus_unique_codes_list)}"

    # Return the filtered DataFrame and message
    return streptococcus_filtered, streptococcus_message, streptococcus_unique_person_ids, streptococcus_unique_concept_codes, streptococcus_unique_codes_list

In [ ]:
# Call the function to read Alzheimer's Disease data
streptococcus_data, streptococcus_info, streptococcus_unique_person_ids, streptococcus_unique_concept_codes, streptococcus_unique_codes_list = read_streptococcus_data()
streptococcus_data = streptococcus_data[streptococcus_data['source_concept_name'].str.contains('Streptoco', case=False)]
streptococcus_data = streptococcus_data[streptococcus_data['source_vocabulary'].str.contains(('ICD9CM|ICD10CM'), case=False)]

# # Display the Alzheimer's Disease data and information
# print(streptococcus_info)
# print(streptococcus_data)
# print("Unique Person IDs:", streptococcus_unique_person_ids)
# print("Unique Concept Codes:", streptococcus_unique_concept_codes)
# print("Unique Codes List:", streptococcus_unique_codes_list)

In [ ]:
# Call the function to read Pseudomonas data
pseudomonas_data, pseudomonas_info, pseudomonas_unique_person_ids, pseudomonas_unique_concept_codes, pseudomonas_unique_codes_list = read_pseudomonas_data()
pseudomonas_data = pseudomonas_data[pseudomonas_data['source_concept_name'].str.contains('Pseu', case=False)]
pseudomonas_data = pseudomonas_data[pseudomonas_data['source_vocabulary'].str.contains(('ICD9CM|ICD10CM'), case=False)]
# # Display the Pseudomonas data and information
# print(pseudomonas_info)
# print(pseudomonas_data)
# print("Unique Person IDs:", pseudomonas_unique_person_ids)
# print("Unique Concept Codes:", pseudomonas_unique_concept_codes)
# print("Unique Codes List:", pseudomonas_unique_codes_list)
# pseudomonas_unique_name_list =pseudomonas_data['source_concept_name'].unique()
# print("Unique source_concept_name:", ', '.join(pseudomonas_unique_name_list))
# print(pseudomonas_data)

In [ ]:
# Call the function to read Alzheimer's Disease data
alzheimer_data, alzheimer_info, alzheimer_unique_person_ids, alzheimer_unique_concept_codes, alzheimer_unique_codes_list = read_alzheimers_data()

alzheimer_data = alzheimer_data[alzheimer_data['source_vocabulary'].str.contains(('ICD9CM|ICD10CM'), case=False)]
postconcussion_data = alzheimer_data[alzheimer_data['source_concept_name'].str.contains('Post|concussion',case=False)]
postconcussion_data
# alzheimer_data = alzheimer_data[alzheimer_data['source_concept_name'].str.contains('Alz', case=False)]
alzheimer_data = alzheimer_data[alzheimer_data['source_concept_name'].str.contains('Alz|Demen', case=False)]
alzheimer_data = alzheimer_data[~alzheimer_data['source_concept_name'].str.contains('Postconcussion','Postconcussional')]
alzheimer_data
alzheimer_unique_name_list =alzheimer_data['source_concept_name'].unique()
# # Display the Alzheimer's Disease data and information
# print(alzheimer_info)
# print(alzheimer_data)
# print("Unique Person IDs:", alzheimer_unique_person_ids)
# print("Unique Concept Codes:", alzheimer_unique_concept_codes)
# print("Unique Codes List:", alzheimer_unique_codes_list)
# print("Unique source_concept_name:", ', '.join(alzheimer_unique_name_list))

## check what phecodes we have in each dataset

In [ ]:
def save_top_concepts_to_csv(disease, unique_person_ids, person_information):
    # Filter conditions_common DataFrame for only the person IDs present in unique_person_ids
    filtered_conditions = person_information[person_information['person_id'].isin(unique_person_ids)]

    # Group by source_concept_code and source_concept_name, count unique person IDs for each concept, and sort by count
    top_concepts = filtered_conditions.groupby(['source_concept_code', 'source_concept_name'])['person_id'].nunique().reset_index(name='unique_ids_count').sort_values(by='unique_ids_count', ascending=False)

    # Select the top 100 most common concepts
    top_100_concepts = top_concepts.head(100)

    # Reset the index of the top 100 concepts DataFrame
    top_100_concepts.reset_index(drop=True, inplace=True)

    # Replace counts less than 20 with "<20"
    top_100_concepts['unique_ids_count'] = top_100_concepts['unique_ids_count'].apply(lambda x: '<20' if x < 20 else x)

    # Create a new DataFrame with the modified counts
    new_df = top_100_concepts.copy()

    # Save the new DataFrame to a CSV file
#     new_df.to_csv(f'./download/{disease}_top_100_concepts.csv', index=False)

disease = 'gram_negative'
gram_negative_unique_person_ids = gram_negative_data['person_id'].unique().tolist()
save_top_concepts_to_csv(disease, gram_negative_unique_person_ids, person_information)
disease = 'gram_negative_filtered'
gram_negative_unique_person_ids = gram_negative_data_filtered['person_id'].unique().tolist()
save_top_concepts_to_csv(disease, gram_negative_unique_person_ids, person_information)

disease = 'staphylococcus'
staphylococcus_unique_person_ids = staphylococcus_data['person_id'].unique().tolist()
save_top_concepts_to_csv(disease, staphylococcus_unique_person_ids, person_information)

disease='streptococcus'
streptococcus_unique_person_ids =streptococcus_data['person_id'].unique().tolist()
save_top_concepts_to_csv(disease, streptococcus_unique_person_ids, person_information)

disease= 'pseudomonas'
pseudomonas_unique_person_ids = pseudomonas_data['person_id'].unique().tolist()
save_top_concepts_to_csv(disease, pseudomonas_unique_person_ids, person_information)

disease='alzheimer'
alzheimer_unique_person_ids = alzheimer_data['person_id'].unique().tolist()
save_top_concepts_to_csv(disease, alzheimer_unique_person_ids, person_information)

disease = 'ecoli'
ecoli_unique_person_ids = ecoli_data['person_id'].unique().tolist()
save_top_concepts_to_csv(disease, ecoli_unique_person_ids, person_information)
disease = 'gram_positive'
gram_positive_unique_person_ids = gram_positive_data['person_id'].unique().tolist()
# gram_positive_unique_person_ids = gram_positive_data[0]['person_id'].unique().tolist()
save_top_concepts_to_csv(disease, gram_positive_unique_person_ids, person_information)
retrovirus_unique_person_ids = retrovirus_data['person_id'].unique().tolist()

## create dataframes that have the earliest onset age onset for each of the Infection or disease

In [ ]:
# Define a custom aggregation function to select the row with the oldest condition_start_datetime
def oldest_condition_start_datetime(group):
    oldest_row_index = group['condition_start_datetime'].idxmin()
    return group.loc[oldest_row_index]

# Group by 'person_id' and aggregate to keep the oldest condition_start_datetime and other columns
ecoli_aggregated_df = ecoli_data.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
# ecoli_aggregated_df
# Group by 'person_id' and aggregate to keep the oldest condition_start_datetime and other columns
retrovirus_aggregated_df = retrovirus_data_filtered.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
# retrovirus_aggregated_df
# Group by 'person_id' and aggregate to keep the oldest condition_start_datetime and other columns
gram_negative_aggregated_df = gram_negative_data_filtered.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
# gram_negative_aggregated_df

# Group by 'person_id' and aggregate to keep the oldest condition_start_datetime and other columns
streptococcus_aggregated_df = streptococcus_data.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
# streptococcus_aggregated_df

# Group by 'person_id' and aggregate to keep the oldest condition_start_datetime and other columns
pseudomonas_aggregated_df = pseudomonas_data.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
# pseudomonas_aggregated_df

# Group by 'person_id' and aggregate to keep the oldest condition_start_datetime and other columns
# aggregated_df = alzheimer_data.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
alzheimer_aggregated_df = alzheimer_data.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
# alzheimer_aggregated_df

# Group by 'person_id' and aggregate to keep the oldest condition_start_datetime and other columns
staphylococcus_aggregated_df = staphylococcus_data.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
staphylococcus_aggregated_df = staphylococcus_aggregated_df[['person_id', 'source_concept_code', 'source_concept_name', 'source_vocabulary', 'condition_start_datetime']]
# staphylococcus_aggregated_df

gram_positive_aggregated_df = gram_positive_data.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
# gram_positive_aggregated_df
# Group by 'person_id' and aggregate to keep the oldest condition_start_datetime and other columns
retrovirus_aggregated_df = retrovirus_data_filtered.groupby('person_id').apply(oldest_condition_start_datetime).reset_index(drop=True)
# retrovirus_aggregated_df

## Find the patients that have the Diseases and what PheCodes they have 

In [ ]:
disease_person_ids = {
    'staphylococcus': staphylococcus_unique_person_ids,
    'alzheimer': alzheimer_unique_person_ids,
    'pseudomonas': pseudomonas_unique_person_ids,
    'streptococcus': streptococcus_unique_person_ids,
    'ecoli': ecoli_unique_person_ids,
    'gram_negative' : gram_negative_unique_person_ids,
    'gram_positive': gram_positive_unique_person_ids,
    'retrovirus': retrovirus_unique_person_ids

}

In [ ]:
# Create an empty set to store unique person IDs
unique_person_ids = set()

# Iterate over each disease and update the set of unique person IDs
for disease, person_ids in disease_person_ids.items():
    # Ensure person_ids is iterable by converting to a set
    if isinstance(person_ids, (int, float)):  # If it's a single value
        person_ids = {person_ids}  # Convert it to a set
    unique_person_ids.update(person_ids)  # Update the set with person_ids

# Create a dataframe with one column containing unique person IDs
unique_ids = pd.DataFrame({'person_id': list(unique_person_ids)})

# # Display the dataframe
# print(unique_ids)

In [ ]:
diseases = ['alzheimer', 'gram_negative', 'staphylococcus',  'pseudomonas', 'streptococcus','ecoli', 'gram_positive',"retrovirus"]

In [ ]:
# Initialize a dictionary to store the count of unique IDs for each disease
unique_ids_count = {}

# Iterate over the diseases
for disease in diseases:
    # Get the list of unique person IDs for the current disease
    unique_person_ids = disease_person_ids.get(disease, [])
    
    # Ensure unique_person_ids is iterable
    if isinstance(unique_person_ids, (int, float)):  # If it's a single value
        unique_person_ids = {unique_person_ids}  # Convert it to a set
    
    # Count the number of unique IDs
    unique_count = len(set(unique_person_ids))
    
    # Store the count in the dictionary
    unique_ids_count[disease] = unique_count

# Print the counts
for disease, count in unique_ids_count.items():
    print(f"{disease}: {count} unique IDs")


## Create a dataframe with the unique codes for each disease

In [ ]:
# Initialize an empty dictionary to store unique source_concept_codes
unique_codes_dict = {}

# List of diseases
diseases = ['alzheimer', 'gram_negative', 'staphylococcus',  'pseudomonas', 'streptococcus','ecoli', 'gram_positive',"retrovirus"]
# Iterate over each disease and its corresponding dataframe
for disease in diseases:
    data = globals()[f"{disease}_aggregated_df"]  # Accessing dataframes dynamically
    unique_codes = data['source_concept_code'].unique()
    unique_codes_dict[disease] = unique_codes

# Find the maximum number of unique codes
max_length = max(len(codes) for codes in unique_codes_dict.values())

# Fill in None values for diseases with fewer unique codes
for disease, codes in unique_codes_dict.items():
    if len(codes) < max_length:
        unique_codes_dict[disease] = list(codes) + [None] * (max_length - len(codes))

# Create a dataframe from the dictionary
unique_codes_df = pd.DataFrame(unique_codes_dict)
# Map the column names to their corresponding dataframes
aggregated_dfs = {
    'alzheimer': alzheimer_aggregated_df,
    'gram_negative': gram_negative_aggregated_df,
    'staphylococcus': staphylococcus_aggregated_df,
    'pseudomonas': pseudomonas_aggregated_df,
    'streptococcus': streptococcus_aggregated_df,
    'ecoli': ecoli_aggregated_df,
    'gram_positive': gram_positive_aggregated_df,
    'retrovirus': retrovirus_aggregated_df
}

# Iterate through each column and match codes to meanings
for column, dataframe in aggregated_dfs.items():
    # Create a dictionary mapping codes to their meanings
    code_to_meaning = dataframe.set_index('source_concept_code')['source_concept_name'].to_dict()
    
    # Map the meanings to the unique_codes_df
    unique_codes_df[f"{column}_ICD9_10_meaning"] = unique_codes_df[column].map(code_to_meaning)

# Display the resulting dataframe
print(unique_codes_df)
unique_codes_df

In [ ]:
# sabe the dataframe 
unique_codes_df.to_csv('./download/unique_codes_df.tsv', sep='\t', index=False)

## Merge the dataframes to enrich the information of the patientts

In [ ]:
# Merge person_df_all with person_information on person_id
merged_person = pd.merge(person_df_all[['person_id', 'date_of_birth','sex_at_birth']], person_information, on='person_id', how='inner')

# Convert date_of_birth to datetime
merged_person['date_of_birth'] = pd.to_datetime(merged_person['date_of_birth'])

# Calculate age_onset by subtracting date_of_birth from condition_start_datetime
merged_person['age_onset'] = (merged_person['condition_start_datetime'] - merged_person['date_of_birth']).dt.days / 365.25

merged_person =merged_person[['person_id', 'date_of_birth', 
        'standard_concept_name',  'condition_start_datetime', 'condition_source_value', 
        'source_concept_name', 'source_concept_code', 'source_vocabulary', 'age_onset']]
merged_person.columns

In [ ]:
# merged_person

# Keep data with EHR more than 10

In [ ]:
# Count the number of entries for each person_id
count_person_entries = person_information['person_id'].value_counts()

# Filter out the person_ids that have more than 10 entries
person_ids_more_than_10 = count_person_entries[count_person_entries > 10].index.tolist()

# Filter the person_df_all DataFrame using the list of person_ids
merged_person_df = person_df_all[person_df_all['person_id'].isin(person_ids_more_than_10)].copy()

# print("Filtered merged_person_df:")
# print(merged_person_df)

## Create columns if they have or not the Disease (1=Have, 0 = Not have)

In [ ]:
# Add columns for each disease and initialize to 0
for disease in diseases:
    merged_person_df[disease] = 0

# Update the disease columns based on the unique IDs
for disease, unique_ids in disease_person_ids.items():
    merged_person_df.loc[merged_person_df['person_id'].isin(unique_ids), disease] = 1

# # Display the new dataframe
# print(merged_person_df.head())

In [ ]:
# merged_person_df

# set the disease name and start the analysis using all Patients

In [ ]:
di='Dementia'

## Filter the data for the disease

In [ ]:
# Define the aggregated dataframes
aggregated_dataframes = {
    "gram_positive_aggregated_df": gram_positive_aggregated_df,
    "staphylococcus_aggregated_df": staphylococcus_aggregated_df,
    "alzheimer_aggregated_df": alzheimer_aggregated_df,
    "pseudomonas_aggregated_df": pseudomonas_aggregated_df,
    "streptococcus_aggregated_df": streptococcus_aggregated_df,
    "gram_negative_aggregated_df": gram_negative_aggregated_df,
    "ecoli_aggregated_df": ecoli_aggregated_df,
    "retrovirus_aggregated_df": retrovirus_aggregated_df
}
# Merge each dataframe with `person_df_all` to add `date_of_birth`
dataframes_with_dob = {
    name: df.merge(person_df_all[['person_id', 'date_of_birth']], on="person_id", how="left")
    for name, df in aggregated_dataframes.items()
}
# Define the columns to keep in all dataframes
columns_to_keep = [
    "person_id", "condition_start_datetime", "source_concept_name",
    "source_concept_code", "source_vocabulary", "date_of_birth"
]
# Keep only the desired columns in each dataframe
cleaned_dataframes_with_selected_columns = {
    name: df[columns_to_keep]
    for name, df in dataframes_with_dob.items()
    if all(col in df.columns for col in columns_to_keep)
}

# Fix `date_of_birth` column conflicts if necessary
dataframes_with_dob_fixed = {
    name: df.rename(columns={"date_of_birth_y": "date_of_birth"}).drop(columns=["date_of_birth_x"], errors="ignore")
    for name, df in dataframes_with_dob.items()
}
# Ensure all desired columns are present
cleaned_dataframes_with_selected_columns_fixed = {
    name: df[columns_to_keep]
    for name, df in dataframes_with_dob_fixed.items()
    if all(col in df.columns for col in columns_to_keep)
}
# Calculate the `age_onset` column for each dataframe
cleaned_dataframes_with_age_onset = {
    name: df.assign(age_onset=(
            (pd.to_datetime(df["condition_start_datetime"]) - pd.to_datetime(df["date_of_birth"])).dt.days / 365.25
    ).round(2))
    for name, df in cleaned_dataframes_with_selected_columns_fixed.items()
    if "condition_start_datetime" in df.columns and "date_of_birth" in df.columns
}
# Assign cleaned dataframes for future analysis
ecoli_df = cleaned_dataframes_with_age_onset["ecoli_aggregated_df"]
alzheimer_df = cleaned_dataframes_with_age_onset["alzheimer_aggregated_df"]
gram_negative_df = cleaned_dataframes_with_age_onset["gram_negative_aggregated_df"]
staphylococcus_df = cleaned_dataframes_with_age_onset["staphylococcus_aggregated_df"]
pseudomonas_df = cleaned_dataframes_with_age_onset["pseudomonas_aggregated_df"]
streptococcus_df = cleaned_dataframes_with_age_onset["streptococcus_aggregated_df"]
gram_positive_df = cleaned_dataframes_with_age_onset["gram_positive_aggregated_df"]
retrovirus_df = cleaned_dataframes_with_age_onset["retrovirus_aggregated_df"]

# Define a function to add `age_onset_{disease}` columns to the filtered dataframe
def add_age_onset_columns(filtered_df, disease_dfs):
    """
    Add age_onset columns for each disease to the filtered dataframe.
    
    Parameters:
        filtered_df (DataFrame): The main filtered dataframe.
        disease_dfs (dict): Dictionary of disease dataframes with age_onset values.

    Returns:
        DataFrame: Updated dataframe with new age_onset columns.
    """
    for disease, df in disease_dfs.items():
        if disease in ["gram_negative", "gram_positive"]:
            age_onset_col = f"age_onset_{disease.split('_')[0]}_{disease.split('_')[1]}"
        else:
            age_onset_col = f"age_onset_{disease.split('_')[0]}"

        if "age_onset" in df.columns:
            filtered_df[age_onset_col] = filtered_df["person_id"].map(
                df.set_index("person_id")["age_onset"]
            )
        else:
            filtered_df[age_onset_col] = None

    return filtered_df

# Define disease dataframes
disease_dfs = {
    "ecoli": cleaned_dataframes_with_age_onset.get("ecoli_aggregated_df"),
    "alzheimer": cleaned_dataframes_with_age_onset.get("alzheimer_aggregated_df"),
    "gram_negative": cleaned_dataframes_with_age_onset.get("gram_negative_aggregated_df"),
    "staphylococcus": cleaned_dataframes_with_age_onset.get("staphylococcus_aggregated_df"),
    "pseudomonas": cleaned_dataframes_with_age_onset.get("pseudomonas_aggregated_df"),
    "streptococcus": cleaned_dataframes_with_age_onset.get("streptococcus_aggregated_df"),
    "gram_positive": cleaned_dataframes_with_age_onset.get("gram_positive_aggregated_df"),
    "retrovirus": cleaned_dataframes_with_age_onset.get("retrovirus_aggregated_df")
}
columns_to_keep_in_merged = [
    "person_id", "date_of_birth", "sex_at_birth",
    "alzheimer",  "gram_negative", "staphylococcus",
    "pseudomonas", "streptococcus",
    "gram_positive","ecoli", "retrovirus"
]

# # Filter and create the merged dataframe
# filtered_merged_person_df = merged_person_df[columns_to_keep_in_merged]

filtered_merged_person_df = merged_person_df[columns_to_keep_in_merged]
filtered_merged_person_df

# Add `age_onset_{disease}` columns to the filtered dataframe
final_filtered_df = add_age_onset_columns(filtered_merged_person_df, disease_dfs)

# Identify people in `person_df_all` not in `final_filtered_df`
remaining_people = person_df_all[~person_df_all['person_id'].isin(final_filtered_df['person_id'])]

# Create a dataframe for remaining people with default values
remaining_people_with_defaults = remaining_people.copy()
disease_columns = ['alzheimer',  'gram_negative', 'staphylococcus',
                   'pseudomonas', 'streptococcus', 'ecoli', 'gram_positive', 'retrovirus']
for col in disease_columns:
    remaining_people_with_defaults[col] = 0

age_difference_columns = [col for col in final_filtered_df.columns if col.startswith("Age_difference_AD_")]
for col in age_difference_columns:
    remaining_people_with_defaults[col] = np.nan

# Ensure disease columns are strictly binary in final_combined_df
def ensure_binary(df, columns):
    for col in columns:
        df[col] = df[col].fillna(0).astype(int)
    return df

# Merge datasets
final_combined_df = pd.concat([
    ensure_binary(final_filtered_df, disease_columns),
    ensure_binary(remaining_people_with_defaults, disease_columns)
], ignore_index=True)

final_filtered_df = add_age_onset_columns(filtered_merged_person_df, disease_dfs)

# Calculate `Age_difference_AD_{disease}` for each disease
for disease in disease_dfs.keys():
    age_onset_col = f"age_onset_{disease}"
    if age_onset_col in final_filtered_df.columns:
        # Calculate the age difference between Alzheimer's and the disease
        final_filtered_df[f"Age_difference_AD_{disease}"] = (
                final_filtered_df["age_onset_alzheimer"] - final_filtered_df[age_onset_col]
        )

# Identify people in `person_df_all` not in `final_filtered_df`
remaining_people = person_df_all[~person_df_all['person_id'].isin(final_filtered_df['person_id'])]

# Create a dataframe for remaining people with default values
remaining_people_with_defaults = remaining_people.copy()
disease_columns = ['alzheimer',  'gram_negative', 'staphylococcus',
                   'pseudomonas', 'streptococcus', 'ecoli', 'gram_positive', 'retrovirus']
for col in disease_columns:
    remaining_people_with_defaults[col] = 0

# Add `Age_difference_AD_{disease}` columns with NaN for remaining people
for disease in disease_dfs.keys():
    remaining_people_with_defaults[f"Age_difference_AD_{disease}"] = np.nan

# Merge datasets
final_combined_df = pd.concat([
    ensure_binary(final_filtered_df, disease_columns),
    ensure_binary(remaining_people_with_defaults, disease_columns)
], ignore_index=True)

# Standardize `date_of_birth` to timezone-naive
final_combined_df['date_of_birth'] = pd.to_datetime(final_combined_df['date_of_birth']).dt.tz_localize(None)

# Calculate current age
# final_combined_df['age'] = (pd.Timestamp.now().tz_localize(None) - final_combined_df['date_of_birth']).dt.days / 365.25
# Define the cutoff date
cutoff_date = pd.Timestamp('2022-07-01')

# Calculate age as of the cutoff date
final_combined_df['age'] = (cutoff_date - final_combined_df['date_of_birth']).dt.days / 365.25

# Define age groups
age_bins = [0, 60, 70, 100]
age_labels = ['<60', '60-70', '>70']
final_combined_df['age_group'] = pd.cut(final_combined_df['age'], bins=age_bins, labels=age_labels, include_lowest=True)

# Ensure every value in disease columns is 0 or 1
for col in disease_columns:
    final_combined_df[col] = final_combined_df[col].astype(int)

# # Output final_combined_df
# final_combined_df

# Remove the EHR less than 10 after finding the ages

## If you do want all data commend this part

In [ ]:
# Filter the final_combined_df to keep only the patients with more than 10 entries
final_combined_df = final_combined_df[final_combined_df['person_id'].isin(person_ids_more_than_10)].copy()

# # Output the filtered final_combined_df
# print("Filtered final_combined_df (patients with >10 entries):")
# print(final_combined_df)

In [ ]:
len(final_combined_df)

In [ ]:
final_combined_df.columns

## Choose the CDR age of the min of the dataset (example we want the patient when they gave the data to be more than 60 with richer EHR data)

In [ ]:
# Filtering patients with Alzheimer's disease onset age > 60
final_combined_df = final_combined_df[final_combined_df["age"] > 60]
ad_over_60_df = final_combined_df[final_combined_df["age_onset_alzheimer"] > 0]
# ad_over_60_df

In [ ]:
len(final_combined_df)

# now that we cleaned the data due to Dataframe manipulations it is nice to keep the list of the patients that we are interesting to Research

In [ ]:
# Extract all person_id values into a list
person_id_list = final_combined_df['person_id'].tolist()
print(f'The length of those that pass the conditions we set are: {len(person_id_list)}')
#___________- make a list with our creteria!!!!!__________#

## Enrichment Analysis if those patients with Dementia/or AD have enrichment in the diseases or the infections.

In [ ]:
# from statsmodels.stats.multitest import multipletests

# Filtering patients with Alzheimer's disease onset age > 60
# ad_over_60_df = final_combined_df[final_combined_df["age_onset_alzheimer"] > 60]

# Defining diseases for enrichment analysis
disease_columns = ['alzheimer', 'gram_negative', 'staphylococcus',
                   'pseudomonas', 'streptococcus', 'ecoli', 'gram_positive','retrovirus']

# Prepare results
results = []

# Perform enrichment analysis using Hail's fisher_exact_test
for disease in disease_columns:
    # Create contingency table values
    ad_over_60_with_disease = int(ad_over_60_df[disease].sum())
    ad_over_60_without_disease = int(len(ad_over_60_df) - ad_over_60_with_disease)
    rest_with_disease = int(final_combined_df[disease].sum() - ad_over_60_with_disease)
    rest_without_disease = int(len(final_combined_df) - len(ad_over_60_df) - rest_with_disease)
    
    # Print the numbers for the enrichment test
    print(f"{disease}:")
    print(f"AD with disease: {ad_over_60_with_disease}")
    print(f"AD without disease: {ad_over_60_without_disease}")
    print(f"Rest with disease: {rest_with_disease}")
    print(f"Rest without disease: {rest_without_disease}")
    
    # Apply Fisher's Exact Test using Hail
    test_result = hl.eval(
        hl.fisher_exact_test(
            hl.int32(ad_over_60_with_disease),
            hl.int32(ad_over_60_without_disease),
            hl.int32(rest_with_disease),
            hl.int32(rest_without_disease)
        )
    )
    
    # Collect the results
    results.append({
        "Disease": disease,
        f"{di}_With_Disease": ad_over_60_with_disease,
        f"{di}_Without_Disease": ad_over_60_without_disease,
        "Rest_With_Disease": rest_with_disease,
        "Rest_Without_Disease": rest_without_disease,
        "Odds_Ratio": test_result.odds_ratio,
        "P_Value": test_result.p_value,
        "CI_95_Lower": test_result.ci_95_lower,
        "CI_95_Upper": test_result.ci_95_upper
    })

# Convert results to DataFrame
enrichment_results_df = pd.DataFrame(results)

# Save the enrichment results with contingency numbers to a TSV file
enrichment_results_path = f"./download/{di}_enrichment_results_with_numbers_hail.tsv"
enrichment_results_df.to_csv(enrichment_results_path, sep='\t', index=False)
enrichment_results_df
# Save another simplified TSV with only Odds Ratio, CI, and Adjusted P-Value
simplified_results_df = enrichment_results_df[["Disease","P_Value", "Odds_Ratio", "CI_95_Lower", "CI_95_Upper"]]
simplified_results_path = f"./download/{di}_enrichment_results_simplified_hail.tsv"
# simplified_results_df.to_csv(simplified_results_path, sep='\t', index=False)

# Print file paths and display results
print(f"Enrichment results saved to: {enrichment_results_path}")
print(f"Simplified enrichment results saved to: {simplified_results_path}")
enrichment_hail_df=enrichment_results_df.copy()
enrichment_hail_df_all=enrichment_results_df.copy()
# enrichment_hail_df

In [ ]:
# Save another simplified TSV with only Odds Ratio, CI, and Adjusted P-Value
simplified_results_df = enrichment_results_df[["Disease", "P_Value","Odds_Ratio", "CI_95_Lower", "CI_95_Upper"]]
# simplified_results_df

In [ ]:
# Correct the order of diseases in the desired order and relabel appropriately
desired_order = ['ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'staphylococcus', 'streptococcus','retrovirus']
y_labels = [
    "E. coli", "Pseudomonas", "Gram-negative", 
    "Gram-positive", "Staphylococcus", "Streptococcus","Retrovirus"
]

# Sort the dataframe by the desired disease order
sorted_df = enrichment_hail_df.set_index('Disease').loc[desired_order].reset_index()

# Create the forest plot
plt.figure(figsize=(10, 6))

# Extract data for plotting
odds_ratios = sorted_df['Odds_Ratio']
ci_lower = sorted_df['CI_95_Lower']
ci_upper = sorted_df['CI_95_Upper']

# Calculate x-errors for error bars
xerr = [odds_ratios - ci_lower, ci_upper - odds_ratios]

# Plot error bars
y_positions = range(len(sorted_df))
plt.errorbar(odds_ratios, y_positions, xerr=xerr, fmt='o', color='gray', ecolor='black', capsize=3)

# Highlight statistically significant points
colors = ['blue' if p < 0.05 else 'salmon' for p in sorted_df['P_Value']]
for i, color in enumerate(colors):
    plt.scatter(odds_ratios.iloc[i], y_positions[i], color=color, zorder=3)

# Add vertical reference line
plt.axvline(x=1, color='red', linestyle='--', zorder=1)

# Customize axes
plt.yticks(y_positions, y_labels)
plt.xlabel('Odds Ratio')
plt.title(f'Forest Plot: Bacteria enrichment analysis for {di.lower()} with age of patients more than 60')
plt.grid(axis='y', linestyle='--', linewidth=0.5)
# remove top and right spines 
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
# make the bottom and left spines thicker and darker
plt.gca().spines['left'].set_linewidth(2)
plt.gca().spines['bottom'].set_linewidth(2)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['bottom'].set_color('black')

plt.tight_layout()

# Save and show the plot
forest_plot_path = f"./download/enrichment_forest_plot_correct_order_{di}.pdf"
plt.savefig(forest_plot_path)
plt.show()


# Enrichment analysis for All, Male and Female 

In [ ]:
# Function to perform enrichment analysis using Hail's Fisher's exact test
def enrichment_analysis_by_group_hail(df, group_name):
    # Define diseases for enrichment analysis
    disease_columns = ['alzheimer', 'gram_negative', 'staphylococcus',
                       'pseudomonas', 'streptococcus', 'ecoli', 'gram_positive','retrovirus']

    # Filtering patients with Alzheimer's disease onset age > 60
    ad_over_60_df = df[df["age_onset_alzheimer"] > 0]

    # Prepare results
    results = []

    for disease in disease_columns:
        # Create contingency table values
        ad_over_60_with_disease = int(ad_over_60_df[disease].sum())
        ad_over_60_without_disease = int(len(ad_over_60_df) - ad_over_60_with_disease)
        rest_with_disease = int(df[disease].sum() - ad_over_60_with_disease)
        rest_without_disease = int(len(df) - len(ad_over_60_df) - rest_with_disease)

        # Print for debugging
        print(f"{group_name} - {disease}:")
        print(f"AD with disease: {ad_over_60_with_disease}")
        print(f"AD without disease: {ad_over_60_without_disease}")
        print(f"Rest with disease: {rest_with_disease}")
        print(f"Rest without disease: {rest_without_disease}")

        # Skip tests if any value in the contingency table is zero
        if any(val == 0 for val in [ad_over_60_with_disease, ad_over_60_without_disease, rest_with_disease, rest_without_disease]):
            print(f"Skipping {group_name} - {disease} due to insufficient data.")
            continue

        # Apply Fisher's Exact Test using Hail
        test_result = hl.eval(
            hl.fisher_exact_test(
                hl.int32(ad_over_60_with_disease),
                hl.int32(ad_over_60_without_disease),
                hl.int32(rest_with_disease),
                hl.int32(rest_without_disease)
            )
        )

        # Collect the results
        results.append({
            "Group": group_name,
            "Disease": disease,
            f"{group_name}_With_Disease": ad_over_60_with_disease,
            f"{group_name}_Without_Disease": ad_over_60_without_disease,
            "Rest_With_Disease": rest_with_disease,
            "Rest_Without_Disease": rest_without_disease,
            "Odds_Ratio": test_result.odds_ratio,
            "P_Value": test_result.p_value,
            "CI_95_Lower": test_result.ci_95_lower,
            "CI_95_Upper": test_result.ci_95_upper
        })

    return pd.DataFrame(results)

# Perform enrichment analysis for all groups
male_df = final_combined_df[final_combined_df["sex_at_birth"] == "Male"]
female_df = final_combined_df[final_combined_df["sex_at_birth"] == "Female"]

male_results_df = enrichment_analysis_by_group_hail(male_df, "Male")
female_results_df = enrichment_analysis_by_group_hail(female_df, "Female")
all_results_df = enrichment_analysis_by_group_hail(final_combined_df, "All")

# Combine results
combined_results_df = pd.concat([male_results_df, female_results_df, all_results_df])

# Save results for each group
male_results_path = "./download/enrichment_results_male.tsv"
female_results_path = "./download/enrichment_results_female.tsv"
all_results_path = "./download/enrichment_results_all.tsv"

male_results_df.to_csv(male_results_path, sep='\t', index=False)
female_results_df.to_csv(female_results_path, sep='\t', index=False)
all_results_df.to_csv(all_results_path, sep='\t', index=False)

# Save combined results
combined_results_path = "./download/enrichment_results_combined.tsv"
combined_results_df.to_csv(combined_results_path, sep='\t', index=False)

# Simplified results
simplified_results_path = "./download/enrichment_results_simplified.tsv"
simplified_results_df = combined_results_df[["Group", "Disease", "P_Value", "Odds_Ratio", "CI_95_Lower", "CI_95_Upper"]]
simplified_results_df.to_csv(simplified_results_path, sep='\t', index=False)

# Print file paths
print(f"Male results saved to: {male_results_path}")
print(f"Female results saved to: {female_results_path}")
print(f"All population results saved to: {all_results_path}")
print(f"Combined enrichment results saved to: {combined_results_path}")
print(f"Simplified enrichment results saved to: {simplified_results_path}")


# Plot the results of the enrichment

In [ ]:
# import matplotlib.lines as mlines
# from statsmodels.stats.multitest import multipletests
# Define the order of diseases and labels
desired_order = ["ecoli", "pseudomonas", "gram_negative", "gram_positive", "staphylococcus", "streptococcus",'retrovirus']
y_labels = ["E. coli", "Pseudomonas", "Gram-negative", "Gram-positive", "Staphylococcus", "Streptococcus",'Retrovirus']

# Filter and sort results for males, females, and all
sorted_df_male = combined_results_df[combined_results_df["Group"] == "Male"].set_index("Disease").loc[desired_order].reset_index()
sorted_df_female = combined_results_df[combined_results_df["Group"] == "Female"].set_index("Disease").loc[desired_order].reset_index()
sorted_df_all = combined_results_df[combined_results_df["Group"] == "All"].set_index("Disease").loc[desired_order].reset_index()

# Extract data for plotting
# Male
odds_ratios_male = sorted_df_male["Odds_Ratio"]
ci_lower_male = sorted_df_male["CI_95_Lower"]
ci_upper_male = sorted_df_male["CI_95_Upper"]
xerr_male = [odds_ratios_male - ci_lower_male, ci_upper_male - odds_ratios_male]
colors_male = ["cornflowerblue" if p < 0.05 else "salmon" for p in sorted_df_male["P_Value"]]

# Female
odds_ratios_female = sorted_df_female["Odds_Ratio"]
ci_lower_female = sorted_df_female["CI_95_Lower"]
ci_upper_female = sorted_df_female["CI_95_Upper"]
xerr_female = [odds_ratios_female - ci_lower_female, ci_upper_female - odds_ratios_female]
colors_female = ["cornflowerblue" if p < 0.05 else "salmon" for p in sorted_df_female["P_Value"]]

# All
odds_ratios_all = sorted_df_all["Odds_Ratio"]
ci_lower_all = sorted_df_all["CI_95_Lower"]
ci_upper_all = sorted_df_all["CI_95_Upper"]
xerr_all = [odds_ratios_all - ci_lower_all, ci_upper_all - odds_ratios_all]
colors_all = ["cornflowerblue" if p < 0.05 else "salmon" for p in sorted_df_all["P_Value"]]


legend_handles = [
    mlines.Line2D([], [], color="black", marker="^", linestyle="None", markersize=8, label="All population"),
    mlines.Line2D([], [], color="black", marker="o", linestyle="None", markersize=8, label="Female"),
    mlines.Line2D([], [], color="black", marker="s", linestyle="None", markersize=8, label="Male"),
    mlines.Line2D([], [], color="cornflowerblue", marker="o", linestyle="None", markersize=8, label="Significant"),
    mlines.Line2D([], [], color="salmon", marker="o", linestyle="None", markersize=8, label="Not significant"),
    mlines.Line2D([], [], color="red", linestyle="--", markersize=8, label="No significant effect (OR = 1)")
]


# Adjust y_positions to center the labels between the three groups
y_positions = np.arange(len(desired_order)) * 1.2  # Add spacing between groups
y_tick_positions = y_positions + 0.2  # Center the labels

# Plot configurations
plt.figure(figsize=(14, 10))
y_positions = np.arange(len(desired_order))

# Plot Male group (square markers)
plt.errorbar(odds_ratios_male, y_positions, xerr=xerr_male, fmt="s", color="black", ecolor="black", capsize=3, label="Male")
for i, color in enumerate(colors_male):
    plt.scatter(odds_ratios_male.iloc[i], y_positions[i], color=color, marker="s", zorder=3)

# Plot Female group (circle markers)
plt.errorbar(odds_ratios_female, y_positions + 0.2, xerr=xerr_female, fmt="o", color="black", ecolor="black", capsize=3, label="Female")
for i, color in enumerate(colors_female):
    plt.scatter(odds_ratios_female.iloc[i], y_positions[i] + 0.2, color=color, marker="o", zorder=3)

# Plot All group (triangle markers)
plt.errorbar(odds_ratios_all, y_positions + 0.4, xerr=xerr_all, fmt="^", color="black", ecolor="black", capsize=3, label="All Population")
for i, color in enumerate(colors_all):
    plt.scatter(odds_ratios_all.iloc[i], y_positions[i] + 0.4, color=color, marker="^", zorder=3)

# Add vertical reference line
plt.axvline(x=1, color="red", linestyle="--", zorder=1, label="No Effect (OR = 1)")

# Customize axes and legend
plt.yticks(y_positions, y_labels)
plt.xlabel("Odds Ratio")
plt.title("Forest Plot: Enrichment Analysis for Males, Females, and All Population")
plt.grid(axis="y", linestyle="--", linewidth=0.5)
plt.legend(handles=legend_handles, loc="upper right", title="Legend", frameon=False, bbox_to_anchor=(1.2, 1.1))

# Remove top and right spines
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["top"].set_visible(False)

# Make the bottom and left spines thicker and darker
plt.gca().spines["left"].set_linewidth(2)
plt.gca().spines["bottom"].set_linewidth(2)
plt.gca().spines["left"].set_color("black")
plt.gca().spines["bottom"].set_color("black")

plt.tight_layout()

# Save and show the plot
forest_plot_path_combined = f"./download/{di}_combined_enrichment_analysis_forest_plot_updated_legend.pdf"
plt.savefig(forest_plot_path_combined)
plt.show()


# Add spacing between diseases to make each set of error bars visually distinct

In [ ]:
# Add spacing between diseases to make each set of error bars visually distinct
spacing = 1.5  # Adjust this value for more or less space
y_positions = np.arange(len(desired_order)) * spacing  # Add spacing between groups

# Plot configurations
plt.figure(figsize=(14, 10))

# Plot All Population group (triangle markers at the top)
plt.errorbar(odds_ratios_all, y_positions + 0.4, xerr=xerr_all, fmt="^", color="black", ecolor="black", capsize=3, label="All Population")
for i, color in enumerate(colors_all):
    plt.scatter(odds_ratios_all.iloc[i], y_positions[i] + 0.4, color=color, marker="^", zorder=3)

# Plot Female group (circle markers in the middle with y_labels aligned)
plt.errorbar(odds_ratios_female, y_positions, xerr=xerr_female, fmt="o", color="black", ecolor="black", capsize=3, label="Female")
for i, color in enumerate(colors_female):
    plt.scatter(odds_ratios_female.iloc[i], y_positions[i], color=color, marker="o", zorder=3)

# Plot Male group (square markers at the bottom)
plt.errorbar(odds_ratios_male, y_positions - 0.4, xerr=xerr_male, fmt="s", color="black", ecolor="black", capsize=3, label="Male")
for i, color in enumerate(colors_male):
    plt.scatter(odds_ratios_male.iloc[i], y_positions[i] - 0.4, color=color, marker="s", zorder=3)

# Add vertical reference line
plt.axvline(x=1, color="red", linestyle="--", zorder=1, label="No Effect (OR = 1)")

# Customize axes and legend
plt.yticks(y_positions, y_labels)  # Align labels with Female in the middle
plt.xlabel("Odds Ratio",fontsize=16)
plt.title(f"Forest Plot: Bacteria enrichment analysis for males, females, and all population in {di} and age>60")
plt.grid(axis="y", linestyle="--", linewidth=0.5)
plt.legend(handles=legend_handles, loc="upper right", title="Legend", frameon=False, bbox_to_anchor=(1.2, 1.1),fontsize=10)
# make x-labels more readable and bigger
plt.xticks(fontsize=14)
# make y-labels more readable and bigger
plt.yticks(fontsize=14)
# Remove top and right spines
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["top"].set_visible(False)

# Make the bottom and left spines thicker and darker
plt.gca().spines["left"].set_linewidth(2)
plt.gca().spines["bottom"].set_linewidth(2)
plt.gca().spines["left"].set_color("black")
plt.gca().spines["bottom"].set_color("black")

plt.tight_layout()

# Save and show the plot
forest_plot_path_spaced = f"./download/{di}_spaced_combined_enrichment_analysis_forest_plot.pdf"
plt.savefig(forest_plot_path_spaced)
plt.show()


In [ ]:
print(enrichment_results_df.columns)


# Filter disease dataframes based on the Dementia or the Alzheimer's Disease


In [ ]:
alzheimer_data_filtered=alzheimer_data.copy()
alzheimer_data_filtered.columns

# Diseases and their respective dataframes
disease_dataframes = {
    "alzheimer": alzheimer_data_filtered,
    "gram_negative": gram_negative_data_filtered,
    "staphylococcus": staphylococcus_data,
    "pseudomonas": pseudomonas_data,
    "streptococcus": streptococcus_data,
    "ecoli": ecoli_data_filtered,
    "gram_positive": gram_positive_data,
    'retrovirus':retrovirus_data
}

In [ ]:
# Filter disease dataframes based on the person_id list from final_combined_df
person_id_list = final_combined_df["person_id"].tolist()

disease_dataframes_filtered = {
    disease: df[df["person_id"].isin(person_id_list)].copy()
    for disease, df in disease_dataframes.items()
}

# Function to count unique phecodes per month for each disease
def count_unique_phecodes_per_month(data, disease_name):
    """
    Count unique phecodes per person, grouped by month, for a specific disease.
    A person's phecodes in the same month are counted as 1.

    Parameters:
        data (DataFrame): The disease dataframe with person_id, source_concept_code, and condition_start_datetime.
        disease_name (str): Name of the disease for column naming.

    Returns:
        DataFrame: Contains person_id and count of unique months with phecodes for this disease.
    """
    # Ensure condition_start_datetime is in datetime format
    data["condition_start_datetime"] = pd.to_datetime(data["condition_start_datetime"])

    # Group by person_id and month-year (as 'year_month')
    data["year_month"] = data["condition_start_datetime"].dt.to_period("M")

    # Group by person_id and year_month, then count unique phecodes (source_concept_code)
    grouped = (
        data.groupby(["person_id", "year_month"])["source_concept_code"]
        .nunique()  # Count unique phecodes per month
        .reset_index()
    )

    # For each person, count the total number of months with phecodes
    person_month_count = (
        grouped.groupby("person_id")["year_month"]
        .count()  # Count unique months
        .reset_index()
        .rename(columns={"year_month": f"{disease_name}_count"})
    )

    return person_month_count

# Process each disease to calculate counts of unique phecodes per month
phecode_counts_list = []
for disease, df in disease_dataframes_filtered.items():
    disease_counts = count_unique_phecodes_per_month(df, disease)
    phecode_counts_list.append(disease_counts)

# Combine all the disease-specific counts into a single dataframe
from functools import reduce

combined_phecode_counts = reduce(
    lambda left, right: pd.merge(left, right, on="person_id", how="outer"),
    phecode_counts_list,
)
# Step 4: Fill NaN values with 0
combined_phecode_counts.fillna(0, inplace=True)
# combined_phecode_counts

In [ ]:
person_id_with_alzheimer = combined_phecode_counts[combined_phecode_counts['alzheimer_count'] > 0]['person_id'].unique()
len(person_id_with_alzheimer)

In [ ]:
person_id_with_alzheimer_pseudomonas = combined_phecode_counts[(combined_phecode_counts['alzheimer_count'] > 0) & (combined_phecode_counts['pseudomonas_count'] > 0)]['person_id'].unique()
len(person_id_with_alzheimer_pseudomonas)

In [ ]:
person_id_with_alzheimer_retrovirus = combined_phecode_counts[(combined_phecode_counts['alzheimer_count'] > 0) & (combined_phecode_counts['retrovirus_count'] > 0)]['person_id'].unique()
len(person_id_with_alzheimer_retrovirus)

In [ ]:
# Step 1: Add a column for total infection count
# Specify infection-related diseases
infection_columns = ["gram_negative_count", "staphylococcus_count", "pseudomonas_count", 
                     "streptococcus_count", "ecoli_count", "gram_positive_count"]

# Calculate the total infections for each person
combined_phecode_counts["total_infections"] = combined_phecode_counts[infection_columns].sum(axis=1)

# Step 2: Add a column to classify individuals by infection level
def classify_infection_level(row):
    if row["total_infections"] == 0:
        return "No infection"
    elif row["total_infections"] == 1:
        return "1 infection"
    elif row["total_infections"] == 2:
        return "2 infections"
    else:
        return "3+ infections"

combined_phecode_counts["infection_level"] = combined_phecode_counts.apply(classify_infection_level, axis=1)

# Step 3: Filter for individuals with Alzheimer's Disease
ad_with_infections = combined_phecode_counts[combined_phecode_counts["alzheimer_count"] > 0]

# Step 4: Count individuals by infection level
infection_summary = ad_with_infections["infection_level"].value_counts().reset_index()
infection_summary.columns = ["Infection Level", "Count"]

# # Step 5: Display the results
# print(infection_summary)


In [ ]:
# Step 1: Add a column for total infection count
infection_columns = [
    "gram_negative_count", "staphylococcus_count", "pseudomonas_count",
    "streptococcus_count", "ecoli_count", "gram_positive_count"
]

# Calculate the total infections for each person
combined_phecode_counts["total_infections"] = combined_phecode_counts[infection_columns].sum(axis=1)

# Correct the apply call to use the total_infections column directly
def classify_infection_level(total_infections):
    if total_infections == 0:
        return "No infection"
    elif total_infections == 1:
        return "1 infection"
    else:
        return "2+ infections"

# Apply the classification function to the total_infections column
combined_phecode_counts["infection_level"] = combined_phecode_counts["total_infections"].apply(classify_infection_level)

# Step 3: Filter for individuals with Alzheimer's Disease
ad_with_infections = combined_phecode_counts[combined_phecode_counts["alzheimer_count"] > 0]

# Step 4: Count individuals by infection level
infection_summary = ad_with_infections["infection_level"].value_counts()

# Step 5: Create a pie chart
plt.figure(figsize=(8, 8))

# Define colors for each group
colors = {
    "No infection":"#ff9999",  # Red
    "1 infection": "#66b3ff" , # Green
    "2+ infections": "#99ff99" # Blue
}

# Plot the pie chart
wedges, texts, autotexts = plt.pie(
    infection_summary,
    labels=infection_summary.index,
    autopct='%1.1f%%',
    colors=[colors[group] for group in infection_summary.index if group in colors],
    startangle=90
)
plt.title(f'Distribution of infection in {di.lower()} patients with CDR age more than 60', fontsize=14)


# Add a legend with the number of unique patients for each level
legend_labels = [
    f"{level}, n={infection_summary[level]}"
    for level in ["No infection", "1 infection", "2+ infections",]
]
plt.legend(wedges, legend_labels, loc='upper right', bbox_to_anchor=(1, 0, 0.5, 1),frameon=False, fontsize=12)
plt.tight_layout()
# Save the pie chart
output_path = f'./download/PIE_infection_levels_{di}.pdf'
plt.savefig(output_path, dpi=300,bbox_inches='tight')
plt.show()

# print(f'Pie chart saved at: {output_path}')
print(f'The total amount of patients are : {len(final_combined_df)}')
total_ad_patients = final_combined_df[final_combined_df["alzheimer"] == 1]["person_id"].nunique()
print(f"Total Alzheimer's patients in final_combined_df: {total_ad_patients}")
# Filter Alzheimer's patients
ad_combined_counts = combined_phecode_counts[combined_phecode_counts["person_id"].isin(
    final_combined_df[final_combined_df["alzheimer"] == 1]["person_id"]
)]

# Count patients with "No infection"
no_infection_count = ad_combined_counts[ad_combined_counts["total_infections"] == 0]["person_id"].nunique()
print(f"Alzheimer's patients with 'No infection': {no_infection_count}")



# Recalculate total infections
combined_phecode_counts["total_infections"] = combined_phecode_counts[infection_columns].sum(axis=1)

# Classify infection levels
combined_phecode_counts["infection_level"] = combined_phecode_counts["total_infections"].apply(classify_infection_level)
import matplotlib.pyplot as plt

# Step 1: Calculate the total non-Alzheimer patients
total_patients = len(final_combined_df)
total_ad_patients = final_combined_df[final_combined_df["alzheimer"] == 1]["person_id"].nunique()
total_no_ad_patients = total_patients - total_ad_patients

print(f"Total patients: {total_patients}")
print(f"Total Alzheimer's patients: {total_ad_patients}")
print(f"Total non-Alzheimer's patients: {total_no_ad_patients}")

# Step 2: Identify patients with no AD and no infections
bacteria_columns = ["gram_negative", "staphylococcus", "pseudomonas", "streptococcus", "ecoli", "gram_positive", "alzheimer"]
no_ad_no_infections_ids = final_combined_df[final_combined_df[bacteria_columns].sum(axis=1) == 0]["person_id"]

print(f"Patients with no AD and no infections: {len(no_ad_no_infections_ids)}")

# Step 3: Filter combined_phecode_counts for non-Alzheimer individuals
no_ad_ids = final_combined_df[final_combined_df["alzheimer"] == 0]["person_id"]
no_alzheimer_counts = combined_phecode_counts[combined_phecode_counts["person_id"].isin(no_ad_ids)].copy()

# Step 4: Calculate total infections for these individuals
infection_columns = ["gram_negative_count", "staphylococcus_count", "pseudomonas_count",
                     "streptococcus_count", "ecoli_count", "gram_positive_count"]

# Fill missing values in infection columns with 0
no_alzheimer_counts[infection_columns] = no_alzheimer_counts[infection_columns].fillna(0)
no_alzheimer_counts["total_infections"] = no_alzheimer_counts[infection_columns].sum(axis=1)

# Step 5: Classify infection levels
def classify_infection_level(total_infections):
    if total_infections == 0:
        return "No infection"
    elif total_infections == 1:
        return "1 infection"
    else:
        return "2+ infections"

no_alzheimer_counts["infection_level"] = no_alzheimer_counts["total_infections"].apply(classify_infection_level)

# Step 6: Count individuals by infection level
infection_summary_no_ad = no_alzheimer_counts["infection_level"].value_counts()

# Step 7: Adjust "No infection" count to include individuals with no AD or bacterial infections
infection_summary_no_ad["No infection"] += len(no_ad_no_infections_ids)

# Step 8: Create a pie chart
plt.figure(figsize=(8, 8))

# Define colors for each group
colors = {
    "No infection": "#66b3ff",  
    "1 infection": "#ff9999",  
    "2+ infections": "#99ff99" 
}

# Plot the pie chart
wedges, texts, autotexts = plt.pie(
    infection_summary_no_ad,
    labels=infection_summary_no_ad.index,
    autopct='%1.1f%%',
    colors=[colors[group] for group in infection_summary_no_ad.index if group in colors],
    startangle=90
)

plt.title(f'Distribution of infection in non-{di.lower()} patients with CDR age more than 60', fontsize=14)
# Add a legend with the number of unique patients for each level
legend_labels = [
    f"{level}, n={infection_summary[level]}"
    for level in ["No infection", "1 infection", "2+ infections"]
]
plt.legend(wedges, legend_labels, loc='upper right', bbox_to_anchor=(1, 0, 0.5, 1),frameon=False, fontsize=12)
plt.tight_layout()
# Save the pie chart
output_path = './download/PIE_infection_levels_no_alzheimer.pdf'
plt.savefig(output_path, dpi=300,bbox_inches='tight')
plt.show()

print(f"Pie chart saved at: {output_path}")


In [ ]:
def calculate_age_at_diagnosis(data, disease_name, person_info):
    """
    Calculate the age at disease diagnosis for each person, ensuring no duplicate diagnoses in the same month.
    """
    # Ensure `condition_start_datetime` and `date_of_birth` are in datetime format
    data['condition_start_datetime'] = pd.to_datetime(data['condition_start_datetime'], errors='coerce')
    person_info['date_of_birth'] = pd.to_datetime(person_info['date_of_birth'], errors='coerce')

    # Drop rows where `condition_start_datetime` could not be parsed
    data = data.dropna(subset=['condition_start_datetime'])

    # Convert both to timezone-naive
    data['condition_start_datetime'] = data['condition_start_datetime'].dt.tz_localize(None)
    person_info['date_of_birth'] = person_info['date_of_birth'].dt.tz_localize(None)

    # Extract the year and month of diagnosis
    data['diagnosis_year_month'] = data['condition_start_datetime'].dt.to_period('M')

    # Drop duplicate diagnoses for the same person and disease in the same month
    data = data.drop_duplicates(subset=['person_id', 'diagnosis_year_month'])

    # Merge with `person_info` to get date_of_birth and age
    merged_data = data.merge(person_info, on='person_id', how='left')

    # Calculate the age at diagnosis
    merged_data['age_at_diagnosis'] = (
        (merged_data['condition_start_datetime'] - merged_data['date_of_birth']).dt.days / 365.25
    )

    # Add disease name for clarity
    merged_data['disease_name'] = disease_name

    # Select relevant columns
    age_data = merged_data[['person_id', 'age_at_diagnosis', 'disease_name', 'age', 'diagnosis_year_month','date_of_birth']]

    return age_data



# Assume `person_info` contains `person_id`, `date_of_birth`, and `age`
person_info = final_combined_df[['person_id', 'date_of_birth', 'age']].copy()

# Calculate age at diagnosis for each disease and combine results
all_diagnoses = []
for disease, df in disease_dataframes.items():
    age_data = calculate_age_at_diagnosis(df, disease, person_info)
    all_diagnoses.append(age_data)

# Combine all diseases into a single DataFrame
combined_age_data = pd.concat(all_diagnoses, ignore_index=True)

# Remove rows with NaN values in the 'age' or 'age_at_diagnosis' columns
combined_age_data_cleaned = combined_age_data.dropna(subset=['age', 'age_at_diagnosis'])
# combined_age_data_cleaned

# Filter out rows with retrovirus-related infections or disease we want to remove from the counting.

In [ ]:
# Filter out rows with retrovirus-related infections
disease_keywords = ['retrovirus']  # Define retrovirus-related keywords
disease_pattern = '|'.join(disease_keywords)  # Create a regex pattern for filtering

# Remove rows where 'disease_name' contains retrovirus-related terms
combined_age_data_filtered = combined_age_data[
    ~combined_age_data['disease_name'].str.contains(disease_pattern, case=False, na=False)
]
# combined_age_data_filtered

## remove those that are not in the list of age we picked before :

In [ ]:
# Remove rows where 'person_id' is not in 'person_id_list'
combined_age_data_filtered = combined_age_data_filtered[
    combined_age_data_filtered['person_id'].isin(person_id_list)
]

# Verify the updated dataset
print(f"Total rows after filtering: {len(combined_age_data_filtered)}")
print(f"Unique person_id count: {combined_age_data_filtered['person_id'].nunique()}")


In [ ]:
# Prepare Kaplan-Meier plot and survival data
group_survival_data = {}
group_labels = {
    "0": "0 Infections",
    "1": "1 Infection",
    "2": "2 Infections",
    "3+": "3+ Infections"
}
# for group in ['0', '1', '2', '3+']:
#     group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
#     group_survival_data[group] = group_data['first_ad_age']
#     kmf.fit(
#         durations=group_data['first_ad_age'], 
#         event_observed=[1] * len(group_data),  # Assume all events observed
#         label=f"{group_labels[group]}, n={len(group_data)}"
#     )
#     kmf.plot_survival_function(color=colors[group], ci_show=False)

# Log-Rank Test
log_rank_results = []
groups = ['0', '1', '2', '3+']
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        if group_1 in group_survival_data and group_2 in group_survival_data:
            result = logrank_test(
                group_survival_data[group_1],
                group_survival_data[group_2]
            )
            log_rank_results.append((group_1, group_2, result.p_value))

log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)


In [ ]:
# Step 1: Count total occurrences for each infection
infection_counts = combined_age_data_filtered['disease_name'].value_counts()

# Step 2: Filter individuals with and without Alzheimer's diagnosis
# Identify individuals with Alzheimer's
alzheimers_patients = combined_age_data_filtered.loc[
    combined_age_data_filtered['disease_name'].str.contains('alzheimer', case=False), 'person_id'
].unique()

# Split data into with and without Alzheimer's
with_alzheimers = combined_age_data_filtered[combined_age_data_filtered['person_id'].isin(alzheimers_patients)]
without_alzheimers = combined_age_data_filtered[~combined_age_data_filtered['person_id'].isin(alzheimers_patients)]

# Step 3: Count infections for each person and classify by infection count
def classify_infections(df):
    infection_counts_per_person = df.groupby('person_id')['disease_name'].nunique()
    one_infection = infection_counts_per_person[infection_counts_per_person == 1].count()
    two_or_more_infections = infection_counts_per_person[infection_counts_per_person >= 2].count()
    return one_infection, two_or_more_infections

# Classify infections for individuals with Alzheimer's
with_alz_one_infection, with_alz_two_or_more = classify_infections(with_alzheimers)

# Classify infections for individuals without Alzheimer's
without_alz_one_infection, without_alz_two_or_more = classify_infections(without_alzheimers)

# Results
print("Total infections:")
print(infection_counts)

print("\nWith Alzheimer's:")
print(f"  Individuals with one infection: {with_alz_one_infection}")
print(f"  Individuals with two or more infections: {with_alz_two_or_more}")

print("\nWithout Alzheimer's:")
print(f"  Individuals with one infection: {without_alz_one_infection}")
print(f"  Individuals with two or more infections: {without_alz_two_or_more}")
# Step 1: Count total occurrences for each infection
infection_counts = combined_age_data_filtered['disease_name'].value_counts()

# Step 2: Filter individuals with and without Alzheimer's diagnosis
# Identify individuals with Alzheimer's
alzheimers_patients = combined_age_data_filtered.loc[
    combined_age_data_filtered['disease_name'].str.contains('alzheimer', case=False), 'person_id'
].unique()

# Split data into with and without Alzheimer's
with_alzheimers = combined_age_data_filtered[combined_age_data_filtered['person_id'].isin(alzheimers_patients)]
without_alzheimers = combined_age_data_filtered[~combined_age_data_filtered['person_id'].isin(alzheimers_patients)]

# Step 3: Count infections for each person and classify by infection count
def classify_infections(df):
    infection_counts_per_person = df.groupby('person_id')['disease_name'].nunique()
    one_infection = infection_counts_per_person[infection_counts_per_person == 1].count()
    two_or_more_infections = infection_counts_per_person[infection_counts_per_person >= 2].count()
    return one_infection, two_or_more_infections

# Classify infections for individuals with Alzheimer's
with_alz_one_infection, with_alz_two_or_more = classify_infections(with_alzheimers)

# Classify infections for individuals without Alzheimer's
without_alz_one_infection, without_alz_two_or_more = classify_infections(without_alzheimers)

# Results
print("Total infections:")
print(infection_counts)

print("\nWith Alzheimer's:")
print(f"  Individuals with one infection: {with_alz_one_infection}")
print(f"  Individuals with two or more infections: {with_alz_two_or_more}")

print("\nWithout Alzheimer's:")
print(f"  Individuals with one infection: {without_alz_one_infection}")
print(f"  Individuals with two or more infections: {without_alz_two_or_more}")


In [ ]:
from collections import defaultdict
# Step 4: Classify individuals by disease for 1 infection or 2+ infections
def classify_infections_by_disease(df):
    results = defaultdict(lambda: {'one_infection': 0, 'two_or_more': 0})
    grouped = df.groupby(['person_id', 'disease_name'])['disease_name'].count()

    for person_disease, count in grouped.items():  # Updated from iteritems() to items()
        _, disease = person_disease
        if count == 1:
            results[disease]['one_infection'] += 1
        else:
            results[disease]['two_or_more'] += 1
    return results

# With Alzheimer's: Classify individuals by disease
with_alz_results = classify_infections_by_disease(with_alzheimers)

# Without Alzheimer's: Classify individuals by disease
without_alz_results = classify_infections_by_disease(without_alzheimers)

# Convert the results to a more readable format
def format_results(results):
    formatted = []
    for disease, counts in results.items():
        formatted.append({
            'Disease': disease,
            'One Infection': counts['one_infection'],
            'Two or More Infections': counts['two_or_more'],
        })
    return pd.DataFrame(formatted)

with_alz_df = format_results(with_alz_results)
without_alz_df = format_results(without_alz_results)

# Display the results
print("With Alzheimer's:")
display(with_alz_df)

print("\nWithout Alzheimer's:")
display(without_alz_df)


In [ ]:
# Merge the infection levels into the dataset
infection_levels = combined_phecode_counts[["person_id", "infection_level"]]
ad_first_diagnosis = ad_first_diagnosis.merge(infection_levels, on="person_id", how="left")

# Categorize infection groups for consistency
ad_first_diagnosis["infection_group"] = ad_first_diagnosis["infection_level"]
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import matplotlib.pyplot as plt
import pandas as pd

# Step 1: Count bacterial infections for each person
bacteria_list = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']
bacteria_counts = (
    combined_age_data_cleaned[
        combined_age_data_cleaned['disease_name'].isin(bacteria_list)
    ]
    .groupby('person_id')['disease_name']
    .nunique()
    .reset_index()
    .rename(columns={'disease_name': 'bacteria_infections'})
)

# Step 2: Identify the first AD diagnosis for each person
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_first_diagnosis = (
    ad_data.groupby('person_id')['age_at_diagnosis']
    .min()
    .reset_index()
    .rename(columns={'age_at_diagnosis': 'first_ad_age'})
)

# Step 3: Merge bacterial infection counts with first AD diagnosis age
ad_first_diagnosis = ad_first_diagnosis.merge(bacteria_counts, on='person_id', how='left').fillna(0)

# Step 4: Filter by minimum age at first AD diagnosis
min_first_ad_age = 0
ad_first_diagnosis = ad_first_diagnosis[ad_first_diagnosis['first_ad_age'] > min_first_ad_age]

# Step 5: Categorize into infection groups
ad_first_diagnosis['infection_group'] = ad_first_diagnosis['bacteria_infections'].apply(
    lambda x: '0' if x == 0 else ('1' if x == 1 else ('2' if x == 2 else "3+"))
)

# Step 6: Perform Kaplan-Meier Analysis
kmf = KaplanMeierFitter()
plt.figure(figsize=(12, 8))
colors = {'0': 'blue', '1': 'purple', '2': 'green', '3+': 'red'}

# Add general dementia line
kmf.fit(
    durations=ad_first_diagnosis['first_ad_age'],
    event_observed=[1] * len(ad_first_diagnosis),
    label="Dementia (All Patients)"
)
kmf.plot_survival_function(color='black', linestyle='--', linewidth=2, ci_show=False)

# Prepare Kaplan-Meier plot and survival data
group_survival_data = {}
group_labels = {
    "0": "0 Infections",
    "1": "1 Infection",
    "2": "2 Infections",
    "3+": "3+ Infections"
}
for group in ['0', '1', '2', '3+']:
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
    group_survival_data[group] = group_data['first_ad_age']
    kmf.fit(
        durations=group_data['first_ad_age'], 
        event_observed=[1] * len(group_data),  # Assume all events observed
        label=f"{group_labels[group]}, n={len(group_data)}"
    )
    kmf.plot_survival_function(color=colors[group], ci_show=False)

# Log-Rank Test
log_rank_results = []
groups = ['0', '1', '2', '3+']
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        if group_1 in group_survival_data and group_2 in group_survival_data:
            result = logrank_test(
                group_survival_data[group_1],
                group_survival_data[group_2]
            )
            log_rank_results.append((group_1, group_2, result.p_value))

log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)

# Enhance the plot
plt.title('Kaplan-Meier Plot for First AD Diagnosis Age by Infection Group', fontsize=16)
plt.xlabel('Age at First AD Diagnosis', fontsize=14)
plt.ylabel('Survival Probability', fontsize=14)
plt.legend(title='Infection Group', fontsize=12, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Save the plot
output_path = "./download/first_ad_diagnosis_by_infection_group_with_logrank.pdf"
plt.savefig(output_path, bbox_inches="tight", dpi=300)
plt.show()

print("Kaplan-Meier plot saved at:", output_path)

# Display log-rank test results
print("\nLog-Rank Test Results:")
print(log_rank_comparisons)


In [ ]:
min_first_ad_age = 0
# Step 1: Count total bacterial infections for each person
bacteria_list = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']
bacteria_counts = (
    combined_age_data_cleaned[
        combined_age_data_cleaned['disease_name'].isin(bacteria_list)
    ]
    .groupby('person_id')['disease_name']
    .count()  # Count total infections (including duplicates)
    .reset_index()
    .rename(columns={'disease_name': 'bacteria_infections'})
)

# Step 2: Identify the first AD diagnosis for each person
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_first_diagnosis = (
    ad_data.groupby('person_id')['age_at_diagnosis']
    .min()
    .reset_index()
    .rename(columns={'age_at_diagnosis': 'first_ad_age'})
)

# Step 3: Merge bacterial infection counts with first AD diagnosis age
ad_first_diagnosis = ad_first_diagnosis.merge(bacteria_counts, on='person_id', how='left').fillna(0)

# Step 4: Filter by minimum age at first AD diagnosis

ad_first_diagnosis = ad_first_diagnosis[ad_first_diagnosis['first_ad_age'] > min_first_ad_age]

# Step 5: Categorize into infection groups
ad_first_diagnosis['infection_group'] = ad_first_diagnosis['bacteria_infections'].apply(
    lambda x: '0' if x == 0 else ('1' if x == 1 else ('2' if x == 2 else "3+"))
)

# Step 6: Perform Kaplan-Meier Analysis
kmf = KaplanMeierFitter()
plt.figure(figsize=(12, 8))
colors = {'0': 'blue', '1': 'purple', '2': 'green', '3+': 'red'}
# Add general dementia line
kmf.fit(
    durations=ad_first_diagnosis['first_ad_age'],
    event_observed=[1] * len(ad_first_diagnosis),
    label=f"{di}"
)
kmf.plot_survival_function(color='black', linestyle='--', linewidth=2,ci_show=False)
# Log-Rank Test
log_rank_results = []
groups = ['0', '1', '2', '3+']
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        if group_1 in group_survival_data and group_2 in group_survival_data:
            result = logrank_test(
                group_survival_data[group_1],
                group_survival_data[group_2]
            )
            log_rank_results.append((group_1, group_2, result.p_value))

log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)

# Create custom legend labels with p-values
group_labels = {}
for group in groups:
    if group == '0':
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])})"
    else:
        p_val = min([res[2] for res in log_rank_results if res[0] == '0' and res[1] == group])
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])}, p={p_val:.4e})"

# Prepare Kaplan-Meier plot and survival data
group_survival_data = {}
for group in ['0', '1', '2', '3+']:
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
    group_survival_data[group] = group_data['first_ad_age']
    kmf.fit(
        durations=group_data['first_ad_age'], 
        event_observed=[1] * len(group_data),  # Assume all events observed
        label=group_labels[group]
    )
    kmf.plot_survival_function(color=colors[group], ci_show=False)

# Enhance the plot
plt.title(f'Kaplan-Meier plot for first {di} diagnosis age by infection group', fontsize=16)
plt.xlabel(f'Age at first {di} diagnosis', fontsize=14)
plt.ylabel('Survival probability', fontsize=14)
plt.legend(title='Infection group', fontsize=12, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Show the plot
plt.tight_layout()
reversed_survival_plot_path = f"./download/first_ad_diagnosis_by_infection_group_with_logrank_{di}.pdf"
plt.savefig(reversed_survival_plot_path, bbox_inches='tight')
plt.show()

# Group data by 'infection_group'
summary_stats = (
    ad_first_diagnosis.groupby('infection_group')['first_ad_age']
    .agg(
        Mean='mean',
        Median='median',
        Min='min',
        Count='count'
    )
    .reset_index()
    .rename(columns={'infection_group': 'Infection Group'})
)

# Display the summary statistics table
print(summary_stats)

# Save the table to a CSV file for further use if needed
summary_stats_path = f"./download/summary_statistics_first_dementia_by_infection_group_{di}.csv"
summary_stats.to_csv(summary_stats_path, index=False)
print(f"Summary statistics saved to: {summary_stats_path}")
# summary_stats

In [ ]:
# Define a custom aggregation function to compute additional statistics
def extended_stats(group):
    return pd.Series({
        'Mean': group.mean(),
        'Median': group.median(),
        'Min': group.min(),
        'Max': group.max(),
        'Std Dev': group.std(),
        'Variance': group.var(),
        '25th Percentile': group.quantile(0.25),
        '75th Percentile': group.quantile(0.75),
        'IQR': group.quantile(0.75) - group.quantile(0.25),
        'Skewness': skew(group),
        'Kurtosis': kurtosis(group),
        'Mode': group.mode().iloc[0] if not group.mode().empty else None,
        'Count': group.count()
    })

# Apply the function to each infection group
detailed_stats = (
    ad_first_diagnosis.groupby('infection_group')['first_ad_age']
    .apply(extended_stats)
    .reset_index()
)

# Display the detailed statistics table
print(detailed_stats)

# Save the detailed statistics to a CSV file
detailed_stats_path = f"./download/detailed_statistics_first_dementia_by_infection_group_{di}.csv"
detailed_stats.to_csv(detailed_stats_path, index=False)
print(f"Detailed statistics saved to: {detailed_stats_path}")


# Create the box plot
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=ad_first_diagnosis,
    x='infection_group',
    y='first_ad_age',
    palette='coolwarm',
    order=['0', '1', '2', '3+']  # Ensure consistent order
)

# Enhance the aesthetics
plt.title("Dementia Onset Age by Infection Group", fontsize=16)
plt.xlabel("Infection Group", fontsize=14)
plt.ylabel("Age at First Dementia Diagnosis", fontsize=14)

# Add annotations for descriptive statistics
for i, group in enumerate(['0', '1', '2', '3+']):
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]['first_ad_age']
    mean_age = group_data.mean()
    median_age = group_data.median()
#     plt.text(i, mean_age, f"Mean: {mean_age:.1f}", color="black", ha="center", va="top", fontsize=10)
#     plt.text(i, median_age, f"Median: {median_age:.1f}", color="blue", ha="center", va="bottom", fontsize=10)

# Customize gridlines and spines
plt.grid(axis='y', linestyle='--', alpha=0.7)
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Adjust tick parameters
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Save the plot
output_path = "./download/dementia_onset_age_by_infection_group_boxplot.pdf"
plt.tight_layout()
plt.savefig(output_path, bbox_inches='tight')
plt.show()

print(f"Box plot saved to: {output_path}")


## (a) One-Way ANOVA (for more than two groups): Checks for significant differences in mean dementia onset age among all infection groups.

In [ ]:
# from scipy.stats import f_oneway
# from scipy.stats import skew, kurtosis
# from lifelines import KaplanMeierFitter
# import matplotlib.lines as mlines
# from statsmodels.stats.multitest import multipletests

# Group ages by infection groups
group_ages = [ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]['first_ad_age']
              for group in ['0', '1', '2', '3+']]

# Perform ANOVA
anova_result = f_oneway(*group_ages)

print(f"ANOVA Results: F-statistic = {anova_result.statistic:.4f}, P-value = {anova_result.pvalue:.4e}")

if anova_result.pvalue < 0.05:
    print("Significant difference in dementia onset age between infection groups.")
else:
    print("No significant difference in dementia onset age between infection groups.")


In [ ]:
# from statsmodels.stats.multicomp import pairwise_tukeyhsd

# # Perform Tukey HSD test
# tukey_result = pairwise_tukeyhsd(
#     ad_first_diagnosis['first_ad_age'], ad_first_diagnosis['infection_group']
# )

# print(tukey_result)

# # Plot Tukey results
# tukey_result.plot_simultaneous()
# plt.title("Tukey HSD: Dementia Onset Age by Infection Group", fontsize=16)
# plt.show()


In [ ]:
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# from scipy.stats import f_oneway
# from scipy.stats import skew, kurtosis
# from lifelines import KaplanMeierFitter
# import matplotlib.lines as mlines
# from statsmodels.stats.multitest import multipletests
# Perform Tukey HSD test
tukey_result = pairwise_tukeyhsd(
    ad_first_diagnosis['first_ad_age'], ad_first_diagnosis['infection_group']
)

print(tukey_result)

# Plot Tukey results
fig, ax = plt.subplots(figsize=(10, 6))
tukey_result.plot_simultaneous(ax=ax, figsize=(10, 6))

# Enhance plot aesthetics
ax.set_title(f"Tukey HSD: {di} onset age by infection group", fontsize=16)
ax.set_xlabel(f"Mean difference onset age of {di} (95% CI)", fontsize=14)
ax.set_ylabel("Infection group comparison", fontsize=14)
ax.grid(False)  # Remove grid

# Customize spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Adjust ticks
ax.tick_params(axis='both', which='major', labelsize=12)

# Save the plot
output_path = f"./download/Tukey_pairwise_comparison_{di}.pdf"
plt.tight_layout()
plt.savefig(output_path, bbox_inches='tight')
plt.show()

# print(f"Tukey HSD plot saved to: {output_path}")
# detailed_stats_path = f"./download/Tukey HSD analysis for the mean {di}.tsv"
# tukey_result.to_csv(detailed_stats_path, index=False)

## If i want to test the median age : I need a different approach because Tukey HSD does not work with medians. Medians are often used when:
IF :
The data is not normally distributed.
There are outliers that heavily skew the mean.

In [ ]:
# from scipy.stats import mannwhitneyu
# import itertools
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# from scipy.stats import f_oneway
# from scipy.stats import skew, kurtosis
# from lifelines import KaplanMeierFitter
# import matplotlib.lines as mlines
# from statsmodels.stats.multitest import multipletests
# Get unique groups
groups = ad_first_diagnosis['infection_group'].unique()

# Perform pairwise Wilcoxon (Mann-Whitney U) tests
results = []
for group1, group2 in itertools.combinations(groups, 2):
    group1_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group1]['first_ad_age']
    group2_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group2]['first_ad_age']
    stat, p_value = mannwhitneyu(group1_data, group2_data, alternative='two-sided')
    results.append((group1, group2, stat, p_value))

# Display results
results_df = pd.DataFrame(results, columns=['Group 1', 'Group 2', 'Statistic', 'P-Value'])
print(results_df)
# Save the detailed statistics to a CSV file
detailed_stats_path = f"./download/pairwise Wilcoxon (Mann-Whitney U) analysis for the median_{di}.tsv"
results_df.to_csv(detailed_stats_path, index=False)

In [ ]:
# import statsmodels.formula.api as smf
# from scipy.stats import mannwhitneyu
# import itertools
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# from scipy.stats import f_oneway
# from scipy.stats import skew, kurtosis
# from lifelines import KaplanMeierFitter
# import matplotlib.lines as mlines
# from statsmodels.stats.multitest import multipletests

# Perform quantile regression for the median
model = smf.quantreg('first_ad_age ~ C(infection_group)', ad_first_diagnosis).fit(q=0.5)
print(model.summary())


In [ ]:
total_patients = len(final_combined_df)
total_ad_patients = final_combined_df[final_combined_df["alzheimer"] == 1]["person_id"].nunique()
total_no_ad_patients = total_patients - total_ad_patients
total_no_ad_patients

In [ ]:
bacterial_data_age = 0

In [ ]:
# from collections import defaultdict
# import statsmodels.formula.api as smf
# from scipy.stats import mannwhitneyu
# import itertools
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# from scipy.stats import f_oneway
# from scipy.stats import skew, kurtosis
# from lifelines import KaplanMeierFitter
# import matplotlib.lines as mlines
# from statsmodels.stats.multitest import multipletests


# Step 1: Define bacterial infections
bacterial_infections = ['staphylococcus', 'ecoli', 'pseudomonas', 
                        'gram_negative', 'gram_positive', 'streptococcus']

# Step 2: Filter bacterial data for individuals older than 65
bacterial_data = combined_age_data_filtered[
    (combined_age_data_filtered['disease_name'].str.contains('|'.join(bacterial_infections), case=False, na=False)) &
    (combined_age_data_filtered['age'] > bacterial_data_age)  # Age filter
]

# Step 3: Identify individuals with Alzheimer's
alzheimers_patients = combined_age_data_filtered.loc[
    (combined_age_data_filtered['disease_name'].str.contains('alzheimer', case=False, na=False)) &
    (combined_age_data_filtered['age'] > bacterial_data_age),  # Age filter for Alzheimer's patients
    'person_id'
].unique()
# Step 4: Split bacterial data into with and without Alzheimer's
with_alzheimers = bacterial_data[bacterial_data['person_id'].isin(alzheimers_patients)]
without_alzheimers = bacterial_data[~bacterial_data['person_id'].isin(alzheimers_patients)]

# Step 5: Identify individuals with Alzheimer's but no bacterial infections
alz_with_no_infections = set(alzheimers_patients) - set(with_alzheimers['person_id'])

# Step 6: Count infections and classify
def classify_infections_v2(df, total_patients):
    infection_counts = df.groupby(['person_id', 'disease_name']).size()
    grouped_counts = infection_counts.groupby(level=0).sum()  # Total infections per person
    multiple_same_disease = infection_counts[infection_counts > 1].groupby(level=0).count()  # Patients with >1 infection of same disease

    one_infection = grouped_counts[grouped_counts == 1].index
    two_or_more_infections = set(grouped_counts[grouped_counts > 1].index) | set(multiple_same_disease.index)

    zero_infections = set(total_patients) - set(df['person_id'])

    return len(zero_infections), len(one_infection), len(two_or_more_infections)

# Total unique Alzheimer’s patients
total_alz_patients = set(alzheimers_patients)

# Classify infections for individuals with Alzheimer's
alz_zero, alz_one, alz_two_or_more = classify_infections_v2(with_alzheimers, total_alz_patients)

# Total unique patients without Alzheimer's
total_non_alz_patients = set(combined_age_data_filtered['person_id']) - total_alz_patients

# Classify infections for individuals without Alzheimer's
non_alz_zero, non_alz_one, non_alz_two_or_more = classify_infections_v2(without_alzheimers, total_non_alz_patients)

# Step 7: Results
print("Bacterial Infections Only:")
print("\nWith Alzheimer's:")
print(f"  Individuals with 0 infections: {alz_zero}")
print(f"  Individuals with 1 infection: {alz_one}")
print(f"  Individuals with 2+ infections: {alz_two_or_more}")

print("\nWithout Alzheimer's:")
print(f"  Individuals with 0 infections: {non_alz_zero}")
print(f"  Individuals with 1 infection: {non_alz_one}")
print(f"  Individuals with 2+ infections: {non_alz_two_or_more}")

# Step 8: Detailed disease classification
def classify_by_disease(df):
    results = defaultdict(lambda: {'one_infection': 0, 'two_or_more': 0})
    infection_counts = df.groupby(['person_id', 'disease_name']).size()

    for (person_id, disease), count in infection_counts.items():
        if count == 1:
            results[disease]['one_infection'] += 1
        else:
            results[disease]['two_or_more'] += 1
    return results

# With Alzheimer's: Classify individuals by disease
with_alz_disease_results = classify_by_disease(with_alzheimers)

# Without Alzheimer's: Classify individuals by disease
without_alz_disease_results = classify_by_disease(without_alzheimers)

# Convert the results to DataFrame for readability
def format_disease_results(results):
    formatted = []
    for disease, counts in results.items():
        formatted.append({
            'Disease': disease,
            'One Infection': counts['one_infection'],
            'Two or More Infections': counts['two_or_more'],
        })
    return pd.DataFrame(formatted)

with_alz_disease_df = format_disease_results(with_alz_disease_results)
without_alz_disease_df = format_disease_results(without_alz_disease_results)

# Display results
print("\nWith Alzheimer's - Infections by Disease:")
display(with_alz_disease_df)

print("\nWithout Alzheimer's - Infections by Disease:")
display(without_alz_disease_df)


In [ ]:
# Filter Alzheimer's individuals based on person_id_list
ad_individuals = set(alzheimers_patients) & set(person_id_list)

# Classify infections into 0, 1, or 2+ for individuals with Alzheimer's
def classify_ad_infections(df, ad_list):
    ad_df = df[df['person_id'].isin(ad_list)]
    infection_counts_per_person = ad_df.groupby('person_id')['disease_name'].nunique()
    
    zero_infection = len(ad_list) - len(infection_counts_per_person)  # Individuals not in the filtered dataframe
    one_infection = infection_counts_per_person[infection_counts_per_person == 1].count()
    two_or_more_infections = infection_counts_per_person[infection_counts_per_person >= 2].count()
    
    return zero_infection, one_infection, two_or_more_infections

# Classify infections for Alzheimer's individuals in the given list
ad_zero, ad_one, ad_two_or_more = classify_ad_infections(bacterial_data, ad_individuals)

# Count the occurrences of each bacteria for Alzheimer's individuals in the list
def count_bacteria_by_person(df, ad_list, bacterial_list):
    ad_df = df[df['person_id'].isin(ad_list)]
    bacteria_counts = ad_df['disease_name'].value_counts()
    return bacteria_counts[bacteria_counts.index.str.contains('|'.join(bacterial_list), case=False)]

# Get bacterial counts for Alzheimer's individuals in the list
ad_bacteria_counts = count_bacteria_by_person(bacterial_data, ad_individuals, bacterial_infections)

# Display Results
print("Infection Categories for Individuals with Alzheimer's:")
print(f"  0 Infections: {ad_zero}")
print(f"  1 Infection: {ad_one}")
print(f"  2+ Infections: {ad_two_or_more}")

print("\nBacterial Counts for Individuals with Alzheimer's:")
print(ad_bacteria_counts)


In [ ]:
# from collections import defaultdict

# Classify infections into 0, 1, or 2+ and count bacteria occurrences
def detailed_infection_bacteria_counts(df, ad_list, bacterial_list):
    ad_df = df[df['person_id'].isin(ad_list)]
    infection_counts_per_person = ad_df.groupby('person_id')['disease_name'].nunique()
    
    # Initialize counters
    bacteria_summary = defaultdict(lambda: {'0': 0, '1': 0, '2+': 0})
    
    # Loop through individuals and classify
    for person_id in ad_list:
        if person_id not in infection_counts_per_person:
            # 0 infections
            for bacteria in bacterial_list:
                bacteria_summary[bacteria]['0'] += 1
        else:
            # Get individual's infections
            person_bacteria = ad_df[ad_df['person_id'] == person_id]['disease_name'].unique()
            infection_count = len(person_bacteria)
            
            # Classify infection count
            category = '1' if infection_count == 1 else '2+'
            for bacteria in bacterial_list:
                if bacteria in person_bacteria:
                    bacteria_summary[bacteria][category] += 1

    # Summarize counts for each bacteria
    detailed_counts = pd.DataFrame.from_dict(bacteria_summary, orient='index')
    detailed_counts.reset_index(inplace=True)
    detailed_counts.rename(columns={'index': 'Bacteria', '0': 'No Infection', '1': '1 Infection', '2+': '2+ Infections'}, inplace=True)
    
    return detailed_counts

# Get the detailed counts
detailed_bacteria_counts = detailed_infection_bacteria_counts(bacterial_data, ad_individuals, bacterial_infections)

# Display results
print("Detailed Breakdown of Infections by Bacteria for Individuals with Alzheimer's:")
print(detailed_bacteria_counts)


# Correct Pies

In [ ]:
# Step 1: Total number of patients in the dataset
total_patients = len(person_id_list)

# Step 2: Identify the list of Alzheimer’s patients
alzheimers_patients = alzheimer_data_filtered['person_id'].unique()

# Step 3: Find non-Alzheimer’s patients
non_alz_patients = set(person_id_list) - set(alzheimers_patients)
total_number_no_ad = len(non_alz_patients)

# Step 4: Filter bacterial data for non-Alzheimer’s patients older than 65

non_alz_bacterial_data = bacterial_data[
    (bacterial_data['person_id'].isin(non_alz_patients)) & 
    (bacterial_data['age'] > bacterial_data_age)
]
# Classify non-Alzheimer’s patients by infection count
def classify_non_ad_patients(non_ad_bacterial_data, non_ad_patient_list):
    infection_counts = non_ad_bacterial_data.groupby(['person_id'])['disease_name'].nunique()
    
    no_infections = len(set(non_ad_patient_list) - set(non_ad_bacterial_data['person_id']))
    one_infection = (infection_counts[infection_counts == 1].count())
    two_or_more_infections = (infection_counts[infection_counts >= 2].count())
    
    return no_infections, one_infection, two_or_more_infections

# Step 5: Calculate infection categories for non-Alzheimer’s patients
non_ad_no_infections, non_ad_one_infection, non_ad_two_or_more = classify_non_ad_patients(
    non_alz_bacterial_data, non_alz_patients
)

# Step 6: Combine results
alz_total = len(alzheimers_patients)
non_ad_total = total_number_no_ad

print(f"Total patients: {total_patients}")
print(f"{di.lower()} patients: {alz_total}")
print(f"Non-{di.lower()}’s patients: {non_ad_total}")
print(f"\nNon-{di.lower()}’s Infection Summary:")
print(f"  No infections: {non_ad_no_infections}")
print(f"  1 infection: {non_ad_one_infection}")
print(f"  2+ infections: {non_ad_two_or_more}")
print(f"\n  No infections with {di}: {alz_zero}")
print(f"  1 infection with {di}: {alz_one}")
print(f"  2+ infections with {di}: {alz_two_or_more}")
# Step 7: Prepare data for two pies
alz_pie_data = {
    "No infection": alz_zero,
    "1 infection": alz_one,
    "2+ infections": alz_two_or_more
}

non_ad_pie_data = {
    "No infection": non_ad_no_infections,
    "1 infection": non_ad_one_infection,
    "2+ infections": non_ad_two_or_more
}

# Step 8: Define colors
colors = {
    "No infection": "#ff9999",  # Red
    "1 infection": "#99ff99",  # Green
    "2+ infections": "#66b3ff",  # Blue
}

# Step 9: Plot pie charts
def plot_pie_chart(data, title, output_path):
    plt.figure(figsize=(8, 8))
    wedges, texts, autotexts = plt.pie(
        data.values(),
        labels=data.keys(),
        autopct='%1.1f%%',
        colors=[colors[group] for group in data.keys()],
        startangle=90,
        textprops={'fontsize': 12}
    )
    plt.title(title, fontsize=14)
    legend_labels = [f"{group}, n={count}" for group, count in data.items()]
    plt.legend(
        wedges,
        legend_labels,
        loc='upper right',
        bbox_to_anchor=(1, 0, 0.5, 1),
        frameon=False,
        fontsize=12
    )
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.show()

# Alzheimer’s pie chart
plot_pie_chart(
    alz_pie_data,
    f"Distribution of infections in {di.lower()} patients",
    f"./download/{di.lower()}_infections_pie_{bacterial_data_age}.pdf"
)

# # Non-Alzheimer’s pie chart
# plot_pie_chart(
#     non_ad_pie_data,
#     "Distribution of Infections in Non-Alzheimer’s Patients",
#     "./non_ad_infections_pie.pdf"
# )


In [ ]:
def plot_pie_chart(data, title, output_path):
    plt.figure(figsize=(8, 8))
    
    # Plot the pie chart with adjusted label distance
    wedges, texts = plt.pie(
        data.values(),
        labels=data.keys(),
        autopct=None,  # Disable default percentage placement
        colors=[colors[group] for group in data.keys()],
        startangle=90,
        textprops={'fontsize': 12},
        labeldistance=1.2  # Move labels further from the center
    )
    
    # Add percentage labels with adjusted positions
    total = sum(data.values())
    for i, (wedge, count, label) in enumerate(zip(wedges, data.values(), data.keys())):
        angle = (wedge.theta2 + wedge.theta1) / 2  # Find the center angle of the wedge
        x = np.cos(np.radians(angle))  # X-coordinate adjustment
        y = np.sin(np.radians(angle))  # Y-coordinate adjustment
        
        # Adjust the distance from the center
        if label == "2+ infections":
            x_adjusted = x * 0.8   # Move left
            y_adjusted = y * 0.9 - 0.2  # Move lower
        elif label == "1 infection":
            x_adjusted = x * 0.8 + 0.01  # Move right
            y_adjusted = y * 0.5 + 0.2  # Move higher
        else:
            x_adjusted = x * 1+0.3
            y_adjusted = y * 1+0.3
        
        # Add the percentage label
        plt.text(
            x_adjusted, y_adjusted,
            f"{(count / total) * 100:.1f}%",
            ha="center", va="center", fontsize=12, color='black'
        )
    
    # Title and legend
    plt.title(title, fontsize=14)
    legend_labels = [f"{group}, n={count}" for group, count in data.items()]
    plt.legend(
        wedges,
        legend_labels,
        loc='upper right',
        bbox_to_anchor=(1, 0, 0.5, 1),
        frameon=False,
        fontsize=12
    )
    
    # Save and show the pie chart
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.show()

# Non-Alzheimer’s pie chart
plot_pie_chart(
    non_ad_pie_data,
    f"Distribution of infections in non-{di.lower()} patients",
    f"./download/non_{di.lower()}_infections_pie_{bacterial_data_age}.pdf"
)


In [ ]:
# import lifelines
# print(lifelines.__version__)


In [ ]:
# Step 1: Count total bacterial infections for each person
bacteria_list = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']
bacteria_counts = (
    combined_age_data_cleaned[
        combined_age_data_cleaned['disease_name'].isin(bacteria_list)
    ]
    .groupby('person_id')['disease_name']
    .count()  # Count total infections (including duplicates)
    .reset_index()
    .rename(columns={'disease_name': 'bacteria_infections'})
)

# Step 2: Identify the first AD diagnosis for each person
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_first_diagnosis = (
    ad_data.groupby('person_id')['age_at_diagnosis']
    .min()
    .reset_index()
    .rename(columns={'age_at_diagnosis': 'first_ad_age'})
)

# Step 3: Merge bacterial infection counts with first AD diagnosis age
ad_first_diagnosis = ad_first_diagnosis.merge(bacteria_counts, on='person_id', how='left').fillna(0)

# Step 4: Filter by minimum age at first AD diagnosis
min_first_ad_age = 0
ad_first_diagnosis = ad_first_diagnosis[ad_first_diagnosis['first_ad_age'] > min_first_ad_age]

# Step 5: Categorize into infection groups
ad_first_diagnosis['infection_group'] = ad_first_diagnosis['bacteria_infections'].apply(
    lambda x: '0' if x == 0 else ('1' if x == 1 else ('2' if x == 2 else "3+"))
)


# Step 6: Perform Kaplan-Meier Analysis
kmf = KaplanMeierFitter()
plt.figure(figsize=(12, 8))
colors = {'0': 'blue', '1': 'purple', '2': 'green', '3+': 'red'}
# Add general dementia line
kmf.fit(
    durations=ad_first_diagnosis['first_ad_age'],
    event_observed=[1] * len(ad_first_diagnosis),
    label="Dementia (All Patients)"
)
kmf.plot_survival_function(color='black', linestyle='--', linewidth=2,ci_show=False)
# Log-Rank Test
log_rank_results = []
groups = ['0', '1', '2', '3+']
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        if group_1 in group_survival_data and group_2 in group_survival_data:
            result = logrank_test(
                group_survival_data[group_1],
                group_survival_data[group_2]
            )
            log_rank_results.append((group_1, group_2, result.p_value))

log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)

# Create custom legend labels with p-values
group_labels = {}
for group in groups:
    if group == '0':
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])})"
    else:
        p_val = min([res[2] for res in log_rank_results if res[0] == '0' and res[1] == group])
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])}, p={p_val:.6f})"

# Prepare Kaplan-Meier plot and survival data
group_survival_data = {}
for group in ['0', '1', '2', '3+']:
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
    group_survival_data[group] = group_data['first_ad_age']
    kmf.fit(
        durations=group_data['first_ad_age'], 
        event_observed=[1] * len(group_data),  # Assume all events observed
        label=group_labels[group]
    )
    kmf.plot_survival_function(color=colors[group], ci_show=False)



# Enhance the plot
plt.title(f'Kaplan-Meier Plot for First {di} Diagnosis Age by Infection Group', fontsize=16)
plt.xlabel(f'Age at First {di} Diagnosis', fontsize=14)
plt.ylabel('Survival Probability', fontsize=14)
plt.legend(title='Infection Group', fontsize=12, frameon=False)
plt.grid(False)
# Set x-axis limit to start at 40
plt.xlim(40, None)  # Start x-axis at 40, let it auto-adjust for the upper limit

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Show the plot
plt.tight_layout()
reversed_survival_plot_path = f"./download/first_ad_diagnosis_by_infection_group_with_logrank_{di}.pdf"
plt.savefig(reversed_survival_plot_path)
plt.show()


In [ ]:
log_rank_comparisons

In [ ]:
# Step 1: Count total bacterial infections for each person
bacteria_list = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']
bacteria_counts = (
    combined_age_data_cleaned[
        combined_age_data_cleaned['disease_name'].isin(bacteria_list)
    ]
    .groupby('person_id')['disease_name']
    .count()  # Count total infections (including duplicates)
    .reset_index()
    .rename(columns={'disease_name': 'bacteria_infections'})
)

# Step 2: Identify the first AD diagnosis for each person
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_first_diagnosis = (
    ad_data.groupby('person_id')['age_at_diagnosis']
    .min()
    .reset_index()
    .rename(columns={'age_at_diagnosis': 'first_ad_age'})
)

# Step 3: Merge bacterial infection counts with first AD diagnosis age
ad_first_diagnosis = ad_first_diagnosis.merge(bacteria_counts, on='person_id', how='left').fillna(0)

# Step 4: Filter by minimum age at first AD diagnosis
min_first_ad_age = 0
ad_first_diagnosis = ad_first_diagnosis[ad_first_diagnosis['first_ad_age'] > min_first_ad_age]

# Step 5: Categorize into infection groups
ad_first_diagnosis['infection_group'] = ad_first_diagnosis['bacteria_infections'].apply(
    lambda x: '0' if x == 0 else ('1' if x == 1 else ('2' if x == 2 else "3+"))
)

# Step 6: Perform Kaplan-Meier Analysis
kmf = KaplanMeierFitter()
plt.figure(figsize=(12, 8))
colors = {'0': 'blue', '1': 'purple', '2': 'green', '3+': 'red'}
# Add general dementia line
kmf.fit(
    durations=ad_first_diagnosis['first_ad_age'],
    event_observed=[1] * len(ad_first_diagnosis),
    label="Dementia (All Patients)"
)
kmf.plot_survival_function(color='black', linestyle='--', linewidth=2,ci_show=False)
# Log-Rank Test
log_rank_results = []
groups = ['0', '1', '2', '3+']
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        if group_1 in group_survival_data and group_2 in group_survival_data:
            result = logrank_test(
                group_survival_data[group_1],
                group_survival_data[group_2]
            )
            log_rank_results.append((group_1, group_2, result.p_value))

log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)
# Set x-axis limit to start at 40
plt.xlim(0, None)  # Start x-axis at 40, let it auto-adjust for the upper limit

# Create custom legend labels with p-values
group_labels = {}
for group in groups:
    if group == '0':
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])})"
    else:
        p_val = min([res[2] for res in log_rank_results if res[0] == '0' and res[1] == group])
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])}, p={p_val:.3e})"

# Prepare Kaplan-Meier plot and survival data
group_survival_data = {}
for group in ['0', '1', '2', '3+']:
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
    group_survival_data[group] = group_data['first_ad_age']
    kmf.fit(
        durations=group_data['first_ad_age'], 
        event_observed=[1] * len(group_data),  # Assume all events observed
        label=group_labels[group]
    )
    kmf.plot_survival_function(color=colors[group], ci_show=False)



# Enhance the plot
plt.title(f'Kaplan-Meier plot for first {di} diagnosis age by infection group', fontsize=16)
plt.xlabel(f'Age at first {di} diagnosis', fontsize=14)
plt.ylabel('Survival probability', fontsize=14)
plt.legend(title='Infection group', fontsize=12, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Show the plot
plt.tight_layout()
reversed_survival_plot_path = f"./download/first_ad_diagnosis_by_infection_group_with_logrank_e_{di}.pdf"
plt.savefig(reversed_survival_plot_path)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Define the list of bacterial infections
bacteria_list = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']

# Step 2: Filter patients with Alzheimer's
alzheimers_patients = combined_age_data_filtered.loc[
    combined_age_data_filtered['disease_name'].str.contains('alzheimer', case=False), 'person_id'
].unique()

# Step 3: Filter bacterial data for patients with Alzheimer's
bacterial_data_alz = combined_age_data_filtered[
    (combined_age_data_filtered['person_id'].isin(alzheimers_patients)) &
    (combined_age_data_filtered['disease_name'].isin(bacteria_list))
]

# Step 4: Count infections for each person
infection_counts_alz = bacterial_data_alz.groupby(['person_id', 'disease_name']).size().reset_index(name='infection_count')

# Step 5: Categorize individuals by the number of infections for each bacteria
infection_summary_alz = (
    bacterial_data_alz.groupby('disease_name')['person_id']
    .apply(lambda x: pd.Series({
        '0 Infections': len(set(alzheimers_patients) - set(x)),
        '1 Infection': (x.value_counts() == 1).sum(),
        '2+ Infections': (x.value_counts() > 1).sum()
    }))
    .unstack()  # Convert the Series into DataFrame columns
    .reset_index()
)

# Step 6: Rename columns for clarity
infection_summary_alz.rename(columns={'disease_name': 'Bacteria'}, inplace=True)

# Display the corrected summary
print("\nCorrected Summary of Infections by Bacteria for Patients with Alzheimer's:")
print(infection_summary_alz)



In [ ]:
# Step 2: Create the box plot
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=ad_first_diagnosis,
    x='infection_group',
    y='first_ad_age',
    palette='tab10',
    order=['0', '1', '2', '3+'],
    showfliers=False
)

# Step 3: Enhance the plot
plt.title(f'Distribution of Age at First {di} Diagnosis by Infection Group', fontsize=16)
plt.xlabel('Infection Group', fontsize=14)
plt.ylabel(f'Age at First {di} Diagnosis', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
# Step 4: Save the plot
box_plot_path = f"./download/{di}_first_diagnosis_box_plot.pdf"
plt.savefig(box_plot_path, bbox_inches='tight')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Box plot saved to {box_plot_path}")
# Step 2: Create the box plot
plt.figure(figsize=(10, 6))
sns.violinplot(
    data=ad_first_diagnosis,
    x='infection_group',
    y='first_ad_age',
    palette='tab10',
    order=['0', '1', '2', '3+'],
    showfliers=False
)

# Step 3: Enhance the plot
plt.title(f'Distribution of Age at First {di} Diagnosis by Infection Group', fontsize=16)
plt.xlabel('Infection Group', fontsize=14)
plt.ylabel(f'Age at First {di} Diagnosis', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
# Step 4: Save the plot
box_plot_path = f"./download/{di}_first_diagnosis_violin_plot.pdf"
plt.savefig(box_plot_path, bbox_inches='tight')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Box plot saved to {box_plot_path}")

In [ ]:
# Step 1: Calculate cumulative probability for age of first encounter for all diseases
first_encounter_diseases = (
    combined_age_data_cleaned
    .groupby(['disease_name', 'person_id'])['age_at_diagnosis']
    .min()
    .reset_index()
)
# Ensure `age_at_diagnosis` is retained correctly in disease_probabilities
disease_probabilities = (
    first_encounter_diseases
    .sort_values('age_at_diagnosis')
    .groupby('disease_name', group_keys=False)
    .apply(lambda x: x.assign(cumulative_probability=x['age_at_diagnosis'].rank(pct=True)))
    .reset_index(drop=True)
)

# Calculate patient counts for each disease
disease_patient_counts = first_encounter_diseases['disease_name'].value_counts().to_dict()

# Create a mapping of disease names to labels with counts
disease_label_mapping = {
    "alzheimer": f"{di} (n={disease_patient_counts.get('alzheimer', 0)})",
    "pseudomonas": f"Pseudomonas (n={disease_patient_counts.get('pseudomonas', 0)})",
    "staphylococcus": f"Staphylococcus (n={disease_patient_counts.get('staphylococcus', 0)})",
    "streptococcus": f"Streptococcus (n={disease_patient_counts.get('streptococcus', 0)})",
    "ecoli": f"E. coli (n={disease_patient_counts.get('ecoli', 0)})",
    "gram_positive": f"Gram-positive (n={disease_patient_counts.get('gram_positive', 0)})",
    "gram_negative": f"Gram-negative (n={disease_patient_counts.get('gram_negative', 0)})",
    "retrovirus": f"Retrovirus (n={disease_patient_counts.get('retrovirus', 0)})",
}
# import matplotlib.cm as cm
# from lifelines import KaplanMeierFitter
# from collections import defaultdict
# import statsmodels.formula.api as smf
# from scipy.stats import mannwhitneyu
# import itertools
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# from scipy.stats import f_oneway
# from scipy.stats import skew, kurtosis
# from lifelines import KaplanMeierFitter
# import matplotlib.lines as mlines
# from statsmodels.stats.multitest import multipletests

colors = cm.get_cmap('tab10')
# Plot cumulative probability vs. age for each disease

plt.figure(figsize=(12, 8))
for idx, (disease, group) in enumerate(disease_probabilities.groupby('disease_name')):
    plt.plot(
        group['age_at_diagnosis'],
        group['cumulative_probability'],
        label=disease_label_mapping.get(disease, disease),  # Use label mapping
        linewidth=2.5,  # Make lines thicker
        color=colors(idx % 10)  # Use a bright, distinguishable color
    )
plt.title('Cumulative probability of first encounter by disease', fontsize=16)
plt.xlabel('Age at first encounter', fontsize=14)
plt.ylabel('Cumulative probability', fontsize=14)
plt.legend(title='Disease name', fontsize=10, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tight_layout()
# Save the plot
plt.savefig(f'./download/cumulative_probability_first_encounter_{di}.pdf', format='pdf')
plt.show()


In [ ]:
# Define dementia_ages based on the dataset
dementia_ages = (
    combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
    .groupby('person_id')['age_at_diagnosis']
    .min()
    .reset_index()
)

# Recompute cumulative probabilities for dementia vs other diseases
disease_vs_dementia_age = pd.merge(
    first_encounter_diseases,
    dementia_ages.rename(columns={'age_at_diagnosis': 'age_at_first_dementia'}),
    on='person_id',
    how='inner'
)

dementia_probabilities = (
    disease_vs_dementia_age
    .sort_values('age_at_first_dementia')
    .groupby('disease_name', group_keys=False)
    .apply(lambda x: x.assign(cumulative_probability=x['age_at_first_dementia'].rank(pct=True)))
    .reset_index(drop=True)
)

# Calculate patient counts for each disease in the dementia-related dataset
disease_patient_counts_dementia = disease_vs_dementia_age['disease_name'].value_counts().to_dict()

# Update disease_label_mapping with counts specific to the dementia dataset
disease_label_mapping_dementia = {
    "alzheimer": f"{di} (n={disease_patient_counts_dementia.get('alzheimer', 0)})",
    "pseudomonas": f"Pseudomonas (n={disease_patient_counts_dementia.get('pseudomonas', 0)})",
    "staphylococcus": f"Staphylococcus (n={disease_patient_counts_dementia.get('staphylococcus', 0)})",
    "streptococcus": f"Streptococcus (n={disease_patient_counts_dementia.get('streptococcus', 0)})",
    "ecoli": f"E. coli (n={disease_patient_counts_dementia.get('ecoli', 0)})",
    "gram_positive": f"Gram-positive (n={disease_patient_counts_dementia.get('gram_positive', 0)})",
    "gram_negative": f"Gram-negative (n={disease_patient_counts_dementia.get('gram_negative', 0)})",
    "retrovirus": f"Retrovirus (n={disease_patient_counts_dementia.get('retrovirus', 0)})",
}

# Plot cumulative probability vs. age of first dementia encounter
plt.figure(figsize=(12, 8))
for disease, group in dementia_probabilities.groupby('disease_name'):
    plt.plot(
        group['age_at_first_dementia'],
        group['cumulative_probability'],
        linewidth=2.5,
        label=disease_label_mapping_dementia.get(disease, f"{disease} (n={disease_patient_counts_dementia.get(disease, 0)})")
    )
plt.title(f'Cumulative probability of diseases by age at first {di} encounter', fontsize=16)
plt.xlabel(f'Age at first {di} encounter', fontsize=14)
plt.ylabel('Cumulative probability', fontsize=14)
plt.legend(title='Disease name', fontsize=10, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tight_layout()
# Save the plot
plt.savefig(f'./download/cumulative_probability_first_{di}_encounter.pdf', format='pdf')
plt.show()



In [ ]:
# Step 1: Define the list of bacterial infections
bacteria_list = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']

# Step 2: Filter patients with Alzheimer's
alzheimers_patients = combined_age_data_filtered.loc[
    combined_age_data_filtered['disease_name'].str.contains('alzheimer', case=False), 'person_id'
].unique()

# Step 3: Filter bacterial data for patients with Alzheimer's
bacterial_data_alz = combined_age_data_filtered[
    (combined_age_data_filtered['person_id'].isin(alzheimers_patients)) &
    (combined_age_data_filtered['disease_name'].isin(bacteria_list))
]

# Step 4: Identify patients with no bacterial infections
alz_no_infections = len(set(alzheimers_patients) - set(bacterial_data_alz['person_id']))

# Step 5: Count infections for each person and classify them
infection_summary_alz = (
    bacterial_data_alz.groupby('disease_name')['person_id']
    .apply(lambda x: pd.Series({
        '1 Infection': (x.value_counts() == 1).sum(),
        '2+ Infections': (x.value_counts() > 1).sum()
    }))
    .unstack()  # Convert the Series into DataFrame columns
    .reset_index()
)

# Add the "0 Infections" row for clarity
infection_summary_alz.loc[len(infection_summary_alz)] = {
    'Bacteria': 'No bacterial infections',
    '1 Infection': 0,
    '2+ Infections': 0
}

# Correct totals for "No bacterial infections"
infection_summary_alz.at[len(infection_summary_alz) - 1, '1 Infection'] = alz_no_infections

In [ ]:
min_first_ad_age = 0
# Step 1: Count total bacterial infections for each person
bacteria_list = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']
bacteria_counts = (
    combined_age_data_cleaned[
        combined_age_data_cleaned['disease_name'].isin(bacteria_list)
    ]
    .groupby('person_id')['disease_name']
    .count()  # Count total infections (including duplicates)
    .reset_index()
    .rename(columns={'disease_name': 'bacteria_infections'})
)

# Step 2: Identify the first AD diagnosis for each person
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_first_diagnosis = (
    ad_data.groupby('person_id')['age_at_diagnosis']
    .min()
    .reset_index()
    .rename(columns={'age_at_diagnosis': 'first_ad_age'})
)

# Step 3: Merge bacterial infection counts with first AD diagnosis age
ad_first_diagnosis = ad_first_diagnosis.merge(bacteria_counts, on='person_id', how='left').fillna(0)

# Step 4: Filter by minimum age at first AD diagnosis

ad_first_diagnosis = ad_first_diagnosis[ad_first_diagnosis['first_ad_age'] > min_first_ad_age]

# Step 5: Categorize into infection groups
ad_first_diagnosis['infection_group'] = ad_first_diagnosis['bacteria_infections'].apply(
    lambda x: '0' if x == 0 else ('1' if x == 1 else ('2' if x == 2 else "3+"))
)


# Step 6: Perform Kaplan-Meier Analysis
kmf = KaplanMeierFitter()
plt.figure(figsize=(12, 8))
colors = {'0': 'blue', '1': 'purple', '2': 'green', '3+': 'red'}
# Add general dementia line
kmf.fit(
    durations=ad_first_diagnosis['first_ad_age'],
    event_observed=[1] * len(ad_first_diagnosis),
    label=f"{di}"
)
kmf.plot_survival_function(color='black', linestyle='--', linewidth=2,ci_show=False)
# Log-Rank Test
log_rank_results = []
groups = ['0', '1', '2', '3+']
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        if group_1 in group_survival_data and group_2 in group_survival_data:
            result = logrank_test(
                group_survival_data[group_1],
                group_survival_data[group_2]
            )
            log_rank_results.append((group_1, group_2, result.p_value))

log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)

# Create custom legend labels with p-values
group_labels = {}
for group in groups:
    if group == '0':
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])})"
    else:
        p_val = min([res[2] for res in log_rank_results if res[0] == '0' and res[1] == group])
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])}, p={p_val:.4e})"

# Prepare Kaplan-Meier plot and survival data
group_survival_data = {}
for group in ['0', '1', '2', '3+']:
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
    group_survival_data[group] = group_data['first_ad_age']
    kmf.fit(
        durations=group_data['first_ad_age'], 
        event_observed=[1] * len(group_data),  # Assume all events observed
        label=group_labels[group]
    )
    kmf.plot_survival_function(color=colors[group], ci_show=False)



# Enhance the plot
plt.title(f'Kaplan-Meier plot for first {di} diagnosis age by infection group', fontsize=16)
plt.xlabel(f'Age at first {di} diagnosis', fontsize=14)
plt.ylabel('Survival probability', fontsize=14)
plt.legend(title='Infection group', fontsize=12, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Show the plot
plt.tight_layout()
reversed_survival_plot_path = f"./download/first_ad_diagnosis_by_infection_group_with_logrank_{di}.pdf"
plt.savefig(reversed_survival_plot_path, bbox_inches='tight')
plt.show()

# Step 6: Perform Kaplan-Meier Analysis
kmf = KaplanMeierFitter()
plt.figure(figsize=(12, 8))
colors = {'0': 'blue', '1': 'purple', '2': 'green', '3+': 'red'}
# Add general dementia line
kmf.fit(
    durations=ad_first_diagnosis['first_ad_age'],
    event_observed=[1] * len(ad_first_diagnosis),
    label=f"{di}"
)
kmf.plot_survival_function(color='black', linestyle='--', linewidth=2,ci_show=False)
# Log-Rank Test
log_rank_results = []
groups = ['0', '1', '2', '3+']
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        if group_1 in group_survival_data and group_2 in group_survival_data:
            result = logrank_test(
                group_survival_data[group_1],
                group_survival_data[group_2]
            )
            log_rank_results.append((group_1, group_2, result.p_value))

log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)

# Create custom legend labels with p-values
group_labels = {}
for group in groups:
    if group == '0':
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])})"
    else:
        p_val = min([res[2] for res in log_rank_results if res[0] == '0' and res[1] == group])
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])}, p={p_val:.4e})"

# Prepare Kaplan-Meier plot and survival data
group_survival_data = {}
for group in ['0', '1', '2', '3+']:
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
    group_survival_data[group] = group_data['first_ad_age']
    kmf.fit(
        durations=group_data['first_ad_age'], 
        event_observed=[1] * len(group_data),  # Assume all events observed
        label=group_labels[group]
    )
    kmf.plot_survival_function(color=colors[group], ci_show=False)



# Enhance the plot
plt.title(f'Kaplan-Meier plot for first {di} diagnosis age by infection group', fontsize=16)
plt.xlabel(f'Age at first {di} diagnosis', fontsize=14)
plt.ylabel('Survival probability', fontsize=14)
plt.legend(title='Infection group', fontsize=12, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.gca().invert_yaxis()
# Show the plot
plt.tight_layout()
reversed_survival_plot_path = f"./download/first_ad_diagnosis_by_infection_group_with_logrank_{di}_reverse.pdf"
plt.savefig(reversed_survival_plot_path, bbox_inches='tight')
plt.show()


In [ ]:
# Step 1: Filter for first occurrence of AD per patient
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_data_first = ad_data.sort_values(by='age_at_diagnosis').drop_duplicates(subset='person_id')

# Step 2: Count unique AD patients and create sequential IDs
num_ad_patients = len(ad_data_first)
ad_data_first['sequential_id'] = range(1, num_ad_patients + 1)
id_mapping = ad_data_first.set_index('person_id')['sequential_id'].to_dict()

# Step 3: Add sequential IDs to the full dataset
filtered_data = combined_age_data_cleaned[combined_age_data_cleaned['person_id'].isin(id_mapping.keys())].copy()
filtered_data['sequential_id'] = filtered_data['person_id'].map(id_mapping)

# Step 4: Separate data into before and after AD onset
# Before AD onset: All diseases except AD below the AD onset age
before_ad_data = []
after_ad_data = []

for person_id in ad_data_first['person_id']:
    person_data = filtered_data[filtered_data['person_id'] == person_id]
    ad_age = person_data[person_data['disease_name'] == 'alzheimer']['age_at_diagnosis'].iloc[0]
    
    # Data before AD onset (only bacterial infections, exclude Alzheimer's)
    before_ad_data.append(person_data[
        (person_data['age_at_diagnosis'] < ad_age) &
        (person_data['disease_name'].str.contains('|'.join(bacterial_infections), case=False))
    ])
    
    # Data after AD onset (Alzheimer only)
    after_ad_data.append(person_data[
        (person_data['age_at_diagnosis'] >= ad_age) & 
        (person_data['disease_name'] == 'alzheimer')
    ])

before_ad_data = pd.concat(before_ad_data)
after_ad_data = pd.concat(after_ad_data)

# # Display Results
# print(f"Before AD Data: {before_ad_data.shape[0]} rows")
# print(f"After AD Data: {after_ad_data.shape[0]} rows")

# # Preview the dataframes
# print("\nBefore AD Data:")
# print(before_ad_data.head())

# print("\nAfter AD Data:")
# print(after_ad_data.head())


In [ ]:
# Define the diseases to include and their order for consistent coloring
disease_list = [
    'staphylococcus', 'ecoli', 'pseudomonas', 
    'gram_negative', 'gram_positive', 'streptococcus'
]

# Define the color order for the diseases
color_order = {
    'staphylococcus': 'blue', 'ecoli': 'green', 
    'pseudomonas': 'black',  # Pseudomonas in black as per request
    'gram_negative': 'orange', 'gram_positive': 'red', 
    'streptococcus': 'brown'
}

# Define the labels for the legend
legend_labels = {
    'staphylococcus': 'Staphylococcus',
    'ecoli': 'E. coli',
    'pseudomonas': 'Pseudomonas',
    'gram_negative': 'Gram-negative Bacteria',
    'gram_positive': 'Gram-positive Bacteria',
    'streptococcus': 'Streptococcus'
}

# Step 5: Plot data
plt.figure(figsize=(20, 10))

# Plot bacterial infections before AD onset
sns.scatterplot(
    data=before_ad_data,
    x='sequential_id',
    y='age_at_diagnosis',
    hue='disease_name',
    palette=color_order,
    hue_order=disease_list,
    s=100,
    alpha=0.7,
    zorder=2,
    legend='full'
)

# Highlight Alzheimer's after AD onset
sns.scatterplot(
    data=after_ad_data[after_ad_data['disease_name'] == 'alzheimer'],
    x='sequential_id',
    y='age_at_diagnosis',
    color='lightblue',
    s=100,
    alpha=0.7,
    label=f"{di} after anset age",
    zorder=1
)

# Plot bacterial infections after AD onset
sns.scatterplot(
    data=after_ad_data[after_ad_data['disease_name'] != 'alzheimer'],
    x='sequential_id',
    y='age_at_diagnosis',
    hue='disease_name',
    palette=color_order,
    hue_order=disease_list,
    s=100,
    alpha=0.7,
    zorder=4,
    legend=False
)

# Add a line for AD onset ages
plt.plot(
    ad_data_first['sequential_id'],
    ad_data_first['age_at_diagnosis'],
    color='blue',
    label=f'{di} onset age',
    linewidth=3,
    zorder=5
)

# Enhance the plot
plt.title(f'Diseases before and after onset age of {di}', fontsize=16)
plt.xlabel(f'Patient ID', fontsize=16)
plt.ylabel('Age at diagnosis', fontsize=16)

# Adjust legend
handles, labels = plt.gca().get_legend_handles_labels()
unique_legend = dict(zip(labels, handles))  # Remove duplicates
plt.legend(
    unique_legend.values(),
    unique_legend.keys(),
    title='Disease',
    fontsize=12,
    frameon=False,
    loc='upper right',
    bbox_to_anchor=(1.2, 1)
)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Improve tick readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Dynamically set X-axis limits
plt.xlim(-5, num_ad_patients + 5)
plt.grid(False)
# Save the plot
output_path = f"./download/diseases_before_after_onset_Best_only_for_those_with_bacteria_infection_before_{di}.pdf"
plt.savefig(output_path, bbox_inches='tight')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Plot saved to {output_path}")


In [ ]:
# Define diseases to include and their order
disease_list = [
    'staphylococcus', 'ecoli', 'pseudomonas', 
    'gram_negative', 'gram_positive', 'streptococcus', 'alzheimer'
]

# Define color order for the diseases
color_order = {
    'staphylococcus': 'blue', 'ecoli': 'green', 
    'pseudomonas': 'black',
    'gram_negative': 'orange', 'gram_positive': 'red', 
    'streptococcus': 'brown', 'alzheimer': 'lightblue'
}

# Step 1: Filter for Alzheimer's data
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_data_first = ad_data.sort_values(by='age_at_diagnosis').drop_duplicates(subset='person_id')

# Step 2: Create sequential IDs for Alzheimer's patients
num_ad_patients = len(ad_data_first)
ad_data_first['sequential_id'] = range(1, num_ad_patients + 1)
id_mapping = ad_data_first.set_index('person_id')['sequential_id'].to_dict()

# Step 3: Add sequential IDs to the full dataset
combined_age_data_cleaned['sequential_id'] = combined_age_data_cleaned['person_id'].map(id_mapping)

# Step 4: Separate data into before and after AD onset
before_ad_data = []
after_ad_data = []

for person_id in ad_data_first['person_id']:
    person_data = combined_age_data_cleaned[combined_age_data_cleaned['person_id'] == person_id]
    ad_age = person_data[person_data['disease_name'] == 'alzheimer']['age_at_diagnosis'].iloc[0]
    
    # Data before AD onset (bacterial infections only)
    before_ad_data.append(person_data[
        (person_data['age_at_diagnosis'] < ad_age) & 
        (person_data['disease_name'].isin(disease_list[:-1]))  # Exclude 'alzheimer'
    ])
    
    # Data after AD onset (bacterial infections + Alzheimer)
    after_ad_data.append(person_data[
        (person_data['age_at_diagnosis'] >= ad_age) & 
        (person_data['disease_name'].isin(disease_list))  # Include all diseases
    ])

before_ad_data = pd.concat(before_ad_data, ignore_index=True)
after_ad_data = pd.concat(after_ad_data, ignore_index=True)

# Step 5: Plot data
plt.figure(figsize=(20, 10))

# Plot Alzheimer's data as a background layer
sns.scatterplot(
    data=after_ad_data[after_ad_data['disease_name'] == 'alzheimer'],
    x='sequential_id',
    y='age_at_diagnosis',
    color='lightblue',
#         color='#A3C1DA',
    s=100,  # Larger size to emphasize the background
    alpha=1,  # More transparency for background effect
    zorder=1,
    label=f'{di}',
     edgecolor='none'
)

# Plot bacterial infections before AD onset
sns.scatterplot(
    data=before_ad_data,
    x='sequential_id',
    y='age_at_diagnosis',
    hue='disease_name',
    palette=color_order,
    hue_order=disease_list[:-1],
    s=100,
    alpha=0.7,
    zorder=2,
    legend='full',
     edgecolor='none'
)

# Plot bacterial infections after AD onset
sns.scatterplot(
    data=after_ad_data[after_ad_data['disease_name'] != 'alzheimer'],
    x='sequential_id',
    y='age_at_diagnosis',
    hue='disease_name',
    palette=color_order,
    hue_order=disease_list[:-1],
    s=100,
    alpha=0.7,
    zorder=3,
    legend='full',
     edgecolor='none'
)

# Add a line for AD onset ages
plt.plot(
    ad_data_first['sequential_id'],
    ad_data_first['age_at_diagnosis'],
    color='blue',
    label=f'{di} onset age',
    linewidth=1,
    zorder=5
)

# Enhance the plot
plt.title(f'Diseases before and after onset age of {di}', fontsize=16)
plt.xlabel('Patient ID', fontsize=16)
plt.ylabel('Age at diagnosis', fontsize=16)

# Adjust legend
handles, labels = plt.gca().get_legend_handles_labels()
unique_legend = dict(zip(labels, handles))  # Remove duplicates
plt.legend(
    unique_legend.values(),
    unique_legend.keys(),
    title='Disease',
    fontsize=12,
    frameon=False,
    loc='upper right',
    bbox_to_anchor=(1.2, 1)
)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Improve tick readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Dynamically set X-axis limits
plt.xlim(-5, num_ad_patients + 5)
plt.grid(False)

# Save the plot
output_path =f"./download/diseases_before_after_onset_with_background_{di}.pdf"
plt.savefig(output_path, bbox_inches='tight')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Plot saved to {output_path}")



In [ ]:
combined_age_data_cleaned.columns

In [ ]:
# Count the number of patients per disease in combined_age_data_cleaned
patient_counts_per_disease = combined_age_data_cleaned.groupby('disease_name')['person_id'].nunique()
patient_counts_per_disease

In [ ]:
# First, find unique IDs for each disease
unique_ids_per_disease = {
    disease: set(combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == disease]['person_id'].unique())
    for disease in combined_age_data_cleaned['disease_name'].unique()
}

# Separate unique IDs for Alzheimer and other disorders
alzheimer_ids = unique_ids_per_disease.pop('alzheimer', set())

# Find unique IDs for each other disorder
other_disorder_ids = {disease: ids for disease, ids in unique_ids_per_disease.items()}

# Calculate the intersection of IDs (common IDs) between Alzheimer and each other disorder
common_ids_counts = {
    disease: len(alzheimer_ids.intersection(ids))
    for disease, ids in other_disorder_ids.items()
}

# Prepare the results
results = {
    'Alzheimer': len(alzheimer_ids),
    **{disease: len(ids) for disease, ids in other_disorder_ids.items()},
    'Common': common_ids_counts
}

import pandas as pd
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Count'])
results_df 



In [ ]:
# Total unique IDs for each disease
disease_counts = {disease: len(ids) for disease, ids in unique_ids_per_disease.items()}

# Common IDs between Alzheimer and each disorder
common_ids_with_ad = {
    disease: len(alzheimer_ids.intersection(ids))
    for disease, ids in unique_ids_per_disease.items()
}

# Combine the data into a DataFrame
disease_overlap_df = pd.DataFrame({
    "Disease": list(disease_counts.keys()),
    "disease_count": list(disease_counts.values()),
    "common_id_with_AD_disease": list(common_ids_with_ad.values())
})

In [ ]:
disease_overlap_df


In [ ]:
# Filter for patients with Alzheimer's disease in `combined_age_data_cleaned`
alzheimer_data_test = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']

# Find the earliest age of Alzheimer diagnosis for each person
earliest_ad_diagnosis = alzheimer_data_test.groupby('person_id').apply(
    lambda group: group.loc[group['age_at_diagnosis'].idxmin()]
).reset_index(drop=True)

# Calculate the age of onset for dementia (earliest age of Alzheimer diagnosis)
earliest_ad_diagnosis['age_onset_of_alzheimer'] = earliest_ad_diagnosis['age_at_diagnosis']

# Select relevant columns for the final DataFrame
ad_earliest_diagnosis_df = earliest_ad_diagnosis[[
    'person_id', 'age_at_diagnosis', 'disease_name', 'age', 'diagnosis_year_month', 'date_of_birth', 'age_onset_of_alzheimer'
]]
# ad_earliest_diagnosis_df

In [ ]:
combined_age_data_cleaned.columns

In [ ]:
# Add a new column to `ad_earliest_diagnosis_df` for the start of the 5-year window
ad_earliest_diagnosis_df['start_window'] = ad_earliest_diagnosis_df['age_onset_of_alzheimer'] - 5

# Merge `combined_age_data_cleaned` with `ad_earliest_diagnosis_df` to get all diseases for AD patients
merged_data = combined_age_data_cleaned.merge(
    ad_earliest_diagnosis_df[['person_id', 'start_window', 'age_onset_of_alzheimer']],
    on='person_id',
    how='inner'
)

# Filter the data for diseases within the 5-year window before AD onset
merged_data['within_window'] = (
    (merged_data['age_at_diagnosis'] >= merged_data['start_window']) &
    (merged_data['age_at_diagnosis'] <= merged_data['age_onset_of_alzheimer'])
)

# Filter only rows within the window
diseases_within_window = merged_data[merged_data['within_window']]

# Count how many patients had each disease within the 5-year window
disease_counts_within_window = diseases_within_window['disease_name'].value_counts()

# Convert to a DataFrame for better presentation
disease_counts_within_window_df = disease_counts_within_window.reset_index()
disease_counts_within_window_df.columns = ['Disease', 'Count']

In [ ]:
disease_counts_within_window_df

In [ ]:
# Step 1: Add categories based on age relative to AD onset
merged_data['time_category'] = 'Other'

# 5 years before AD onset (exclusive of earlier categories)
merged_data.loc[
    (merged_data['age_at_diagnosis'] >= merged_data['start_window']) &
    (merged_data['age_at_diagnosis'] <= merged_data['age_onset_of_alzheimer']),
    'time_category'
] = '5 years before AD'

# Before 5 years window
merged_data.loc[
    (merged_data['age_at_diagnosis'] < merged_data['start_window']),
    'time_category'
] = 'Before 5 years'

# After AD onset
merged_data.loc[
    (merged_data['age_at_diagnosis'] > merged_data['age_onset_of_alzheimer']),
    'time_category'
] = 'After AD onset'

# Step 2: Assign categories uniquely, prioritizing '5 years before AD'
merged_data = merged_data.sort_values(by='time_category', key=lambda col: col.map({
    '5 years before AD': 1,
    'Before 5 years': 2,
    'After AD onset': 3,
    'Other': 4
})).drop_duplicates(subset=['person_id', 'disease_name'])

# Step 3: Count diseases in each category
category_counts = (
    merged_data.groupby(['time_category', 'disease_name'])['person_id']
    .nunique()
    .reset_index()
    .rename(columns={'person_id': 'Patient_Count'})
)
# category_counts

In [ ]:
# Set the plot style
sns.set(style="whitegrid")

# Create the bar plot
plt.figure(figsize=(14, 8))
bar_plot = sns.barplot(
    data=category_counts[category_counts['disease_name']!='alzheimer'],
    x='disease_name',
    y='Patient_Count',
    hue='time_category',
    palette='tab10'
)

# Enhance the plot
plt.title("Patient Count by Disease and Time Category", fontsize=16)
plt.xlabel("Disease Name", fontsize=14)
plt.ylabel("Patient Count", fontsize=14)
plt.xticks(rotation=45, fontsize=12, ha="right")
plt.legend(title="Time Category", fontsize=12,frameon=False)
# Customize the plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

plt.tight_layout()
plt.grid(False)

# Save the plot
output_plot_path = "./download/patient_count_by_disease_time_category.pdf"
plt.savefig(output_plot_path, dpi=300, bbox_inches="tight")

# Show the plot
plt.show()

print(f"Plot saved to: {output_plot_path}")

In [ ]:
# Replace values in 'Patient_Count' column if less than 20
category_counts['Patient_Count'] = category_counts['Patient_Count'].apply(
    lambda x: '<20' if x < 20 else x
)

# Save the modified DataFrame to a new file
category_counts_path = "./download/modified_category_counts.tsv"
category_counts.to_csv(category_counts_path, sep='\t', index=False)

# Print the file path
print(f"Modified category_counts saved to: {category_counts_path}")

# Display the modified DataFrame
category_counts


In [ ]:
category_counts.to_csv(f'./download/Disease_infection_5_year_pior_of_{di}.tsv', sep='\t', index=False)

In [ ]:
# Save the results to a file in the specified directory
output_path = "./download/disease_count_by_time_category.tsv"
category_counts.to_csv(output_path, sep='\t', index=False)

print(f"Disease count by time category saved to: {output_path}")


In [ ]:
def test_age_difference_direction(df, disease_dfs):
    """
    Perform one-sample t-test to determine if a disease occurs before or after AD.
    """
    results = []
    for disease in disease_dfs.keys():
        age_difference_col = f"Age_difference_AD_{disease}"
        if age_difference_col in df.columns:
            data = df[age_difference_col].dropna()
            if len(data) > 1:
                # Perform one-sample t-test
                t_stat, t_pval = ttest_1samp(data, 0)

                # Determine direction
                mean_difference = data.median()
                if mean_difference < 0:
                    direction = "First AD"
                elif mean_difference > 0:
                    direction = "First Disease"
                else:
                    direction = "No Difference"

                # Append results
                results.append({
                    "Disease": disease,
                    "Mean Age Difference": mean_difference,
                    "T-Test Statistic": t_stat,
                    "T-Test P-Value": t_pval,
                    "Direction": direction
                })
    return pd.DataFrame(results)
# Function to annotate with direction
def annotate_with_direction(ax, test_results):
    for i, result in enumerate(test_results.to_dict(orient="records")):
        disease = result["Disease"]
        direction = result["Direction"]
        t_pval = result["T-Test P-Value"]
        ax.text(
            i,
            ax.get_ylim()[1] - 1,  # Adjust based on data range
            f"{direction}\nP(T): {mean_differencel:.3f}",
            ha="center",
            fontsize=10,
            color="black"
        )
direction_test_results = test_age_difference_direction(final_combined_df, disease_dfs)
direction_test_results
from scipy.stats import ttest_1samp, wilcoxon
from statsmodels.stats.proportion import proportions_ztest

# Function to prepare combined_plot_data
def prepare_plot_data(df, disease_dfs):
    """
    Prepare data for plotting Age Difference by Disease.
    """
    plot_data = []
    for disease in disease_dfs.keys():
        age_difference_col = f"Age_difference_AD_{disease}"
        if age_difference_col in df.columns:
            temp_df = df[[age_difference_col]].dropna()
            temp_df["Disease"] = disease
            temp_df["Age Difference"] = temp_df[age_difference_col]
            plot_data.append(temp_df[["Disease", "Age Difference"]])
    return pd.concat(plot_data, ignore_index=True)

# Statistical tests function
def test_age_difference(df, disease_dfs):
    """
    Perform one-sample t-test, Wilcoxon test, and proportion test for Age Difference.
    """
    results = []
    for disease in disease_dfs.keys():
        age_difference_col = f"Age_difference_AD_{disease}"
        if age_difference_col in df.columns:
            data = df[age_difference_col].dropna()
            if len(data) > 1:
                # T-test
                t_stat, t_pval = ttest_1samp(data, 0)

                # Wilcoxon signed-rank test
                try:
                    w_stat, w_pval = wilcoxon(data - 0)
                except ValueError:
                    w_stat, w_pval = None, None

                # Proportion test for ±0.5 year difference
                within_half_year = ((data >= -0.5) & (data <= 0.5)).sum()
                n_total = len(data)
                proportion = within_half_year / n_total
                expected_proportion = 0.5  # Null hypothesis: 50% of cases fall within ±0.5 years
                z_stat, prop_pval = proportions_ztest(within_half_year, n_total, value=expected_proportion)

                results.append({
                    "Disease": disease,
                    "T-Test P-Value": t_pval,
                    "Wilcoxon P-Value": w_pval,
                    "Proportion P-Value": prop_pval,
                    "Mean Age Difference": data.mean(),
                    "Median Age Difference": data.median(),
                    "Proportion ±0.5 Year": proportion
                })
    return pd.DataFrame(results)

# Function to annotate plot with p-values
def annotate_plot_with_pvalues(ax, test_results):
    for i, result in enumerate(test_results.to_dict(orient="records")):
        disease = result["Disease"]
        t_pval = result["T-Test P-Value"]
        prop_pval = result["Proportion P-Value"]
        ax.text(
            i,
            ax.get_ylim()[1] - 3,  # Adjust based on data range
            f"T-Test P: {t_pval:.3f}\n",
            ha="center",
            fontsize=10,
            color="black"
        )

# Simulated data
diseases = ['ecoli', 'gram_negative', 'gram_positive', 'pseudomonas', 'staphylococcus', 'streptococcus', 'retrovirus']
disease_dfs = {disease: None for disease in diseases}

# Prepare data
combined_plot_data = prepare_plot_data(final_combined_df, disease_dfs)

# Perform statistical tests
one_sample_test_results = test_age_difference(final_combined_df, disease_dfs)

# Annotate with direction and t-test results for direction
def annotate_with_direction(ax, test_results):
    for i, result in enumerate(test_results.to_dict(orient="records")):
        disease = result["Disease"]
        direction = result["Direction"]
        t_pval = result["T-Test P-Value"]
        mean_difference= result["Mean Age Difference"]
        ax.text(
            i,
            ax.get_ylim()[1] - 4.4,  # Adjust based on data range
            f"{direction}\nMedian: {mean_difference:.3f}",
            ha="center",
            fontsize=10,
            color="black"
        )

# Run direction test
direction_test_results = test_age_difference_direction(final_combined_df, disease_dfs)

# Plot the existing graph
plt.figure(figsize=(14, 12))
sns.violinplot(
    x="Disease",
    y="Age Difference",
    data=combined_plot_data,
    palette="Set2",
    showfliers=False
)
sns.stripplot(
    x="Disease",
    y="Age Difference",
    data=combined_plot_data,
    color="black",
    alpha=0.5,
    jitter=True
)
plt.axhline(-0.5, color="Grey", linestyle="--", linewidth=1.5, label="-1 Year Difference")
plt.axhline(+0.5, color="Grey", linestyle="--", linewidth=1.5, label="+1 Year Difference")
plt.axhline(0, color="red", linestyle="--", linewidth=1.5, label="Same Time Reference")
plt.title("Distribution of Age Difference (AD Onset - Disease Onset) by Disease", fontsize=16)
plt.xlabel("Disease", fontsize=14)
plt.ylabel("Age Difference (Years)", fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.legend(loc="upper right", bbox_to_anchor=(1.2, 0.9), frameon=False)
plt.grid(axis='y', linestyle='--', linewidth=0.1)
plt.ylim(-45, 45)
# Annotate counts below each x-axis label
disease_counts = combined_plot_data.groupby("Disease")["Age Difference"].count()

for i, disease in enumerate(disease_counts.index):
    count = disease_counts[disease]
    if count <20:
        plt.text(
            i, ax.get_ylim()[1] - 6,  # Position the text below the x-axis
            f"n is <20",
            ha="center",
            fontsize=10,
            color="black"
        )

    else:
        plt.text(
            i, ax.get_ylim()[1] - 6,  # Position the text below the x-axis
            f"n is {count}",
            ha="center",
            fontsize=10,
            color="black"
        )
# Customize plot spines
ax = plt.gca()
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Annotate with statistical test p-values
annotate_plot_with_pvalues(ax, one_sample_test_results)
annotate_with_direction(ax, direction_test_results)  # Add this annotation

plt.tight_layout()

forest_plot_path = f'./download/Violin_plot_stripplot_{di}.pdf'
plt.savefig(forest_plot_path, format='pdf')
plt.show()




In [ ]:
# Function to prepare combined_plot_data
def prepare_plot_data(df, disease_dfs):
    """
    Prepare data for plotting Age Difference by Disease.
    """
    plot_data = []
    for disease in disease_dfs.keys():
        age_difference_col = f"Age_difference_AD_{disease}"
        if age_difference_col in df.columns:
            temp_df = df[[age_difference_col]].dropna()
            temp_df["Disease"] = disease
            temp_df["Age Difference"] = temp_df[age_difference_col]
            plot_data.append(temp_df[["Disease", "Age Difference"]])
    return pd.concat(plot_data, ignore_index=True)

# Statistical tests function
def test_age_difference(df, disease_dfs):
    """
    Perform one-sample t-test, Wilcoxon test, and proportion test for Age Difference.
    """
    results = []
    for disease in disease_dfs.keys():
        age_difference_col = f"Age_difference_AD_{disease}"
        if age_difference_col in df.columns:
            data = df[age_difference_col].dropna()
            if len(data) > 1:
                # T-test
                t_stat, t_pval = ttest_1samp(data, 0)

                # Wilcoxon signed-rank test
                try:
                    w_stat, w_pval = wilcoxon(data - 0)
                except ValueError:
                    w_stat, w_pval = None, None

                # Proportion test for ±0.5 year difference
                within_half_year = ((data >= -0.5) & (data <= 0.5)).sum()
                n_total = len(data)
                proportion = within_half_year / n_total
                expected_proportion = 0.5  # Null hypothesis: 50% of cases fall within ±0.5 years
                z_stat, prop_pval = proportions_ztest(within_half_year, n_total, value=expected_proportion)

                results.append({
                    "Disease": disease,
                    "T-Test P-Value": t_pval,
                    "Wilcoxon P-Value": w_pval,
                    "Proportion P-Value": prop_pval,
                    "Mean Age Difference": data.mean(),
                    "Median Age Difference": data.median(),
                    "Proportion ±0.5 Year": proportion
                })
    return pd.DataFrame(results)

# Function to annotate plot with p-values
def annotate_plot_with_pvalues(ax, test_results):
    for i, result in enumerate(test_results.to_dict(orient="records")):
        disease = result["Disease"]
        t_pval = result["T-Test P-Value"]
        prop_pval = result["Proportion P-Value"]
        ax.text(
            i,
            ax.get_ylim()[1] - 3,  # Adjust based on data range
            f"T-Test P: {t_pval:.3f}\n",
            ha="center",
            fontsize=10,
            color="black"
        )

# Simulated data
diseases = ['ecoli', 'gram_negative', 'gram_positive', 'pseudomonas', 'staphylococcus', 'streptococcus', 'retrovirus']
disease_dfs = {disease: None for disease in diseases}
# Define a mapping for better disease labels
disease_labels = {
    'ecoli': 'E. coli',
    'pseudomonas': 'Pseudomonas',
    'gram_negative': 'Gram-Negative',
    'gram_positive': 'Gram-Positive',
    'staphylococcus': 'Staphylococcus',
    'streptococcus': 'Streptococcus',
    'retrovirus': 'Retrovirus'
}

# Map the disease labels in the combined_plot_data dataframe
combined_plot_data['Disease'] = combined_plot_data['Disease'].map(disease_labels)

# Perform statistical tests and map disease labels in test results
one_sample_test_results['Disease'] = one_sample_test_results['Disease'].map(disease_labels)

# Run direction test and map disease labels
direction_test_results['Disease'] = direction_test_results['Disease'].map(disease_labels)

# Plot the updated graph
plt.figure(figsize=(14, 12))
sns.violinplot(
    x="Disease",
    y="Age Difference",
    data=combined_plot_data,
    palette="Set2",
    showfliers=False
)
plt.axhline(-0.5, color="Grey", linestyle="--", linewidth=1.5, label="-1 Year Difference")
plt.axhline(+0.5, color="Grey", linestyle="--", linewidth=1.5, label="+1 Year Difference")
plt.axhline(0, color="red", linestyle="--", linewidth=1.5, label="Same Time Reference")
plt.title(f"Distribution of Age Difference ({di} Onset - Disease Onset) by Disease", fontsize=16)
plt.xlabel("Disease", fontsize=14)
plt.ylabel("Age Difference (Years)", fontsize=14)
plt.xticks(rotation=90, fontsize=12)
plt.legend(loc="upper right", bbox_to_anchor=(1.2, 0.9), frameon=False)
plt.grid(axis='y', linestyle='--', linewidth=0.1)
plt.ylim(-45, 45)

# Annotate counts below each x-axis label
disease_counts = combined_plot_data.groupby("Disease")["Age Difference"].count()

for i, disease in enumerate(disease_counts.index):
    count = disease_counts[disease]
    if count < 20:
        plt.text(
            i, ax.get_ylim()[1] - 6,  # Position the text below the x-axis
            f"n < 20",
            ha="center",
            fontsize=10,
            color="black"
        )
    else:
        plt.text(
            i, ax.get_ylim()[1] - 6,  # Position the text below the x-axis
            f"n = {count}",
            ha="center",
            fontsize=10,
            color="black"
        )

# Customize plot spines
ax = plt.gca()
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Annotate with statistical test p-values
annotate_plot_with_pvalues(ax, one_sample_test_results)
annotate_with_direction(ax, direction_test_results)  # Add this annotation

plt.tight_layout()

# Save the plot
forest_plot_path = f'./download/Violin_plot_{di}.pdf'
plt.savefig(forest_plot_path, format='pdf')
plt.show()

In [ ]:
def calculate_age_at_diagnosis(data, disease_name, person_info):
    """
    Calculate the age at disease diagnosis for each person, ensuring no duplicate diagnoses in the same month.
    """
    # Ensure `condition_start_datetime` and `date_of_birth` are in datetime format
    data['condition_start_datetime'] = pd.to_datetime(data['condition_start_datetime'], errors='coerce')
    person_info['date_of_birth'] = pd.to_datetime(person_info['date_of_birth'], errors='coerce')

    # Drop rows where `condition_start_datetime` could not be parsed
    data = data.dropna(subset=['condition_start_datetime'])

    # Convert both to timezone-naive
    data['condition_start_datetime'] = data['condition_start_datetime'].dt.tz_localize(None)
    person_info['date_of_birth'] = person_info['date_of_birth'].dt.tz_localize(None)

    # Extract the year and month of diagnosis
    data['diagnosis_year_month'] = data['condition_start_datetime'].dt.to_period('M')

    # Drop duplicate diagnoses for the same person and disease in the same month
    data = data.drop_duplicates(subset=['person_id', 'diagnosis_year_month'])

    # Merge with `person_info` to get date_of_birth and age
    merged_data = data.merge(person_info, on='person_id', how='left')

    # Calculate the age at diagnosis
    merged_data['age_at_diagnosis'] = (
        (merged_data['condition_start_datetime'] - merged_data['date_of_birth']).dt.days / 365.25
    )

    # Add disease name for clarity
    merged_data['disease_name'] = disease_name

    # Select relevant columns
    age_data = merged_data[['person_id', 'age_at_diagnosis', 'disease_name', 'age', 'diagnosis_year_month','date_of_birth']]

    return age_data



# Assume `person_info` contains `person_id`, `date_of_birth`, and `age`
person_info = final_combined_df[['person_id', 'date_of_birth', 'age']].copy()

# Calculate age at diagnosis for each disease and combine results
all_diagnoses = []
for disease, df in disease_dataframes.items():
    age_data = calculate_age_at_diagnosis(df, disease, person_info)
    all_diagnoses.append(age_data)

# Combine all diseases into a single DataFrame
combined_age_data = pd.concat(all_diagnoses, ignore_index=True)


# Remove rows with NaN values in the 'age' or 'age_at_diagnosis' columns
combined_age_data_cleaned = combined_age_data.dropna(subset=['age', 'age_at_diagnosis'])
# combined_age_data_cleaned

In [ ]:

# alzheimer_data_filtered=alzheimer_data.copy()
# Ensure `final_combined_df` contains the necessary columns
# final_combined_df = final_combined_df[['person_id', 'age', 'sex_at_birth']]

# Convert `age` to numeric
final_combined_df['age'] = pd.to_numeric(final_combined_df['age'], errors='coerce')

# Diseases and their respective dataframes
disease_dataframes = {
    "alzheimer": alzheimer_data_filtered,
    "gram_negative": gram_negative_data_filtered,
    "staphylococcus": staphylococcus_data,
    "pseudomonas": pseudomonas_data,
    "streptococcus": streptococcus_data,
    "ecoli": ecoli_data_filtered,
    "gram_positive": gram_positive_data
}

def calculate_age_at_diagnosis(data, disease_name, person_info):
    """
    Calculate the age at disease diagnosis for each person, ensuring no duplicate diagnoses in the same month.
    """
    # Ensure `condition_start_datetime` is in datetime format
    data['condition_start_datetime'] = pd.to_datetime(data['condition_start_datetime'], errors='coerce')

    # Drop rows where `condition_start_datetime` could not be parsed
    data = data.dropna(subset=['condition_start_datetime'])

    # Extract the year and month of diagnosis
    data['diagnosis_year_month'] = data['condition_start_datetime'].dt.to_period('M')

    # Drop duplicate diagnoses for the same person and disease in the same month
    data = data.drop_duplicates(subset=['person_id', 'diagnosis_year_month'])

    # Merge with `person_info` to get the base age
    merged_data = data.merge(person_info, on='person_id', how='left')

    # Calculate the age at diagnosis
    merged_data['age_at_diagnosis'] = merged_data['age'] + \
        (merged_data['condition_start_datetime'].dt.year - 2022)

    # Add disease name for clarity
    merged_data['disease_name'] = disease_name

    # Select relevant columns
    age_data = merged_data[['person_id', 'age_at_diagnosis', 'disease_name', 'age', 'diagnosis_year_month']]

    return age_data

# Process each disease to calculate ages at diagnosis
age_at_diagnosis_list = []
for disease, df in disease_dataframes.items():
    if 'condition_start_datetime' not in df.columns:
        print(f"Skipping {disease} due to missing 'condition_start_datetime'")
        continue
    disease_ages = calculate_age_at_diagnosis(df, disease, final_combined_df)
    age_at_diagnosis_list.append(disease_ages)

# Combine all ages into one dataframe
final_age_data = pd.concat(age_at_diagnosis_list, ignore_index=True)

# Display the table with ages at diagnosis
# print(final_age_data)

# keep patients that are in the creteria 
final_age_data = final_age_data[final_age_data['person_id'].isin(person_id_list)]
final_age_data
final_age_data = final_age_data[final_age_data['person_id'].isin(person_ids_more_than_10)].copy()
final_age_data
final_age_data[final_age_data['disease_name']=='alzheimer']['person_id'].nunique()
# Step 1: Find the total number of unique individuals for each disease
disease_counts = final_age_data.groupby('disease_name')['person_id'].nunique().reset_index()
disease_counts.columns = ['disease_name', 'total_individuals']

# Step 2: Identify individuals with Alzheimer's disease
alzheimer_individuals = set(final_age_data[final_age_data['disease_name'] == 'alzheimer']['person_id'])

# Step 3: For each disease, calculate overlap with Alzheimer's and age statistics
summary_data = []
for disease in disease_counts['disease_name']:
    if disease == 'alzheimer':
        continue  # Skip Alzheimer's as we are calculating overlap with it
    
    # Get individuals with the current disease
    disease_individuals = set(final_age_data[final_age_data['disease_name'] == disease]['person_id'])
    
    # Calculate overlap and exclusivity
    both_alzheimer_and_disease = alzheimer_individuals & disease_individuals
    only_disease = disease_individuals - alzheimer_individuals
    only_alzheimer = alzheimer_individuals - disease_individuals
    neither = len(set(final_age_data['person_id'])) - len(both_alzheimer_and_disease) - len(only_disease) - len(only_alzheimer)
    
    # Calculate age statistics
    both_ages = final_age_data[final_age_data['person_id'].isin(both_alzheimer_and_disease) & (final_age_data['disease_name'] == disease)]
    only_disease_ages = final_age_data[final_age_data['person_id'].isin(only_disease) & (final_age_data['disease_name'] == disease)]
    
    # Append results
    summary_data.append({
        "Disease": disease,
        "Total_With_Disease": len(disease_individuals),
        f"With_{di}_And_Disease": len(both_alzheimer_and_disease),
        f"Mean_Age_{di}_And_Disease": both_ages['age_at_diagnosis'].mean(),
        f"Median_Age_{di}_And_Disease": both_ages['age_at_diagnosis'].median(),
        "With_Only_Disease": len(only_disease),
        "Mean_Age_Only_Disease": only_disease_ages['age_at_diagnosis'].mean(),
        "Median_Age_Only_Disease": only_disease_ages['age_at_diagnosis'].median(),
        f"With_Only_v": len(only_alzheimer),
        "With_Neither": neither
    })

# Step 4: Convert summary data to a DataFrame
summary_df = pd.DataFrame(summary_data)

# Save the summary DataFrame to a TSV file
summary_df_path = f"./download/{di}_disease_overlap_summary_with_ages.tsv"
summary_df.to_csv(summary_df_path, sep='\t', index=False)

# Display the file path
print(f"Disease overlap summary with ages saved to: {summary_df_path}")
summary_df



In [ ]:
# Ensure the data is sorted by person_id and disease_name
final_age_data = final_age_data.sort_values(by=['person_id', 'disease_name'])

# Filter the data for Alzheimer's disease
alzheimer_data_test = final_age_data[final_age_data['disease_name'] == 'alzheimer']

# Group by person_id and get the minimum age_at_diagnosis for Alzheimer's
min_ad_age = alzheimer_data_test.groupby('person_id', as_index=False)['age_at_diagnosis'].min()

# Rename the columns for clarity
min_ad_age.columns = ['person_id', 'min_ad_age_at_diagnosis']


In [ ]:
# Filter for the first occurrence of AD per patient
ad_data = final_age_data[final_age_data['disease_name'] == 'alzheimer']
ad_data_first = ad_data.sort_values(by='age_at_diagnosis').drop_duplicates(subset='person_id')

# Count unique AD patients
num_ad_patients = len(ad_data_first)
print(f"Number of unique Alzheimer's disease (AD) patients: {num_ad_patients}")

# Create a mapping for sequential IDs
ad_data_first['sequential_id'] = range(1, num_ad_patients + 1)
id_mapping = ad_data_first.set_index('person_id')['sequential_id'].to_dict()

# Add sequential IDs to the full dataset
filtered_data = final_age_data[final_age_data['person_id'].isin(id_mapping.keys())].copy()
filtered_data['sequential_id'] = filtered_data['person_id'].map(id_mapping)

# Sort by sequential ID and age at diagnosis
filtered_data = filtered_data.sort_values(by=['sequential_id', 'age_at_diagnosis'])

# Create separate DataFrames for infections before and after AD onset
pre_ad_data = []
post_ad_data = []
for person_id in ad_data_first['person_id']:
    person_data = filtered_data[filtered_data['person_id'] == person_id]
    ad_age = person_data[person_data['disease_name'] == 'alzheimer']['age_at_diagnosis'].iloc[0]
    
    # Separate diagnoses before and after AD onset
    pre_ad = person_data[person_data['age_at_diagnosis'] <= ad_age]
    post_ad = person_data[person_data['age_at_diagnosis'] > ad_age]
    
    pre_ad_data.append(pre_ad)
    post_ad_data.append(post_ad)

# Combine all data
pre_ad_data = pd.concat(pre_ad_data)
post_ad_data = pd.concat(post_ad_data)

# Add a flag to indicate before or after AD
pre_ad_data['relative_to_ad'] = 'Before AD'
post_ad_data['relative_to_ad'] = 'After AD'

# Combine the two datasets for plotting
combined_data = pd.concat([pre_ad_data, post_ad_data])


In [ ]:

# Apply min-max scaling to the age_at_diagnosis column
min_age = combined_data['age_at_diagnosis'].min()
max_age = combined_data['age_at_diagnosis'].max()

combined_data['scaled_age'] = (combined_data['age_at_diagnosis'] - min_age) / (max_age - min_age)
ad_data_first['scaled_age'] = (ad_data_first['age_at_diagnosis'] - min_age) / (max_age - min_age)

# Smooth AD onset age line
x = ad_data_first['sequential_id']
y = ad_data_first['scaled_age']
valid_indices = ~np.isnan(y)
x_clean = x[valid_indices]
y_clean = y[valid_indices]
x_smooth = np.linspace(x_clean.min(), x_clean.max(), 50)
y_smooth = make_interp_spline(x_clean, y_clean + 0.005)(x_smooth)
color_order = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']

# Separate data for "Before AD" and "After AD"
before_ad = combined_data[(combined_data['relative_to_ad'] == 'Before AD') & (combined_data['disease_name'] != 'alzheimer')]
after_ad = combined_data[(combined_data['relative_to_ad'] == 'After AD') & (combined_data['disease_name'] != 'alzheimer')]

# Combined plot for both before and after AD infections
def combined_line_plot(before_data, after_data, ad_data_first, title, output_path):
    plt.figure(figsize=(18, 9))

    # Plot lines for before AD infections
    for sequential_id in before_data['sequential_id'].unique():
        person_data = before_data[before_data['sequential_id'] == sequential_id]
        ad_scaled_age = ad_data_first.loc[ad_data_first['sequential_id'] == sequential_id, 'scaled_age'].values[0]
        for _, row in person_data.iterrows():
            # Line from infection to AD onset
            plt.plot(
                [sequential_id, sequential_id],
                [row['scaled_age'], ad_scaled_age],
                color='gray',
                linestyle='--',
                alpha=0.5
            )

    # Plot lines for after AD infections
    for sequential_id in after_data['sequential_id'].unique():
        person_data = after_data[after_data['sequential_id'] == sequential_id]
        ad_scaled_age = ad_data_first.loc[ad_data_first['sequential_id'] == sequential_id, 'scaled_age'].values[0]
        for _, row in person_data.iterrows():
            # Line from AD onset to infection
            plt.plot(
                [sequential_id, sequential_id],
                [ad_scaled_age, row['scaled_age']],
                color='gray',
                linestyle='--',
                alpha=0.5
            )

    # Scatter plot for before AD infections
    sns.scatterplot(
        data=before_data,
        x='sequential_id',
        y='scaled_age',
        hue='disease_name',
        palette='tab10',
        hue_order=color_order,
        s=40,
        alpha=0.5
        # label='Before AD'
    )

    # Scatter plot for after AD infections
    sns.scatterplot(
        data=after_data,
        x='sequential_id',
        y='scaled_age',
        hue='disease_name',
        palette='tab10',
        hue_order=color_order,
        s=40,
        alpha=0.5,
        legend=False
        # label='After AD'
    )

    # Add the smooth line for AD onset ages
    plt.plot(
        x_smooth,
        y_smooth,
        color='blue',
        label='Dementia Onset Age (Smooth)',
        linewidth=2.5,
        alpha=0.8
    )

    # Enhance the plot
    plt.title(title, fontsize=16)
    plt.xlabel(f'Sequential Patient ID (Sorted by {di} Onset Age)', fontsize=14)
    plt.ylabel('Scaled Age at Diagnosis (0-1)', fontsize=14)
    plt.xticks(rotation=45, fontsize=10)
    plt.legend(title='Legend', fontsize=12, frameon=False, loc='upper right', bbox_to_anchor=(1.2, 0.6))
    plt.grid(False)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    plt.xlim(-5, num_ad_patients + 5)
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight')
    plt.show()
    print(f"Plot saved to {output_path}")

# Generate combined plot
combined_line_plot(
    before_ad,
    after_ad,
    ad_data_first,
    f"Lines Connecting Before and After {di} Infections to {di} Onset Age",
    "./download/Combined_Lines_Before_After_AD.pdf"
)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import make_interp_spline
import numpy as np

# Smooth AD onset age line using the original `age_at_diagnosis`
x = ad_data_first['sequential_id']
y = ad_data_first['age_at_diagnosis']
valid_indices = ~np.isnan(y)
x_clean = x[valid_indices]
y_clean = y[valid_indices]
x_smooth = np.linspace(x_clean.min(), x_clean.max(), 50)
y_smooth = make_interp_spline(x_clean, y_clean + 0.5)(x_smooth)  # Add slight offset for visibility
color_order = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']

# Combined plot for both before and after AD infections
def combined_line_plot(before_data, after_data, ad_data_first, title, output_path):
    plt.figure(figsize=(18, 9))

    # Plot lines for before AD infections
    for sequential_id in before_data['sequential_id'].unique():
        person_data = before_data[before_data['sequential_id'] == sequential_id]
        ad_age = ad_data_first.loc[ad_data_first['sequential_id'] == sequential_id, 'age_at_diagnosis'].values[0]
        for _, row in person_data.iterrows():
            # Line from infection to AD onset
            plt.plot(
                [sequential_id, sequential_id],
                [row['age_at_diagnosis'], ad_age],
                color='gray',
                linestyle='--',
                alpha=0.5
            )

    # Plot lines for after AD infections
    for sequential_id in after_data['sequential_id'].unique():
        person_data = after_data[after_data['sequential_id'] == sequential_id]
        ad_age = ad_data_first.loc[ad_data_first['sequential_id'] == sequential_id, 'age_at_diagnosis'].values[0]
        for _, row in person_data.iterrows():
            # Line from AD onset to infection
            plt.plot(
                [sequential_id, sequential_id],
                [ad_age, row['age_at_diagnosis']],
                color='gray',
                linestyle='--',
                alpha=0.5
            )

    # Scatter plot for before AD infections
    sns.scatterplot(
        data=before_data,
        x='sequential_id',
        y='age_at_diagnosis',
        hue='disease_name',
        palette='tab10',
        hue_order=color_order,
        s=40,
        alpha=0.5
    )

    # Scatter plot for after AD infections
    sns.scatterplot(
        data=after_data,
        x='sequential_id',
        y='age_at_diagnosis',
        hue='disease_name',
        palette='tab10',
        hue_order=color_order,
        s=40,
        alpha=0.5,
        legend=False
    )

    # Add the smooth line for AD onset ages
    plt.plot(
        x_smooth,
        y_smooth,
        color='blue',
        label='Dementia Onset Age (Smooth)',
        linewidth=2.5,
        alpha=0.8
    )

    # Enhance the plot
    plt.title(title, fontsize=16)
    plt.xlabel('Sequential Patient ID (Sorted by Dementia Onset Age)', fontsize=14)
    plt.ylabel('Age at Diagnosis (Years)', fontsize=14)
    plt.xticks(rotation=45, fontsize=10)
    plt.legend(title='Legend', fontsize=12, frameon=False, loc='upper right', bbox_to_anchor=(1.2, 0.6))
    plt.grid(False)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    plt.xlim(-5, num_ad_patients + 5)
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight')
    plt.show()
    print(f"Plot saved to {output_path}")

# Generate combined plot
combined_line_plot(
    before_ad,
    after_ad,
    ad_data_first,
    f"Lines Connecting Before and After {di} Infections to {di} Onset Age",
    f"./download/Combined_Lines_Before_After_A_no_scaling.pdf"
)


In [ ]:
# Define age bins for grouping
age_bins = [20, 30, 40, 50, 60, 70, 80, 90, 100]
age_labels = [f"{age_bins[i]}-{age_bins[i+1]-1}" for i in range(len(age_bins)-1)]

# Group data into decades for age of first bacterial disease and Alzheimer's diagnosis
combined_age_data_cleaned['age_bin'] = pd.cut(
    combined_age_data_cleaned['age_at_diagnosis'], bins=age_bins, labels=age_labels, right=False
)

In [ ]:
# Adjusting the plots with the preferred styles and applying them to all diseases
diseases = list(disease_dataframes.keys())
output_path = "./download"  # Update to your preferred output directory

# KDE plot function with preferred styles
def kde_plot(data, x_col, title, xlabel, output_filename):
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data[x_col], fill=True, common_norm=False)
    plt.title(title, fontsize=14)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.legend(title='Legend', fontsize=12, frameon=False, loc='upper right', bbox_to_anchor=(1.2, 0.6))
    plt.grid(False)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    plt.tight_layout()
    output_file = f"{output_path}/{output_filename}.png"
    plt.savefig(output_file, bbox_inches='tight')
    plt.show()
    print(f"Plot saved to {output_file}")

# # Plot KDEs for each disease for Alzheimer's patients
# for disease in diseases:
#     if disease == "alzheimer":
#         continue  # Skip Alzheimer's as it will be handled separately

#     disease_data = combined_age_data_cleaned[
#         (combined_age_data_cleaned['disease_name'] == disease) &
#         (combined_age_data_cleaned['person_id'].isin(ad_with_infections['person_id']))
#     ]

#     title = f'KDE Plot: Age of First {disease.capitalize()} Disease (Alzheimer Patients)'
#     xlabel = 'Age of First Disease'
#     kde_plot(disease_data, 'age_at_diagnosis', title, xlabel, f"alzheimer_first_{disease}")

# # Plot KDEs for age at Alzheimer's diagnosis for each disease's patients
# for disease in diseases:
#     disease_data = combined_age_data_cleaned[
#         (combined_age_data_cleaned['disease_name'] == 'alzheimer') &
#         (combined_age_data_cleaned['person_id'].isin(
#             combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == disease]['person_id']
#         ))
#     ]

#     title = f'KDE Plot: Age at Alzheimer Diagnosis ({disease.capitalize()} Patients)'
#     xlabel = 'Age at Alzheimer Diagnosis'
#     kde_plot(disease_data, 'age_at_diagnosis', title, xlabel, f"{disease}_first_alzheimer")

# Combined KDE plot for all diseases (Alzheimer patients and first bacterial disease)
plt.figure(figsize=(12, 8))
for disease in diseases:
    if disease == "alzheimer":
        continue  # Skip Alzheimer's as it's not a bacterial disease
    disease_data = combined_age_data_cleaned[
        (combined_age_data_cleaned['disease_name'] == disease) &
        (combined_age_data_cleaned['person_id'].isin(ad_with_infections['person_id']))
    ]
    sns.kdeplot(disease_data['age_at_diagnosis'], fill=True, label=f"{disease.capitalize()}")

plt.title('Combined KDE Plot: Age of First Bacterial Disease for Alzheimer Patients', fontsize=14)
plt.xlabel('Age of First Disease', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.legend(title='Legend', fontsize=12, frameon=False, loc='upper right', bbox_to_anchor=(1.2, 0.6))
plt.grid(False)
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.tight_layout()
combined_plot_path = f"{output_path}/combined_bacterial_first_disease.png"
plt.savefig(combined_plot_path, bbox_inches='tight')
plt.show()
print(f"Combined bacterial disease plot saved to {combined_plot_path}")

# Combined KDE plot for all diseases and age at Alzheimer's diagnosis
plt.figure(figsize=(12, 8))
for disease in diseases:
    if disease == "alzheimer":
        continue
    disease_data = combined_age_data_cleaned[
        (combined_age_data_cleaned['disease_name'] == 'alzheimer') &
        (combined_age_data_cleaned['person_id'].isin(
            combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == disease]['person_id']
        ))
    ]
    sns.kdeplot(disease_data['age_at_diagnosis'], fill=True, label=f"{disease.capitalize()}")

plt.title('Combined KDE Plot: Age at Alzheimer Diagnosis for Bacterial Disease Patients', fontsize=14)
plt.xlabel('Age at Alzheimer Diagnosis', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.legend(title='Legend', fontsize=12, frameon=False, loc='upper right', bbox_to_anchor=(1.2, 0.6))
plt.grid(False)
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.tight_layout()
# combined_plot_path_ad = f"{output_path}/combined_alzheimer_diagnosis.png"
# plt.savefig(combined_plot_path_ad, bbox_inches='tight')
plt.show()
# print(f"Combined Alzheimer's diagnosis plot saved to {combined_plot_path_ad}")


In [ ]:

# Ensure minimum age for the first disease is calculated for each person
min_age_data = combined_age_data_cleaned.groupby(['person_id', 'disease_name'])['age_at_diagnosis'].min().reset_index()
min_age_data.rename(columns={'age_at_diagnosis': 'min_age_at_diagnosis'}, inplace=True)

# Process the age of first disease and first dementia
first_disease_dementia_data = min_age_data.pivot(index='person_id', columns='disease_name', values='min_age_at_diagnosis')

# Iterate over diseases to plot the graphs
for disease in diseases:
    if disease == 'alzheimer': 
        continue  # Skip Alzheimer for the first disease plot
    
    if disease not in first_disease_dementia_data.columns or 'alzheimer' not in first_disease_dementia_data.columns:
        print(f"Skipping {disease} as it doesn't have sufficient data.")
        continue

    # Extract data for the disease and Alzheimer
    disease_data = first_disease_dementia_data[[disease, 'alzheimer']].dropna()
    disease_data.columns = ['age_first_disease', 'age_first_dementia']

    # Plotting the KDEs
    plt.figure(figsize=(10, 6))
    sns.kdeplot(disease_data['age_first_disease'], fill=True, color='blue', label='Age of First Disease')
    sns.kdeplot(disease_data['age_first_dementia'], fill=True, color='red', label='Age of First Dementia')
    plt.title(f'KDE Plot: Age of First {disease.capitalize()} Disease and Dementia', fontsize=14)
    plt.xlabel('Age', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.legend(title='Legend', fontsize=12, frameon=False, loc='upper right', bbox_to_anchor=(1.2, 0.6))
    plt.grid(False)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    plt.tight_layout()
    plt.show()



## count as categories if they have the infection in general. For example if a patient have in his life ecoli then count 1, if he had 4 times ecoli and one Pseudomonas then we count it as 2

In [ ]:
# Step 1: Add a column for total infection count
# Specify infection-related diseases
infection_columns = ["gram_negative_count", "staphylococcus_count", "pseudomonas_count", 
                     "streptococcus_count", "ecoli_count", "gram_positive_count"]

# Calculate the total infections for each person
combined_phecode_counts["total_infections"] = combined_phecode_counts[infection_columns].sum(axis=1)

# Correct the apply call to use the total_infections column directly
def classify_infection_level(total_infections):
    if total_infections == 0:
        return "No infection"
    elif total_infections == 1:
        return "1 infection"
    else:
        return "2+ infections"

# Apply the classification function to the total_infections column
combined_phecode_counts["infection_level"] = combined_phecode_counts["total_infections"].apply(classify_infection_level)


# Step 3: Filter for individuals with Alzheimer's Disease
ad_with_infections = combined_phecode_counts[combined_phecode_counts["alzheimer_count"] > 0]

# Step 4: Count individuals by infection level
infection_summary = ad_with_infections["infection_level"].value_counts()
infection_summary.columns = ["Infection Level", "Count"]

# Step 5: Display the results
print(infection_summary)


In [ ]:
# Classify infection levels
ad_combined_counts["infection_level"] = ad_combined_counts["total_infections"].apply(classify_infection_level)

# Count infection levels
infection_summary_ad = ad_combined_counts["infection_level"].value_counts()
print("Infection level summary for Alzheimer's patients:")
print(infection_summary_ad)


In [ ]:
# Step 1: Count total occurrences for each infection
infection_counts = combined_age_data_filtered['disease_name'].value_counts()

# Step 2: Filter individuals with and without Alzheimer's diagnosis
# Identify individuals with Alzheimer's
alzheimers_patients = combined_age_data_filtered.loc[
    combined_age_data_filtered['disease_name'].str.contains('alzheimer', case=False), 'person_id'
].unique()
def classify_infections(df):
    infection_counts_per_person = df.groupby('person_id')['disease_name'].nunique()
    ad_infection = infection_counts_per_person[infection_counts_per_person == 1].count()
    one_infections = infection_counts_per_person[infection_counts_per_person == 2].count()
    two_or_more_infections = infection_counts_per_person[infection_counts_per_person >= 3].count()
    return ad_infection, one_infections,two_or_more_infections

# Split data into with and without Alzheimer's
with_alzheimers = combined_age_data_filtered[combined_age_data_filtered['person_id'].isin(alzheimers_patients)]
without_alzheimers = combined_age_data_filtered[~combined_age_data_filtered['person_id'].isin(alzheimers_patients)]


# Classify infections for individuals with Alzheimer's
with_alz_ad_infection, with_alz_one_infections,with_alz_two_or_more_infections = classify_infections(with_alzheimers)

# Results
print("Total infections:")
print(infection_counts)

print(f"\nWith {di}:")
print(f"  Individuals with {di}: {with_alz_ad_infection}")
print(f"  Individuals with one infections in category and {di}: {with_alz_one_infections}")
print(f"  Individuals with two or more infections and {di}: {with_alz_two_or_more_infections}")

# Step 3: Count infections for each person and classify by infection count
def classify_infections(df):
    infection_counts_per_person = df.groupby('person_id')['disease_name'].nunique()
    one_infection = infection_counts_per_person[infection_counts_per_person == 1].count()
    two_or_more_infections = infection_counts_per_person[infection_counts_per_person >= 2].count()
    
    return one_infection, two_or_more_infections
# Classify infections for individuals without Alzheimer's
without_alz_one_infection, without_alz_two_or_more = classify_infections(without_alzheimers)

print(f"\nWithout {di}:")
print(f"  Individuals with one infection: {without_alz_one_infection}")
print(f"  Individuals with two or more infections: {without_alz_two_or_more}")



In [ ]:
# Merge the infection levels into the dataset
infection_levels = combined_phecode_counts[["person_id", "infection_level"]]
ad_first_diagnosis = ad_first_diagnosis.merge(infection_levels, on="person_id", how="left")

# Categorize infection groups for consistency
ad_first_diagnosis["infection_group"] = ad_first_diagnosis["infection_level"]
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import matplotlib.pyplot as plt
import pandas as pd

# Step 1: Count bacterial infections for each person
bacteria_list = ['staphylococcus', 'ecoli', 'pseudomonas', 'gram_negative', 'gram_positive', 'streptococcus']
bacteria_counts = (
    combined_age_data_cleaned[
        combined_age_data_cleaned['disease_name'].isin(bacteria_list)
    ]
    .groupby('person_id')['disease_name']
    .nunique()
    .reset_index()
    .rename(columns={'disease_name': 'bacteria_infections'})
)

# Step 2: Identify the first AD diagnosis for each person
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_first_diagnosis = (
    ad_data.groupby('person_id')['age_at_diagnosis']
    .min()
    .reset_index()
    .rename(columns={'age_at_diagnosis': 'first_ad_age'})
)

# Step 3: Merge bacterial infection counts with first AD diagnosis age
ad_first_diagnosis = ad_first_diagnosis.merge(bacteria_counts, on='person_id', how='left').fillna(0)

# Step 4: Filter by minimum age at first AD diagnosis
min_first_ad_age = 0
ad_first_diagnosis = ad_first_diagnosis[ad_first_diagnosis['first_ad_age'] > min_first_ad_age]

# Step 5: Categorize into infection groups
ad_first_diagnosis['infection_group'] = ad_first_diagnosis['bacteria_infections'].apply(
    lambda x: '0' if x == 0 else ('1' if x == 1 else ('2' if x == 2 else "3+"))
)

# Step 6: Perform Kaplan-Meier Analysis
kmf = KaplanMeierFitter()
plt.figure(figsize=(12, 8))
colors = {'0': 'blue', '1': 'purple', '2': 'green', '3+': 'red'}

# Add general dementia line
kmf.fit(
    durations=ad_first_diagnosis['first_ad_age'],
    event_observed=[1] * len(ad_first_diagnosis),
    label="Dementia (All Patients)"
)
kmf.plot_survival_function(color='black', linestyle='--', linewidth=2, ci_show=False)

# Prepare Kaplan-Meier plot and survival data
group_survival_data = {}
group_labels = {
    "0": "0 Infections",
    "1": "1 Infection",
    "2": "2 Infections",
    "3+": "3+ Infections"
}
for group in ['0', '1', '2', '3+']:
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
    group_survival_data[group] = group_data['first_ad_age']
    kmf.fit(
        durations=group_data['first_ad_age'], 
        event_observed=[1] * len(group_data),  # Assume all events observed
        label=f"{group_labels[group]}, n={len(group_data)}"
    )
    kmf.plot_survival_function(color=colors[group], ci_show=False)

# Log-Rank Test
log_rank_results = []
groups = ['0', '1', '2', '3+']
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        if group_1 in group_survival_data and group_2 in group_survival_data:
            result = logrank_test(
                group_survival_data[group_1],
                group_survival_data[group_2]
            )
            log_rank_results.append((group_1, group_2, result.p_value))

log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)

# Enhance the plot
plt.title('Kaplan-Meier Plot for First AD Diagnosis Age by Infection Group', fontsize=16)
plt.xlabel('Age at First AD Diagnosis', fontsize=14)
plt.ylabel('Survival Probability', fontsize=14)
plt.legend(title='Infection Group', fontsize=12, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Save the plot
output_path = "./download/first_ad_diagnosis_by_infection_group_with_logrank.pdf"
plt.savefig(output_path, bbox_inches="tight", dpi=300)
plt.show()

print("Kaplan-Meier plot saved at:", output_path)

# Display log-rank test results
print("\nLog-Rank Test Results:")
print(log_rank_comparisons)


In [ ]:
from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
import pandas as pd

# Step 1: Count bacterial infections for each person
bacteria_counts = (
    combined_age_data_cleaned[
        combined_age_data_cleaned['disease_name'].isin(['staphylococcus', 'ecoli', 'pseudomonas', 
                                                        'gram_negative', 'gram_positive', 'streptococcus'])
    ]
    .groupby('person_id')['disease_name']
    .nunique()
    .reset_index()
    .rename(columns={'disease_name': 'bacteria_infections'})
)

# Step 2: Identify the first AD diagnosis for each person
ad_data = combined_age_data_cleaned[combined_age_data_cleaned['disease_name'] == 'alzheimer']
ad_first_diagnosis = (
    ad_data.groupby('person_id')['age_at_diagnosis']
    .min()
    .reset_index()
    .rename(columns={'age_at_diagnosis': 'first_ad_age'})
)

# Step 3: Merge bacterial infection counts with first AD diagnosis age
ad_first_diagnosis = ad_first_diagnosis.merge(bacteria_counts, on='person_id', how='left').fillna(0)

# Step 4: Categorize into infection groups
ad_first_diagnosis['infection_group'] = ad_first_diagnosis['bacteria_infections'].apply(
    lambda x: '0' if x == 0 else ('1' if x == 1 else ('2' if x == 2 else '3+'))
)

# Prepare Kaplan-Meier Analysis
kmf = KaplanMeierFitter()
plt.figure(figsize=(12, 8))
colors = {'0': 'blue', '1': 'purple', '2': 'green', '3+': 'red'}

# Prepare log-rank test statistics
groups = ['0', '1', '2', '3+']
group_survival_data = {group: ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]['first_ad_age'] for group in groups}
log_rank_results = []

for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1, group_2 = groups[i], groups[j]
        result = logrank_test(
            group_survival_data[group_1],
            group_survival_data[group_2]
        )
        log_rank_results.append((group_1, group_2, result.p_value))

# Save log-rank comparisons as a TSV
log_rank_comparisons = pd.DataFrame(
    log_rank_results,
    columns=["Group_1", "Group_2", "P_Value"]
)
output_tsv_path = "./download/log_rank_comparisons.tsv"
log_rank_comparisons.to_csv(output_tsv_path, sep='\t', index=False)
print(f"Log-rank comparison TSV file saved to: {output_tsv_path}")

# Create custom legend labels with p-values
group_labels = {}
for group in groups:
    if group == '0':
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])})"
    else:
        p_val = min([res[2] for res in log_rank_results if res[0] == '0' and res[1] == group])
        group_labels[group] = f"{group} Infections (n={len(ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group])}, p={p_val:.3e})"

# Plot Kaplan-Meier curves with updated legend
for group in groups:
    group_data = ad_first_diagnosis[ad_first_diagnosis['infection_group'] == group]
    kmf.fit(
        durations=group_data['first_ad_age'],
        event_observed=[1] * len(group_data),
        label=group_labels[group]
    )
    kmf.plot_survival_function(color=colors[group], ci_show=False)

# Enhance the plot
plt.title(f'Kaplan-Meier plot for first {di} diagnosis age by the number of bacterial infections', fontsize=16)
plt.xlabel(f'Age at first {di} diagnosis', fontsize=14)
plt.ylabel(f'Survival probability', fontsize=14)
plt.legend(title='Number of bacteria infections', fontsize=12, frameon=False)
plt.grid(False)

# Customize plot aesthetics
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)

# Save the plot
survival_plot_path = f"./download/first_ad_diagnosis_by_infection_group_with_logrank_{di}.pdf"
plt.savefig(survival_plot_path, bbox_inches='tight')

# Show the plot
plt.tight_layout()
plt.show()

print(f"Survival plot saved to {survival_plot_path}")


# Code from the old file for age difference

In [ ]:
dementia_and_no_disease = {}
no_alzheimer_and_no_disease = {}
alzheimer_and_disease = {}
no_alzheimer_and_disease = {}

# Iterate over each disease to populate the dictionaries
for disease in disease_person_ids.keys():
    if disease != 'alzheimer':
        # Patients with Dementia and no other disease
        dementia_and_no_disease[disease] = older_people[(older_people['alzheimer'] == 1) & (older_people[disease] == 0)].shape[0]

        # Patients with no Alzheimer's and no other disease
        no_alzheimer_and_no_disease[disease] = older_people[(older_people['alzheimer'] == 0) & (older_people[disease] == 0)].shape[0]

        # Patients with Alzheimer's and the disease
        alzheimer_and_disease[disease] = older_people[(older_people['alzheimer'] == 1) & (older_people[disease] == 1)].shape[0]

        # Patients with no Alzheimer's and the disease
        no_alzheimer_and_disease[disease] = older_people[(older_people['alzheimer'] == 0) & (older_people[disease] == 1)].shape[0]

# Print the comparison counts
print("\nNumber of patients with Dementia and no other disease:")
for disease, count in dementia_and_no_disease.items():
    print(f"Dementia and {disease}: {count}")

print("\nNumber of patients with no Dementia and no other disease:")
for disease, count in no_alzheimer_and_no_disease.items():
    print(f"No Alzheimer's and {disease}: {count}")

print("\nNumber of patients with Dementia and each other disease:")
for disease, count in alzheimer_and_disease.items():
    print(f"Alzheimer's and {disease}: {count}")

print("\nNumber of patients with no Dementia and each other disease:")
for disease, count in no_alzheimer_and_disease.items():
    print(f"No Alzheimer's and {disease}: {count}")

# Total number of patients with Alzheimer's
total_alzheimer = older_people['alzheimer'].sum()
print(f"\nTotal number of patients with Alzheimer's: {total_alzheimer}")

# Total number of patients overall
sum_total = len(older_people['person_id'])
print(f"Total number of patients: {sum_total}")

In [ ]:
# Ensure condition_start_datetime is in date format
for df in [ecoli_aggregated_df, gram_negative_aggregated_df, retrovirus_aggregated_df, epilepsy_aggregated_df,
           streptococcus_aggregated_df, pseudomonas_aggregated_df, alzheimer_aggregated_df, postconcussion_aggregated_df,
           staphylococcus_aggregated_df, guillain_barre_aggregated_df, gram_positive_aggregated_df]:
    df['condition_start_datetime'] = pd.to_datetime(df['condition_start_datetime']).dt.date

# Function to calculate age_onset while handling NaN values
def calculate_age_onset(aggregated_df, older_people):
    # Merge with df_common to get the date_of_birth
    merged_df = aggregated_df.merge(older_people[['person_id', 'date_of_birth']], on='person_id', how='left')

    # Ensure date_of_birth is in date format and extract date only
    merged_df['date_of_birth'] = pd.to_datetime(merged_df['date_of_birth']).dt.date

    # Calculate age_onset and handle NaN values
    merged_df['age_onset'] = merged_df.apply(
        lambda row: (row['condition_start_datetime'] - row['date_of_birth']).days / 365.25 if pd.notnull(row['condition_start_datetime']) and pd.notnull(row['date_of_birth']) else np.nan,
        axis=1
    )

    # Round the age_onset to 2 decimal places
    merged_df['age_onset'] = merged_df['age_onset'].round(1)

    return merged_df[['person_id', 'condition_start_datetime', 'source_concept_name', 'source_concept_code', 'source_vocabulary', 'age_onset']]

# Dictionary to hold the aggregated DataFrames with age_onset calculated
aggregated_dfs = {
    'ecoli': calculate_age_onset(ecoli_aggregated_df, older_people),
    'gram_negative': calculate_age_onset(gram_negative_aggregated_df, older_people),
    'retrovirus': calculate_age_onset(retrovirus_aggregated_df, older_people),
    'epilepsy': calculate_age_onset(epilepsy_aggregated_df, older_people),
    'streptococcus': calculate_age_onset(streptococcus_aggregated_df, older_people),
    'pseudomonas': calculate_age_onset(pseudomonas_aggregated_df, older_people),
    'alzheimer': calculate_age_onset(alzheimer_aggregated_df, older_people),
    'postconcussion': calculate_age_onset(postconcussion_aggregated_df, older_people),
    'staphylococcus': calculate_age_onset(staphylococcus_aggregated_df, older_people),
    'gram_positive': calculate_age_onset(gram_positive_aggregated_df, older_people)
}

# Display a sample of the aggregated DataFrame for Alzheimer's
aggregated_dfs['alzheimer']


In [ ]:
# Function to calculate age_onset and return rows with NaN values for debugging
def calculate_age_onset_with_debugging(aggregated_df, older_people):
    # Merge with df_common to get the date_of_birth
    merged_df = aggregated_df.merge(older_people[['person_id', 'date_of_birth']], on='person_id', how='left')

    # Ensure date_of_birth is in date format and extract date only
    merged_df['date_of_birth'] = pd.to_datetime(merged_df['date_of_birth']).dt.date

    # Calculate age_onset and handle NaN values
    merged_df['age_onset'] = merged_df.apply(
        lambda row: (row['condition_start_datetime'] - row['date_of_birth']).days / 365.25 if pd.notnull(row['condition_start_datetime']) and pd.notnull(row['date_of_birth']) else np.nan,
        axis=1
    )

    # Identify rows where age_onset is NaN
    nan_rows = merged_df[merged_df['age_onset'].isna()]

    # Print the problematic rows for debugging
    print("Rows with NaN in age_onset:")
    print(nan_rows[['person_id', 'condition_start_datetime', 'date_of_birth', 'source_concept_name', 'source_concept_code', 'source_vocabulary']])

    # Return the dataframe including age_onset
    return merged_df[['person_id', 'condition_start_datetime', 'source_concept_name', 'source_concept_code', 'source_vocabulary', 'age_onset']]

# Example: Check for NaN issues in Alzheimer's data
alzheimer_debug_df = calculate_age_onset_with_debugging(alzheimer_aggregated_df, older_people)


In [ ]:
# Function to remove rows where age_onset is NaN
def remove_nan_age_onset(aggregated_df):
    return aggregated_df.dropna(subset=['age_onset'])

# Apply the removal function to each dataframe in the dictionary
aggregated_dfs_cleaned = {disease: remove_nan_age_onset(df) for disease, df in aggregated_dfs.items()}

# Display a sample of the cleaned aggregated DataFrame for Alzheimer's
aggregated_dfs_cleaned['alzheimer']


In [ ]:
import matplotlib.pyplot as plt

# Function to calculate age differences between two diseases
def calculate_age_difference(disease1, disease2, aggregated_dfs_cleaned):
    df1 = aggregated_dfs_cleaned[disease1]
    df2 = aggregated_dfs_cleaned[disease2]

    # Merge on person_id to find patients with both diseases
    merged_df = df1.merge(df2, on='person_id', suffixes=(f'_{disease1}', f'_{disease2}'))

    # Calculate the age difference
    merged_df['age_difference'] = merged_df[f'age_onset_{disease1}'] - merged_df[f'age_onset_{disease2}']

    return merged_df['age_difference']

# Example: Compare Alzheimer's with Retrovirus
age_diff_alzheimers_pseudomonas = calculate_age_difference('alzheimer', 'pseudomonas', aggregated_dfs_cleaned)

# Display the first few differences for inspection
print(age_diff_alzheimers_pseudomonas.head())


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# Function to calculate age differences between two diseases
def calculate_age_difference(disease1, disease2, aggregated_dfs_cleaned):
    df1 = aggregated_dfs_cleaned[disease1]
    df2 = aggregated_dfs_cleaned[disease2]

    # Merge on person_id to find patients with both diseases
    merged_df = df1.merge(df2, on='person_id', suffixes=(f'_{disease1}', f'_{disease2}'))

    # Calculate the age difference
    merged_df['age_difference'] = merged_df[f'age_onset_{disease1}'] - merged_df[f'age_onset_{disease2}']

    return merged_df['age_difference'], merged_df[f'age_onset_{disease1}']

# Function to plot histogram of age differences with colors based on Alzheimer's age onset groups
def plot_age_difference_histogram(age_differences, age_onset_alzheimer, disease1, disease2):
    plt.figure(figsize=(10, 6))

    # Define 20 age groups and corresponding colors
    bins = np.linspace(0, 100, 21)  # 20 bins (0-5, 6-10, ..., 96-100)
    colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(bins)-1))

    # Assign each patient to an age group based on Alzheimer's onset age
    age_groups = pd.cut(age_onset_alzheimer, bins=bins, labels=False, include_lowest=True)

    # Define the histogram bins to use
    hist_bins = np.linspace(min(age_differences), max(age_differences), 20)  # 40 bins for age differences

    # Create an array to store the cumulative values for stacking
    cumulative_values = np.zeros(len(hist_bins) - 1)

    # Initialize lists to store legend handles and labels
    legend_handles = []
    legend_labels = []

    # Calculate total count
    total_count = len(age_differences)
    count_display = "<20" if total_count < 20 else str(total_count)

    # Plot the histogram in layers, stacking each group
    for i in reversed(range(len(bins) - 1)):
        hist_values, _ = np.histogram(
            age_differences[age_groups == i],
            bins=hist_bins
        )
        if np.any(hist_values):  # Only plot and add to legend if the group has data
            # Convert counts to percentages
            hist_values_percentage = (hist_values / total_count) * 100
            bar = plt.bar(
                hist_bins[:-1],
                hist_values_percentage,
                width=np.diff(hist_bins),
                color=colors[i],
                edgecolor='black',
                bottom=cumulative_values
            )
            # Update the cumulative values to stack the next group on top
            cumulative_values += hist_values_percentage
            # Add to legend
            legend_handles.append(bar[0])
            legend_labels.append(f'{bins[i]:.0f}-{bins[i+1]:.0f} years')

    # Add a vertical line at x=0
    plt.axvline(0, color='grey', linestyle='--', linewidth=2)

    # Add labels and title with sample count
    plt.xlabel('Age difference (years)')
    plt.ylabel('Percentage of patients (%)')
    plt.title(f'Age difference between onsets of {ad} and {disease2.capitalize()} ({count_display} patients)')
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_linewidth(2)
    plt.gca().spines['bottom'].set_linewidth(2)
    plt.gca().spines['left'].set_color('black')
    plt.gca().spines['bottom'].set_color('black')
    plt.grid(False)
    plt.tick_params(axis='x', colors='grey')
    plt.tick_params(axis='y', colors='grey')

    # Add legend with bbox to the right, flipping the order of labels in the legend
    plt.legend(legend_handles[::-1], legend_labels[::-1], title=f"{ad} onset age groups", frameon=False, bbox_to_anchor=(1, 1), loc='upper left')
    plt.tight_layout()

    # Save the plot as a PDF in the ./download/ folder
    os.makedirs('./download/', exist_ok=True)
    file_name = f'./download/age_difference_{ad}_vs_{disease2}_percentage.pdf'
    plt.savefig(file_name, format='pdf', dpi=300)

    # Show the plot
    plt.show()

# List of diseases to compare with Alzheimer's
diseases = ['retrovirus','gram_negative', 'staphylococcus', 'postconcussion', 'pseudomonas', 'streptococcus', 'epilepsy', 'ecoli', 'gram_positive']

# Loop over the diseases and generate histograms
for disease in diseases:
    age_diff, age_onset_alzheimer = calculate_age_difference('alzheimer', disease, aggregated_dfs_cleaned)
    plot_age_difference_histogram(age_diff, age_onset_alzheimer, 'Alzheimer', disease)


In [ ]:
def plot_age_difference_bell_curve_fixed(age_differences, age_onset_alzheimer, disease1, disease2):
    plt.figure(figsize=(10, 6))

    # Define Alzheimer's onset age bins starting from 20 to 90, with 90+ as the last group
    age_onset_bins = [20, 30, 40, 50, 60, 70, 80, 90, float('inf')]
    colors = plt.get_cmap('tab20')(np.linspace(0, 1, len(age_onset_bins) - 1))

    # Assign Alzheimer's onset age groups
    age_groups = pd.cut(age_onset_alzheimer, bins=age_onset_bins, labels=False, include_lowest=True)

    # Define a consistent range for all KDEs
    global_min = age_differences.min() - 5  # Extend range slightly for aesthetics
    global_max = age_differences.max() + 5
    x_range = np.linspace(global_min, global_max, 500)

    # Plot a KDE curve for each age group
    legend_handles = []
    legend_labels = []

    for i in range(len(age_onset_bins) - 1):
        # Select age differences for the current Alzheimer's onset age group
        group_data = age_differences[age_groups == i]
        if len(group_data) > 1:  # KDE requires at least two points
            kde = gaussian_kde(group_data)
            y_values = kde(x_range)
            plt.plot(x_range, y_values, color=colors[i], linewidth=2)

            # Add to legend
            if age_onset_bins[i+1] == float('inf'):
                legend_labels.append('90+ years')
            else:
                legend_labels.append(f'{age_onset_bins[i]:.0f}-{age_onset_bins[i+1]:.0f} years')
            legend_handles.append(plt.Line2D([0], [0], color=colors[i], linewidth=2))

    # Add vertical line at x=0
    plt.axvline(0, color='grey', linestyle='--', linewidth=2)

    # Add labels, title, and formatting
    plt.xlabel('Age difference (years)')
    plt.ylabel('Density')
    plt.title(f'Age difference KDE between onsets of {disease1} and {disease2}')
    plt.legend(legend_handles, legend_labels, title=f"{disease1} onset age groups", frameon=False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_linewidth(2)
    plt.gca().spines['bottom'].set_linewidth(2)
    plt.gca().spines['left'].set_color('black')
    plt.gca().spines['bottom'].set_color('black')
    plt.grid(False)
    plt.tick_params(axis='x', colors='grey')
    plt.tick_params(axis='y', colors='grey')
    plt.tight_layout()

    # Save the plot as PDF
    os.makedirs('./download/', exist_ok=True)
    file_name = f'./download/age_difference_{disease1}_vs_{disease2}_bell_curve_fixed.pdf'
    plt.savefig(file_name, format='pdf', dpi=300)

    # Show the plot
    plt.show()

# Call the function with the updated range
plot_age_difference_bell_curve_fixed(
    age_differences,
    age_onset_alzheimer,
    'Alzheimer',
    'Bacterial Groups'
)


In [ ]:
def plot_age_difference_bell_curve_by_group(age_differences, age_onset_alzheimer, disease1, group_name):
    plt.figure(figsize=(10, 6))

    # Define Alzheimer's onset age bins starting from 20 to 90, with 90+ as the last group
    age_onset_bins = [20, 30, 40, 50, 60, 70, 80, 90, float('inf')]
    colors = plt.get_cmap('tab10')(np.linspace(0, 1, len(age_onset_bins) - 1))

    # Assign Alzheimer's onset age groups
    age_groups = pd.cut(age_onset_alzheimer, bins=age_onset_bins, labels=False, include_lowest=True)

    # Define a consistent range for all KDEs
    global_min = age_differences.min() - 5  # Extend range slightly for aesthetics
    global_max = age_differences.max() + 5
    x_range = np.linspace(global_min, global_max, 500)

    # Plot a KDE curve for each age group
    legend_handles = []
    legend_labels = []

    for i in range(len(age_onset_bins) - 1):
        # Select age differences for the current Alzheimer's onset age group
        group_data = age_differences[age_groups == i]
        if len(group_data) > 1:  # KDE requires at least two points
            kde = gaussian_kde(group_data)
            y_values = kde(x_range)
            plt.plot(x_range, y_values, color=colors[i], linewidth=2)

            # Add to legend
            if age_onset_bins[i+1] == float('inf'):
                legend_labels.append('90+ years')
            else:
                legend_labels.append(f'{age_onset_bins[i]:.0f}-{age_onset_bins[i+1]:.0f} years')
            legend_handles.append(plt.Line2D([0], [0], color=colors[i], linewidth=2))

    # Add vertical line at x=0
    plt.axvline(0, color='grey', linestyle='--', linewidth=2)

    # Add labels, title, and formatting
    plt.xlabel('Age difference (years)')
    plt.ylabel('Density')
    plt.title(f'Age difference KDE for {group_name} and {disease1}')
    plt.legend(legend_handles, legend_labels, title=f"{disease1} onset age groups", frameon=False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_linewidth(2)
    plt.gca().spines['bottom'].set_linewidth(2)
    plt.gca().spines['left'].set_color('black')
    plt.gca().spines['bottom'].set_color('black')
    plt.grid(False)
    plt.tick_params(axis='x', colors='grey')
    plt.tick_params(axis='y', colors='grey')
    plt.tight_layout()

    # Save the plot as PDF
    os.makedirs('./download/', exist_ok=True)
    file_name = f'./download/age_difference_{group_name}_vs_{disease1}_bell_curve.pdf'
    plt.savefig(file_name, format='pdf', dpi=300)

    # Show the plot
    plt.show()

# Gram-negative bacteria (gram_negative, ecoli, pseudomonas)
gram_negative_combined = pd.concat(
    [aggregated_dfs_cleaned['gram_negative'], aggregated_dfs_cleaned['ecoli'], aggregated_dfs_cleaned['pseudomonas']],
    ignore_index=True
)
merged_gram_negative = aggregated_dfs_cleaned['alzheimer'].merge(
    gram_negative_combined,
    on='person_id',
    suffixes=('_alzheimer', '_bacteria')
)
age_diff_gram_negative = merged_gram_negative['age_onset_alzheimer'] - merged_gram_negative['age_onset_bacteria']
plot_age_difference_bell_curve_by_group(
    age_diff_gram_negative,
    merged_gram_negative['age_onset_alzheimer'],
    'Alzheimer',
    'Gram-negative Bacteria'
)

# Gram-positive bacteria (gram_positive, staphylococcus, streptococcus)
gram_positive_combined = pd.concat(
    [aggregated_dfs_cleaned['gram_positive'], aggregated_dfs_cleaned['staphylococcus'], aggregated_dfs_cleaned['streptococcus']],
    ignore_index=True
)
merged_gram_positive = aggregated_dfs_cleaned['alzheimer'].merge(
    gram_positive_combined,
    on='person_id',
    suffixes=('_alzheimer', '_bacteria')
)
age_diff_gram_positive = merged_gram_positive['age_onset_alzheimer'] - merged_gram_positive['age_onset_bacteria']
plot_age_difference_bell_curve_by_group(
    age_diff_gram_positive,
    merged_gram_positive['age_onset_alzheimer'],
    'Alzheimer',
    'Gram-positive Bacteria'
)


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# Create directories for saving output files
os.makedirs('./download/', exist_ok=True)

# Function to plot histograms of age differences
def plot_age_difference_histogram_exact_bins(age_differences, age_onset_alzheimer, group_name, file_prefix):
    plt.figure(figsize=(10, 6))

    # Define one-year bins for the histogram
    min_age_diff = int(age_differences.min()) - 1
    max_age_diff = int(age_differences.max()) + 1
    #     bins = np.arange(min_age_diff, max_age_diff + 1, 1)
    # Define one-year bins for the histogram within the range -20 to 20
    bins = np.arange(-20, 21, 1)  # Bins with 1-year increments

    # Define simplified age groups: <60, 60-70, >70
    age_onset_bins = [0, 60, 70, float('inf')]
    age_onset_labels = ['<60', '60-70', '>70']
    colors = ['#ff9999', '#66b3ff', '#99ff99']

    # Assign Alzheimer's onset age groups
    age_groups = pd.cut(age_onset_alzheimer, bins=age_onset_bins, labels=age_onset_labels, include_lowest=True)

    # Initialize cumulative values for stacking
    cumulative_values = np.zeros(len(bins) - 1)

    # Initialize legend handles and labels
    legend_handles = []
    legend_labels = []

    # Total count of age differences
    total_count = len(age_differences)

    # Plot histogram layers for each age group
    for i, label in enumerate(age_onset_labels):
        hist_values, _ = np.histogram(age_differences[age_groups == label], bins=bins)
        if np.any(hist_values):  # Only plot if the group has data
            hist_values_percentage = (hist_values / total_count) * 100
            bar = plt.bar(
                bins[:-1],
                hist_values_percentage,
                width=np.diff(bins),
                color=colors[i],
                edgecolor='black',
                bottom=cumulative_values
            )
            cumulative_values += hist_values_percentage
            legend_labels.append(label)
            legend_handles.append(bar[0])

    # Add vertical line at x=0
    plt.axvline(0, color='grey', linestyle='--', linewidth=2)

    # Add labels, title, and formatting
    plt.xlabel('Age difference (years)')
    plt.ylabel('Percentage of patients (%)')
    plt.title(f'Age difference between {group_name} onset and Dementia onset')
    plt.legend(legend_handles, legend_labels, title="Dementia onset age groups", frameon=False)
    plt.gca().spines['left'].set_color('black')
    plt.gca().spines['bottom'].set_color('black')
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_linewidth(2)
    plt.gca().spines['bottom'].set_linewidth(2)
    plt.tight_layout()

    # Save the plot as PDF
    file_name = f'./download/{file_prefix}_histogram.pdf'
    plt.savefig(file_name, format='pdf', dpi=300)
    plt.show()
()

# Function to generate bell curve plots
def plot_age_difference_bell_curve(age_differences, age_onset_alzheimer, group_name, file_prefix):
    plt.figure(figsize=(10, 6))

    # Define age bins and range for KDE
    age_onset_bins = [20, 30, 40, 50, 60, 70, 80, 90, float('inf')]
    colors = plt.get_cmap('tab10')(np.linspace(0, 1, len(age_onset_bins) - 1))
    age_groups = pd.cut(age_onset_alzheimer, bins=age_onset_bins, labels=False, include_lowest=True)
    x_range = np.linspace(age_differences.min() - 5, age_differences.max() + 5, 500)

    # Initialize legend handles and labels
    legend_handles = []
    legend_labels = []

    # Plot KDE for each group
    for i in range(len(age_onset_bins) - 1):
        group_data = age_differences[age_groups == i]
        if len(group_data) > 1:
            kde = gaussian_kde(group_data)
            y_values = kde(x_range)
            plt.plot(x_range, y_values, color=colors[i], linewidth=2)
            if age_onset_bins[i+1] == float('inf'):
                legend_labels.append('90+ years')
            else:
                legend_labels.append(f'{age_onset_bins[i]:.0f}-{age_onset_bins[i+1]:.0f} years')
            legend_handles.append(plt.Line2D([0], [0], color=colors[i], linewidth=2))

    # Add vertical line at x=0
    plt.axvline(0, color='grey', linestyle='--', linewidth=2)

    # Add labels, title, and formatting
    plt.xlabel('Age difference (years)')
    plt.ylabel('Density')
    plt.title(f'Bell Curve of Age difference between {group_name} and Dementia onset')
    plt.legend(legend_handles, legend_labels, title="Dementia onset age groups", frameon=False)
    plt.gca().spines['left'].set_color('black')
    plt.gca().spines['bottom'].set_color('black')
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_linewidth(2)
    plt.gca().spines['bottom'].set_linewidth(2)
    plt.tight_layout()

    # Save the plot as PDF
    file_name = f'./download/{file_prefix}_bell_curve.pdf'
    plt.savefig(file_name, format='pdf', dpi=300)
    plt.show()

# Save statistics to TSV files
def save_statistics(age_differences, group_name, file_prefix):
    stats = {
        'Mean Age Difference': [age_differences.mean()],
        'Median Age Difference': [age_differences.median()],
        'Min Age Difference': [age_differences.min()],
        'Max Age Difference': [age_differences.max()]
    }
    stats_df = pd.DataFrame(stats)
    stats_file = f'./download/{file_prefix}_statistics.tsv'
    stats_df.to_csv(stats_file, sep='\t', index=False)
    return stats_file

# Analyze gram-negative bacteria
plot_age_difference_histogram_exact_bins(age_diff_gram_negative, merged_gram_negative['age_onset_alzheimer'], "Gram-negative Bacteria", "gram_negative")
plot_age_difference_bell_curve(age_diff_gram_negative, merged_gram_negative['age_onset_alzheimer'], "Gram-negative Bacteria", "gram_negative")
gram_negative_stats = save_statistics(age_diff_gram_negative, "Gram-negative Bacteria", "gram_negative")

# Analyze gram-positive bacteria
plot_age_difference_histogram_exact_bins(age_diff_gram_positive, merged_gram_positive['age_onset_alzheimer'], "Gram-positive Bacteria", "gram_positive")
plot_age_difference_bell_curve(age_diff_gram_positive, merged_gram_positive['age_onset_alzheimer'], "Gram-positive Bacteria", "gram_positive")
gram_positive_stats = save_statistics(age_diff_gram_positive, "Gram-positive Bacteria", "gram_positive")

# Display paths to saved files
gram_negative_stats, gram_positive_stats


In [ ]:
# Ensure proper sorting of age groups in the desired order
age_group_order = ['<60', '60-70', '>70', 'Overall']

# Convert Age Group to a categorical type with the specified order
enrichment_results_combined_df['Age Group'] = pd.Categorical(
    enrichment_results_combined_df['Age Group'],
    categories=age_group_order,
    ordered=True
)

# Remove the 'Overall' group and sort by Disease and then Odds Ratio
filtered_df = enrichment_results_combined_df[enrichment_results_combined_df['Age Group'] != 'Overall']
filtered_df_sorted = filtered_df.sort_values(by=['Disease', 'Odds Ratio'], ascending=[True, False])

# Save the sorted results to a TSV file
output_file_path = './download/alzheimer_enrichment_analysis_sorted_disease_odds_ratio.tsv'
filtered_df_sorted.to_csv(output_file_path, sep='\t', index=False)

# Visualize the sorted results with a forest plot
plt.figure(figsize=(12, 14))

# Calculate 95% confidence intervals
ci_lower = []
ci_upper = []

for i in range(len(filtered_df_sorted)):
    row = filtered_df_sorted.iloc[i]
    disease = row['Disease']
    age_group = row['Age Group']
    odds_ratio = row['Odds Ratio']

    # Use age-specific counts
    a = df[(df['age_group'] == age_group) & (df['person_id'].isin(alzheimer_unique_person_ids))].shape[0]
    b = df[(df['age_group'] == age_group) & (~df['person_id'].isin(alzheimer_unique_person_ids))].shape[0]
    c = len(alzheimer_unique_person_ids) - a
    d = len(older_people) - len(alzheimer_unique_person_ids) - b

    # Calculate confidence intervals
    se = np.sqrt(1 / a + 1 / b + 1 / c + 1 / d)
    log_or = np.log(odds_ratio)
    ci_lower.append(np.exp(log_or - 1.96 * se))
    ci_upper.append(np.exp(log_or + 1.96 * se))

# Create the forest plot
xerr = [
    np.array(filtered_df_sorted['Odds Ratio']) - np.array(ci_lower),
    np.array(ci_upper) - np.array(filtered_df_sorted['Odds Ratio'])
]
colors = ['blue' if p < 0.05 else 'salmon' for p in filtered_df_sorted['P-Value']]
y_positions = range(len(filtered_df_sorted))

plt.errorbar(filtered_df_sorted['Odds Ratio'], y_positions, xerr=xerr, fmt='o', color='gray', ecolor='black', capsize=3)

# Highlight statistically significant points
for i, color in enumerate(colors):
    plt.scatter(filtered_df_sorted['Odds Ratio'].iloc[i], y_positions[i], color=color, zorder=3)

# Add vertical reference line
plt.axvline(x=1, color='red', linestyle='--', zorder=1)

# Customize plot
plt.gca().spines['left'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['left'].set_linewidth(2)
plt.gca().spines['bottom'].set_linewidth(2)

plt.yticks(y_positions, filtered_df_sorted.apply(lambda row: f"{row['Disease']} ({row['Age Group']})", axis=1))
plt.xlabel('Odds Ratio (log scale)')
plt.title('Enrichment Analysis: Alzheimer vs. Selected Diseases\n(Odds Ratio: Likelihood of Co-occurrence)', fontsize=14, weight='bold')
plt.suptitle('Odds Ratio > 1: Disease more common in AD patients | Odds Ratio < 1: Disease less common', fontsize=10, style='italic')
plt.tight_layout()

# Save and display the forest plot
forest_plot_path = './download/alzheimer_enrichment_analysis_forest_plot_sorted_disease_odds_ratio.pdf'
plt.savefig(forest_plot_path, format='pdf')
plt.show()

# Display paths of saved files
output_file_path, forest_plot_path

